In [1]:
import numpy as np
import pandas as pd
from functools import reduce
import time
import sys
import os   
import re
import importlib
import math
import sklearn
from sklearn import preprocessing
from scipy import signal
from scipy.spatial import Delaunay
from scipy.spatial.transform import Rotation
from scipy import stats
from matplotlib import cm
import matplotlib.animation as animation
import seaborn as sns

# import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.transforms as mtrans
import matplotlib.font_manager as fontmanager


%matplotlib widget

## DO THIS TO RUN R CODE WITHIN PYTHON NOTEBOOK
# %pip install r2py
# os.environ['R_HOME'] = "/usr/local/Cellar/r/4.0.4/lib/R"
# import rpy2.robjects as robjects

## Definitions
### Plot specs

In [2]:
viewYZ = (0,0)
viewXZ = (0,90)
viewXY = (90,90)
# viewYX = (270,180)
viewYX = (90, 0)
view3Q = (45,45)
view3Qst = (-45,45)
view3Qsw = (135,45)
view3Qpos = (45,45)
view3Qneg = (135,-135)
colors = {'red':'#B74B4B', 'green':'#8EC15A', 'blue':'#5083D2'}

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Roboto Condensed'

barlow_medium = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Barlow-Medium.ttf')
barlow_bold = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Barlow-Bold.ttf')
barlow_black = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Barlow-Black.ttf')
barlow_condensed_bold = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/BarlowCondensed-Bold.ttf')
barlow_italic = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Barlow-Italic.ttf')
chivo_regular = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Chivo-Regular.ttf')
chivo_italic = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Chivo-Regular.ttf')
chivo_bold = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Chivo-Bold.ttf')
chivo_bold_italic = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Chivo-BoldItalic.ttf')
chivo_black = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Chivo-Black.ttf')
chivo_black_italic = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Chivo-BlackItalic.ttf')
roboto_regular = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Roboto-Regular.ttf')
roboto_italic = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Roboto-Italic.ttf')
roboto_bold = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Roboto-Bold.ttf')
roboto_bold_italic = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Roboto-BoldItalic.ttf')
roboto_black = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Roboto-Black.ttf')
roboto_black_italic = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Roboto-BlackItalic.ttf')
roboto_condensed = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Roboto-Condensed.ttf')
roboto_condensed_italic = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Roboto-CondensedItalic.ttf')
roboto_bold_condensed = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Roboto-BoldCondensed.ttf')
roboto_bold_condensed_italic = fontmanager.FontProperties(fname='/Users/phil/Library/Fonts/Roboto-BoldCondensedItalic.ttf')

### JCS conversion

In [3]:
def convertDataBetweenJCS(destination_matrix, source_matrix, source_df):
    source_from_dest_matrix = np.matmul(source_matrix, destination_matrix.T)
    rotations = Rotation.from_euler('XYZ', source_df.loc[:,['Rx','Ry','Rz']].to_numpy()*-1, degrees=True)
    converted = rotations*Rotation.from_matrix(source_from_dest_matrix)
    eulers = converted.as_euler('XYZ', degrees=True) *-1   
    result_df = source_df.copy()
    result_df.loc[:,['Rx','Ry','Rz']] = eulers
    return result_df

### Python alpha shapes

In [4]:
from collections import defaultdict

def alpha_shape_3D(pos, alpha):
    """
    Compute the alpha shape (concave hull) of a set of 3D points.
    Parameters:
        pos - np.array of shape (n,3) points.
        alpha - alpha value.
    return
        outer surface vertex indices, edge indices, and triangle indices
    """

    tetra = Delaunay(pos)
    # Find radius of the circumsphere.
    # By definition, radius of the sphere fitting inside the tetrahedral needs 
    # to be smaller than alpha value
    # http://mathworld.wolfram.com/Circumsphere.html
    tetrapos = np.take(pos,tetra.vertices,axis=0)
    normsq = np.sum(tetrapos**2,axis=2)[:,:,None]
    ones = np.ones((tetrapos.shape[0],tetrapos.shape[1],1))
    a = np.linalg.det(np.concatenate((tetrapos,ones),axis=2))
    Dx = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[1,2]],ones),axis=2))
    Dy = -np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,2]],ones),axis=2))
    Dz = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,1]],ones),axis=2))
    c = np.linalg.det(np.concatenate((normsq,tetrapos),axis=2))
    r = np.sqrt(Dx**2+Dy**2+Dz**2-4*a*c)/(2*np.abs(a))
    # Find tetrahedrals
    tetras = tetra.vertices[r<alpha,:]
    # triangles
    TriComb = np.array([(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)])
    Triangles = tetras[:,TriComb].reshape(-1,3)
    Triangles = np.sort(Triangles,axis=1)

    # Remove triangles that occurs twice, because they are within shapes
    TrianglesDict = defaultdict(int)
    for tri in Triangles:TrianglesDict[tuple(tri)] += 1
    Triangles=np.array([tri for tri in TrianglesDict if TrianglesDict[tri] ==1])
    #edges
    EdgeComb=np.array([(0, 1), (0, 2), (1, 2)])
    Edges=Triangles[:,EdgeComb].reshape(-1,2)
    Edges=np.sort(Edges,axis=1)
    Edges=np.unique(Edges,axis=0)

    Vertices = np.unique(Edges)
    return Vertices,Edges,Triangles

### Plotting

In [5]:

def format3dPlot(ax, title, xRange, yRange, zRange, view=None, mode='rotations', joint='glenoid',color='grey', tickInterval=30):
    ax.set_title(title)
    ax.patch.set_alpha(0)
    if view:
        ax.view_init(view[0], view[1])
    if mode == 'rotations':
        if joint=='elbow':
            if view: 
                if view ==(0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- radial deviation   Y  ulnar deviation +', size='small', color=colors['green'])
                    ax.set_zlabel('- flexion   Z  extension +', size='small', color=colors['blue'])  
                elif view == (0,90):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- flexion   Z   extension +', size='small', color=colors['blue'])  
                elif view == (90,90):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('+ ulnar deviation   Y   radial deviation -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,0):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('- radial deviation   Y   ulnar deviation +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (45,45):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('- radial deviation   Y   ulnar deviation +', size='small', color=colors['green'])
                    ax.set_zlabel('- flexion   Z   extension +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- supination   X   pronation +', size='small', color=colors['red'])
                    ax.set_ylabel('+ ulnar deviation   Y   radial deviation -', size='small', color=colors['green'])
                    ax.set_zlabel('+ extension   Z   flexion -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                ax.set_ylabel('+ ulnar deviation   Y   radial deviation -', size='small', color=colors['green'])
                ax.set_zlabel('- flexion   Z   extension +', size='small', color=colors['blue'])
        elif joint=='scap':
            if view:
                if view == (0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- caudal pitch   Y   cranial pitch +', size='small', color=colors['green'])
                    ax.set_zlabel('- lateral yaw   Z   medial yaw +', size='small', color=colors['blue'])  
                elif view == (0,90):
                    ax.set_xlabel('+ medial roll   X   lateral roll -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- lateral yaw   Z   medial yaw +', size='small', color=colors['blue'])  
                elif view == (90,0):
                    ax.set_xlabel('+ medial roll   X   lateral roll -', size='small', color=colors['red'])
                    ax.set_ylabel('- caudal pitch   Y   cranial pitch +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,90):
                    ax.set_xlabel('+ medial roll   X   lateral roll -', size='small', color=colors['red'])
                    ax.set_ylabel('+ cranial pitch   Y   caudal pitch -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (45,45):
                    ax.set_xlabel('+ medial roll   X   lateral roll -', size='small', color=colors['red'])
                    ax.set_ylabel('- caudal pitch   Y   cranial pitch +', size='small', color=colors['green'])
                    ax.set_zlabel('- lateral yaw   Z   medial yaw +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- lateral roll   X   medial roll +', size='small', color=colors['red'])
                    ax.set_ylabel('+ cranial pitch   Y   caudal pitch -', size='small', color=colors['green'])
                    ax.set_zlabel('+ medial yaw   Z   lateral yaw -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ medial roll   X   lateral roll -', size='small', color=colors['red'])
                ax.set_ylabel('+ cranial pitch   Y   caudal pitch -', size='small', color=colors['green'])
                ax.set_zlabel('- lateral yaw   Z   medial yaw +', size='small', color=colors['blue'])
        else:
            if view:
                if view == (0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- depression   Y   elevation +', size='small', color=colors['green'])
                    ax.set_zlabel('- retraction   Z   protraction +', size='small', color=colors['blue'])
                elif view == (0,90):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- retraction   Z   protraction +', size='small', color=colors['blue']) 
                elif view == (90,0):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('- depression   Y   elevation +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,90):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('+ elevation   Y   depression -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue'])  
                elif view == (45,45):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('- depression   Y   elevation +', size='small', color=colors['green'])
                    ax.set_zlabel('- retraction   Z   protraction +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- supination   X   pronation +', size='small', color=colors['red'])
                    ax.set_ylabel('+ elevation   Y   depression -', size='small', color=colors['green'])
                    ax.set_zlabel('+ protraction   Z   retraction -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                ax.set_ylabel('+ elevation   Y   depression -', size='small', color=colors['green'])
                ax.set_zlabel('- retraction   Z   protraction +', size='small', color=colors['blue'])
        
    elif mode == 'translations':
        if joint=='elbow':
            if view: 
                if view ==(0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- dorsal   Y  ventral +', size='small', color=colors['green'])
                    ax.set_zlabel('- ulnar   Z  radial +', size='small', color=colors['blue'])  
                elif view == (0,90):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- ulnar   Z   radial +', size='small', color=colors['blue'])  
                elif view == (90,0):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('- dorsal   Y   ventral +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,90):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('+ ventral   Y   dorsal -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (45,45):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('- dorsal   Y   ventral +', size='small', color=colors['green'])
                    ax.set_zlabel('- ulnar   Z   radial +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- distraction   X   compression +', size='small', color=colors['red'])
                    ax.set_ylabel('+ ventral   Y   dorsal -', size='small', color=colors['green'])
                    ax.set_zlabel('+ radial   Z   ulnar -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                ax.set_ylabel('+ ventral   Y   dorsal -', size='small', color=colors['green'])
                ax.set_zlabel('+ radial   Z   ulnar -', size='small', color=colors['blue'])
        elif joint=='scap':
            if view:
                if view == (0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- lateral   Y   medial +', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])  
                elif view == (0,90):
                    ax.set_xlabel('+ cranial   X   caudal -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])  
                elif view == (90,0):
                    ax.set_xlabel('+ cranial   X   caudal -', size='small', color=colors['red'])
                    ax.set_ylabel('- lateral   Y   medial +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,90):
                    ax.set_xlabel('+ cranial   X   caudal -', size='small', color=colors['red'])
                    ax.set_ylabel('+ medial   Y   lateral -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (45,45):
                    ax.set_xlabel('+ cranial   X   caudal -', size='small', color=colors['red'])
                    ax.set_ylabel('- lateral   Y   medial +', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- caudal   X   cranial +', size='small', color=colors['red'])
                    ax.set_ylabel('+ medial   Y   lateral -', size='small', color=colors['green'])
                    ax.set_zlabel('+ dorsal   Z   ventral -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ cranial   X   caudal -', size='small', color=colors['red'])
                ax.set_ylabel('+ medial   Y   lateral -', size='small', color=colors['green'])
                ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])
        else:
            if view:
                if view == (0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- anterior   Y   posterior +', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])
                elif view == (0,90):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue']) 
                elif view == (90,0):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('- anterior   Y   posterior +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,90):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('+ posterior   Y   anterior -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue'])  
                elif view == (45,45):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('- anterior   Y   posterior +', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- distraction   X   compression +', size='small', color=colors['red'])
                    ax.set_ylabel('+ posterior   Y   anterior -', size='small', color=colors['green'])
                    ax.set_zlabel('+ dorsal   Z   ventral -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                ax.set_ylabel('+ posterior   Y   anterior -', size='small', color=colors['green'])
                ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])

    elif mode == 'pca':
        ax.set_xlabel('PC1', size='small')
        ax.set_ylabel('PC2', size='small')
        ax.set_zlabel('PC3', size='small')
    if view:
        if view == (0,0):
            ax.yaxis.labelpad = 10
            ax.zaxis.labelpad = 10
        elif view == (0,90):
            ax.xaxis.labelpad = 10
        elif view == (90,90):
            ax.xaxis.labelpad = 15
            ax.yaxis.labelpad = 3
        elif view == (90,0):
            ax.xaxis.labelpad = 15
            ax.yaxis.labelpad = 3
    ax.tick_params(reset=True,colors=color, labelsize='x-small')
    ax.set_xlim(xRange[0], xRange[1])
    ax.set_ylim(yRange[0], yRange[1])
    ax.set_zlim(zRange[0], zRange[1])  
    xTicks = np.arange(xRange[0],xRange[1]+1, tickInterval) if view and view!=(0,0) else []
    yTicks = np.arange(yRange[0],yRange[1]+1, tickInterval) if view and view!=(0,90) else []
    zTicks = np.arange(zRange[0],zRange[1]+1, tickInterval) if view and view!=(90,90) and view!=(90,0) else []
    xRotation = 45 if view and (view == (0,90) or view == (90,90)) else 0
    yRotation = 45 if view and view == (0,0) else 0
    ax.set_xticks(xTicks)
    ax.set_yticks(yTicks)
    ax.set_zticks(zTicks)
#     ax.set_xticklabels(rotation = xRotation, va ='top', ha='center')
#     ax.set_yticklabels(rotation = yRotation, va ='top', ha='center')
    ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0)) 
    rangeX, rangeY, rangeZ = abs(xRange[0]-xRange[1]), abs(yRange[0]-yRange[1]), abs(zRange[0]-zRange[1])
    minRange = min(rangeX, rangeY, rangeZ)
    xAspect, yAspect, zAspect = rangeX/minRange, rangeY/minRange, rangeZ/minRange
    ax.set_box_aspect([xAspect,yAspect,zAspect])
    ax.w_xaxis.line.set_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_yaxis.line.set_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))

    ax.minorticks_off()
    return ax

def on_move(event):
    if event.inaxes == ax0:
        ax1.view_init(elev=ax0.elev, azim=ax0.azim)
    elif event.inaxes == ax1:
        ax0.view_init(elev=ax1.elev, azim=ax1.azim)
    else:
        return
    fig.canvas.draw_idle()
    

def on_move4(event):
    if event.inaxes == ax0:
        ax1.view_init(elev=ax0.elev, azim=ax0.azim)
        ax2.view_init(elev=ax0.elev, azim=ax0.azim)
        ax3.view_init(elev=ax0.elev, azim=ax0.azim)
    elif event.inaxes == ax1:
        ax0.view_init(elev=ax1.elev, azim=ax1.azim)
        ax2.view_init(elev=ax1.elev, azim=ax1.azim)
        ax3.view_init(elev=ax1.elev, azim=ax1.azim)
    elif event.inaxes == ax2:
        ax0.view_init(elev=ax2.elev, azim=ax2.azim)
        ax1.view_init(elev=ax2.elev, azim=ax2.azim)
        ax3.view_init(elev=ax2.elev, azim=ax2.azim)
    elif event.inaxes == ax3:
        ax0.view_init(elev=ax3.elev, azim=ax3.azim)
        ax1.view_init(elev=ax3.elev, azim=ax3.azim)
        ax2.view_init(elev=ax3.elev, azim=ax3.azim)
    else:
        return
    fig.canvas.draw_idle()
    
def on_move6(event):
    if event.inaxes == ax0:
        ax1.view_init(elev=ax0.elev, azim=ax0.azim)
        ax2.view_init(elev=ax0.elev, azim=ax0.azim)
        ax3.view_init(elev=ax0.elev, azim=ax0.azim)
        ax4.view_init(elev=ax0.elev, azim=ax0.azim)
        ax5.view_init(elev=ax0.elev, azim=ax0.azim)
    elif event.inaxes == ax1:
        ax0.view_init(elev=ax1.elev, azim=ax1.azim)
        ax2.view_init(elev=ax1.elev, azim=ax1.azim)
        ax3.view_init(elev=ax1.elev, azim=ax1.azim)
        ax4.view_init(elev=ax1.elev, azim=ax1.azim)
        ax5.view_init(elev=ax1.elev, azim=ax1.azim)
    elif event.inaxes == ax2:
        ax0.view_init(elev=ax2.elev, azim=ax2.azim)
        ax1.view_init(elev=ax2.elev, azim=ax2.azim)
        ax3.view_init(elev=ax2.elev, azim=ax2.azim)
        ax4.view_init(elev=ax2.elev, azim=ax2.azim)
        ax5.view_init(elev=ax2.elev, azim=ax2.azim)
    elif event.inaxes == ax3:
        ax0.view_init(elev=ax3.elev, azim=ax3.azim)
        ax1.view_init(elev=ax3.elev, azim=ax3.azim)
        ax2.view_init(elev=ax3.elev, azim=ax3.azim)
        ax4.view_init(elev=ax3.elev, azim=ax3.azim)
        ax5.view_init(elev=ax3.elev, azim=ax3.azim)
    elif event.inaxes == ax4:
        ax0.view_init(elev=ax4.elev, azim=ax4.azim)
        ax1.view_init(elev=ax4.elev, azim=ax4.azim)
        ax2.view_init(elev=ax4.elev, azim=ax4.azim)
        ax3.view_init(elev=ax4.elev, azim=ax4.azim)
        ax5.view_init(elev=ax4.elev, azim=ax4.azim)
    elif event.inaxes == ax5:
        ax0.view_init(elev=ax5.elev, azim=ax5.azim)
        ax1.view_init(elev=ax5.elev, azim=ax5.azim)
        ax2.view_init(elev=ax5.elev, azim=ax5.azim)
        ax3.view_init(elev=ax5.elev, azim=ax5.azim)
        ax4.view_init(elev=ax5.elev, azim=ax5.azim)
    else:
        return
    fig.canvas.draw_idle()
    
def addCosGrid(gridAx, xRange, yRange, zRange, interval, xLevels=1, alpha=0, **kwargs):  
    xMin = math.floor(xRange[0])
    xMax = math.ceil(xRange[1])+1
    yMin = math.floor(yRange[0])
    yMax = math.ceil(yRange[1])+1
    zMin = math.floor(zRange[0])
    zMax = math.ceil(zRange[1])+1
    zs= np.arange(zMin, zMax+1, 1)
    ys= np.arange(yMin, yMax, 1)
    xSize = abs(xMin)+abs(xMax)
    ySize = abs(yMin)+abs(yMax)
    zSize = abs(zMin)+abs(zMax)
    alphas = np.ones(zSize)*alpha
    zz, yy = np.meshgrid(zs, ys)
    czz = zz*np.cos(np.radians(yy))
    xMaxMinMax = max((abs(xMin),abs(xMax)))
    if xLevels <2:
        xs = np.zeros(xSize)
    else:
        xs = np.linspace(xMaxMinMax*-1, xMaxMinMax, xLevels)
    for xLevel in list(range(xLevels)):
        xx = np.ones((czz.shape[0],czz.shape[1]))*xs[xLevel]
        gridAx.plot_wireframe(xx, yy, czz, rcount = zSize/interval, ccount=ySize/interval, **kwargs) 

    
def addRegularGrid(gridAx, xRange, yRange, zRange, interval, alpha=0, **kwargs):  
    xMin = math.floor(xRange[0])
    xMax = math.ceil(xRange[1])+1
    yMin = math.floor(yRange[0])
    yMax = math.ceil(yRange[1])+1
    zMin = math.floor(zRange[0])
    zMax = math.ceil(zRange[1])+1
    zs= np.arange(zMin, zMax+1, 1)
    ys= np.arange(yMin, yMax, 1)
    xSize = abs(xMin)+abs(xMax)
    ySize = abs(yMin)+abs(yMax)
    zSize = abs(zMin)+abs(zMax)
    alphas = np.ones(zSize)*alpha
    zz, yy = np.meshgrid(zs, ys)
    czz = zz*np.cos(np.radians(yy))
    xMaxMinMax = max((abs(xMin),abs(xMax)))
    if xLevels <2:
        xs = np.zeros(xSize)
    else:
        xs = np.linspace(xMaxMinMax*-1, xMaxMinMax, xLevels)
    for xLevel in list(range(xLevels)):
        xx = np.ones((czz.shape[0],czz.shape[1]))*xs[xLevel]
        gridAx.plot_wireframe(xx, yy, czz, rcount = zSize/interval, ccount=ySize/interval, **kwargs) 


## Imports
### Import model data

In [2334]:
# phil_matrix = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])[0:3,0:3]
# rob_tegu_matrix = np.array([[0.917443254918, -0.0507813546731, -0.39461263034, 0.0], [-0.0504523728816, 0.968965969705, -0.241990718053, 0.0], [0.394654826496, 0.2419218956, 0.886409253309, 0.0], [2.88657986403e-15, 3.5527136788e-15, 8.881784197e-16, 1.0]])[0:3,0:3]
# rob_opossum_matrix = np.array([[0.988997647945, 0.0952296421902, 0.113203213768, 0.0], [-0.144901479974, 0.777702952339, 0.611703914508, 0.0], [-0.0297861286566, -0.6213770459, 0.782945306754, 0.0], [0.0, -2.22044604925e-16, -8.881784197e-16, 1.0]])[0:3,0:3]
# rob_opossum_matrix1RLflip = np.array([[-0.988997647945, -0.0952296421902, -0.113203213768, 0.0], [0.144901479974, -0.777702952339, -0.611703914508, 0.0], [-0.0297861286566, -0.6213770459, 0.782945306754, 0.0], [0.0, -2.22044604925e-16, -8.881784197e-16, 1.0]])[0:3,0:3]


# df_M_S = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/Tegu_ROM_output_210712_correct.csv',index_col=None)
# df_M_D = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/Opossum_ROM_output_210713_correct.csv',index_col=None)
# df_M_S.loc[:,'species'] = 'Tegu'
# df_M_D.loc[:,'species'] = 'Opossum'
# df_M = pd.concat([df_M_S,df_M_D])
# df_M.columns = ['rx', 'ry', 'rz', 'species']
# df_M['radRy'] = np.radians(df_M['ry'])
# df_M['cosRy'] = np.cos(df_M['radRy'])
# df_M['RzcosRy'] = df_M['cosRy'].multiply(df_M['rz'])
# df_M['crz'] = df_M['RzcosRy']
# df_M_S = df_M.loc[df_M['species']=='Tegu']
# df_M_D = df_M.loc[df_M['species']=='Opossum']

### Data processing functions

In [6]:
aliases = { 'pec_crn':r'(?=.*pec)(?=.*cr).*',
            'pec_int':r'(?=.*pec)(?=.*int).*',
            'pec_cdl':r'(?=.*pec)(?=.*cau|.*c.*d.*l).*',
            'lat_crn':r'(?=.*lat)(?!.*cau|.*c.*d.*l).*',
            'lat_cdl':r'(?=.*lat)(?=.*cau|.*c.*d.*l).*',
            'dac':r'(.*dac|.*acr).*',
            'dsc':r'(.*dsc|.*sc.*p).*',
            'bic':r'(.*bic).*',
            'trs':r'(.*tri).*', 
            'tmj':r'(.*tmj|.*teres).*' }





def hexToRgb(hexStr):
    decR = int(hexStr[1:3],16)
    decG = int(hexStr[3:5],16)
    decB = int(hexStr[5:7],16)
    decTup = (decR, decG, decB)
    return decTup


def generateNewCols(inputDict, mode='jcs'):
    newDict = inputDict.copy() 
    for (key, value) in newDict.items():
#         if mode=='mma':
#             newDict[key] *= 10
        newDict[key].loc[:,['frame']] = newDict[key].index.values.reshape(-1,1)
        newDict[key].loc[:,['animal']] = key.split('_')[1]
        newDict[key].loc[:,['side']] = key.split('_')[-1]
        newDict[key].loc[:,['species']] = species_dict[key.split('_')[1]]
        newDict[key].loc[:,['humeralLength']] = humeral_lengths[key.split('_',1)[1]]
        newDict[key].loc[:,['humeralLengthFactor']] = humeral_scalefactor[key.split('_',1)[1]]
        newDict[key].loc[:,['humeralVolCubeRoot']] = humeral_volumes_cuberoot[key.split('_',1)[1]]
        newDict[key].loc[:,['UID']] = newDict[key]['species']+newDict[key]['animal']+'_frame'+newDict[key]['frame'].astype(str)
        newDict[key] = newDict[key].set_index('UID')
    return newDict

def renameAndDrop(inputDict):
    newDict = inputDict.copy() 
    for (key, value) in newDict.items():
        newDict[key] = newDict[key].rename(columns = lambda x: re.sub('(stern)+.*(scap)+.*data','acromion',x))
        newDict[key] = newDict[key].rename(columns = lambda x: re.sub('(scap)+.*(hum)+.*data','glenoid',x))
        newDict[key] = newDict[key].rename(columns = lambda x: re.sub('(hum)+.*(uln)+.*data','elbow',x))
        newDict[key] = newDict[key].replace(0.0, np.nan)
        newDict[key] = newDict[key].dropna(how='any', axis=0)
        newDict[key] = newDict[key].replace(np.nan, 0.0)
    return newDict


def getMuscleNames(maDict):
    muscle_names = [set(value.columns) for (key,value) in maDict.items()]
    muscle_names = set().union(*muscle_names)
    muscle_names = {name for name in muscle_names}
    return muscle_names

def getAliases(aliasDict, muscleNamesList):
    result = {}
    for (key,value) in aliasDict.items():
        matches = [re.match(value, name, re.IGNORECASE) for name in muscleNamesList]
        matched_names = [name.group(0) for name in matches if name]
        result[key] = matched_names
    all_matches = {item for sublist in result.values() for item in sublist}
    missedA = all_matches.difference(muscleNamesList)
    missedB = muscleNamesList.difference(all_matches)
    missed = missedA.union(missedB)
    if missed:
        print('Failed to match some muscle names; edit regexes and try again: ' +str(missed))
    return result

def reverseAliasMap(mappedAliasesDict, add_split=True):
    result = {}
    for (key, value) in mappedAliasesDict.items():
        for subvalue in value:
            result[subvalue]=key+'_'+subvalue.split('.')[-1] if add_split else key
    return result

def renameMusclesWithMap(maDict, nameMap):
    newDict = maDict.copy() 
    for (key, value) in newDict.items():
        newDict[key] = newDict[key].rename(columns=nameMap)
    return newDict

def findComparisons(phil_df, animal, side):
    filt_df = phil_df.loc[(phil_df['animal']==animal)&(phil_df['side']==side)]
    phil_mmas = [col for col in filt_df.columns if 'Xma' in col and 'scaled' not in col]
    phil_mmas = [col.rsplit('_',1)[0] for col in phil_mmas if pd.notnull(filt_df[col]).all()]
    print(phil_mmas)
    return phil_mmas

def makeMMA3plot(save_dir, phil_df, rob_df, animal, side, phil_muscle, rob_muscle=None, fig_title=None, plot_suffix=''):
    phil_df = phil_df.loc[(phil_df['animal']==animal)&(phil_df['side']==side)].reset_index()
    rob_df = rob_df.loc[phil_df.index]
    fig, ax = plt.subplots(3,1,figsize=(16,8), sharey='all', sharex='all')
    rob_muscle = rob_muscle if rob_muscle else phil_muscle
    plt.suptitle(fig_title if fig_title else phil_muscle)
    ax[0].plot(phil_df[phil_muscle+'_Xma'], linestyle='solid', color='red')
    ax[0].plot(rob_df['model_'+rob_muscle+'_glen_Xma'], linestyle='dotted', color='red')
    ax[1].plot(phil_df[phil_muscle+'_Yma'], linestyle='solid', color='green')
    ax[1].plot(rob_df['model_'+rob_muscle+'_glen_Yma'], linestyle='dotted', color='green')
    ax[2].plot(phil_df[phil_muscle+'_Zma'], linestyle='solid', color='blue')
    ax[2].plot(rob_df['model_'+rob_muscle+'_glen_Zma'], linestyle='dotted', color='blue')
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir,phil_muscle+plot_suffix+'.png'), dpi=150, transparent=True)
    return ax

def makeMMA3plotFromMerged(save_dir, phil_df, animal, side, phil_muscle, rob_muscle=None, fig_title=None, plot_suffix=''):
    df = phil_df.loc[(phil_df['animal']==animal)&(phil_df['side']==side)]
    fig, ax = plt.subplots(3,1,figsize=(16,8), sharey='all', sharex='all')
    rob_muscle = rob_muscle if rob_muscle else phil_muscle
    plt.suptitle(fig_title if fig_title else phil_muscle)
    ax[0].plot(df[phil_muscle+'_Xma'], linestyle='solid', color='red')
    ax[0].plot(df['model_'+rob_muscle+'_glen_Xma'], linestyle='dotted', color='red')
    ax[1].plot(df[phil_muscle+'_Yma'], linestyle='solid', color='green')
    ax[1].plot(df['model_'+rob_muscle+'_glen_Yma'], linestyle='dotted', color='green')
    ax[2].plot(df[phil_muscle+'_Zma'], linestyle='solid', color='blue')
    ax[2].plot(df['model_'+rob_muscle+'_glen_Zma'], linestyle='dotted', color='blue')
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir,phil_muscle+plot_suffix+'.png'), dpi=150, transparent=True)
    return ax


def makeAllComparisonsForAnimalSide(phil_df, rob_df, animal, side, save_dir):
    plt.close('all')
    muscle_list = findComparisons(phil_df, animal, side)
    for muscle in muscle_list:
        makeMMA3plot(save_dir, phil_df, rob_df, animal, side, muscle)

def getPercentageDifference(phil_df, rob_df, animal, side, phil_muscle, rob_muscle):
    rob_df = rob_df.loc[(phil_df['animal']==animal)&(phil_df['side']==side)]
    difference = phil_df[phil_muscle] - rob_df[rob_muscle]
    percentage_difference = difference/phil_df[phil_muscle]*100
    return filter_outliers(percentage_difference)


def getRobAliases(aliasDict, rob_df):
    muscleNamesList = set(rob_df.columns)
    result = {}
    for (key,value) in aliasDict.items():
        matches = [re.search(value, name, re.IGNORECASE) for name in muscleNamesList]
        matched_names = [name.group(0) for name in matches if name]
        result[key] = matched_names
    all_matches = {item for sublist in result.values() for item in sublist}
    return result

def dictFromScanDir(directory, extension):
    key_dict ={'X':'X','Y':'Y','Z':'Z','h':'length'}
    file_dict={'Acromion':{},'Glenoid':{},'Elbow':{},'Length':''}
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.lower().endswith(extension):
                filename = os.path.join(root, name)
                joint = 'Acromion' if ('cromion' in name) else 'Glenoid' if ('leno' in name) else 'Elbow' if ('lbow' in name) else 'Length' if ('ength' in name) else None
                if joint in ['Acromion','Glenoid','Elbow']:
                    axis_key = key_dict[os.path.splitext(name)[0][-1]]
                    file_dict[joint][axis_key]=filename
                elif joint == 'Length':
                    file_dict['Length'] =filename
            else:
                continue
    return(file_dict)

def dfs_from_sto_dir(sto_dir, side_name, mmToCm=True):
    sto_dict = dictFromScanDir(sto_dir, '.sto')
    dfs = []
    dfs.append(import_sto(sto_dict['Length'], 'Length', side_name))
    for joint in ['Acromion','Glenoid','Elbow']:
        for axis in ['X','Y','Z']:
            dfs.append(import_sto(sto_dict[joint][axis], joint, side_name, axis=axis))
    big_df = pd.concat(dfs, axis=1)
    if mmToCm:
        big_df /= 10
    return big_df

def getSplitSubIndices(df, animal, side):
    filt = df.loc[(df['animal']==animal)&(df['side']==side)]
    return (list(filt.index))
def restorePhilIndex(phil_df, rob_df, animal, side):
    index_to_use = getSplitSubIndices(phil_df,animal,side)
    rob_df.index = index_to_use
    return(rob_df)
def mergeRobIntoPhil(phil_df, rob_df, animal, side):
    phil_df = phil_df.copy()
    if len(phil_df.columns)<len(rob_df.columns):
        phil_df = generateRobColumns(phil_df, rob_df)    

    reindexed_rob = restorePhilIndex(phil_df, rob_df, animal, side)
    result = phil_df.combine_first(reindexed_rob)
    return(result)
def generateRobColumns(phil_df, rob_df):
    cols = list(rob_df.columns)
    result = phil_df.copy()
    old_cols = list(result.columns)
    new_cols = old_cols + cols
    result = result.reindex(columns=new_cols)
    return(result)    
    
def dropMMA0s(df):
    df = df.replace(0.0, np.nan)
    df = df.dropna(how='any', axis=0)
    df = df.replace(np.nan, 0.0)
    return df

def import_sto(path, joint, side_name, axis=None):
    axis_dict = {'X':'_Xma', 'Y':'_Yma', 'Z':'_Zma'}
    df = pd.read_csv(path, sep='\t', index_col=None, skiprows=np.linspace(0,10))
    actual_side = 'right' if side_name=="left" else 'left'
    df = df.filter(regex=re.compile(f"{actual_side}", re.IGNORECASE))
    axis_name = axis_dict[axis] if axis else ''
    rename_map = reverseAliasMap(getRobAliases(rob_aliases, df), add_split=False)
    df = df.rename(columns=rename_map)
    joint_name = '_elbo' if joint == 'Elbow' else '_acro' if joint == 'Acromion' else '_glen' if joint == 'Glenoid' else '_length' if joint == 'Length' else ''
    df.columns = ['model_'+col+joint_name+axis_name for col in df.columns]
    df = df.loc[:, (df != 0).any(axis=0)]
    return df




### Import shoulder and elbow JCS data
NB: l and r are swapped in XROMM data. variable names reflect swapped aka wrong sides. JCS are set up such that rotations are correct but left-side (labeled right-side) translations must be reversed. Additionally, these rotations were output using a - extend + flex elbow JCS, must be reversed in code


In [7]:
rob_aliases = { 'pec_crn':r'([a-z]*peca[a-z]*)',
            'pec_int':r'([a-z]*pecm[a-z]*)',
            'pec_cdl':r'([a-z]*pecp[a-z]*)',
            'lat_crn':r'([a-z]*lada[a-z]*)',
            'lat_int':r'([a-z]*ladm[a-z]*)',
            'lat_cdl':r'([a-z]*ladp[a-z]*)',
            'dac':r'([a-z]*dac[a-z]*)',
            'dsc':r'([a-z]*dsc[a-z]*)',
            'dcl':r'([a-z]*dcl[a-z]*)',
            'bic':r'([a-z]*biculn[a-z]*)',
            'bicr':r'([a-z]*bicrad[a-z]*)',
            'bicr2':r'([a-z]*bicrad2[a-z]*)',
            'bra':r'([a-z]*bra[a-z]*)',
            'brar':r'([a-z]*brarad[a-z]*)',
            'cbb':r'([a-z]*cbb[a-z]*)',
            'cbl':r'([a-z]*cbl[a-z]*)',
            'trs':r'([a-z]*trs[a-z]*)', 
            'trl':r'([a-z]*trl[a-z]*)', 
            'trm':r'([a-z]*trm[a-z]*)', 
            'tmj':r'([a-z]*tmj[a-z]*)',
            'tmn':r'([a-z]*tmn[a-z]*|[a-z]*sha[a-z]*)',
            'isp':r'([a-z]*isp[a-z]*)',
            'ssp':r'([a-z]*ssp[a-z]*|[a-z]*spc[a-z]*)',
            'sbs':r'([a-z]*sbs[a-z]*)',
            'sbc':r'([a-z]*sbc[a-z]*)',
              }


jcs85R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_right_triceps_lat-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs88L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_left_teresmaj_pec-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs88R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_trilong_pec-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs92R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv92_biceps_3pec_delt-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs101L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101_11Apr-Phil-2020-04-13-diff/maya/JCS-withScap.csv', 
                     index_col=0)
jcs105R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm105-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs126 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm126-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs126L = jcs126.filter(like='Ldata', axis=1)
jcs126R = jcs126.filter(like='Rdata', axis=1)   
jcs85L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_left_biceps_teres_lat-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs101R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101right_biceps_triceps-Phil-2020-06-08/maya/JCS-withScap.csv', 
                     index_col=0)
jcs108 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm108-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs108L = jcs108.filter(like='ldata', axis=1)
jcs108R = jcs108.filter(like='rdata', axis=1)


jcs_dfs = {'jcs_85_R':jcs85R, 'jcs_85_L':jcs85L, 'jcs_88_L':jcs88L, 'jcs_88_R':jcs88R, 'jcs_92_R':jcs92R, 'jcs_101_L':jcs101L, 'jcs_101_R':jcs101R, 'jcs_105_R':jcs105R, 'jcs_108_L':jcs108L, 'jcs_108_R':jcs108R, 'jcs_126_L':jcs126L, 'jcs_126_R':jcs126R}
jcs_dfs_cols = {item for sublist in [list(df.columns) for df in jcs_dfs.values()] for item in sublist}
jcs_dfs_cols
# jcs_cols = ['Tx','Ty','Tz','Rx','Ry','Rz']
species_dict = {'85':'Opossum','88':'Opossum','92':'Opossum','101':'Opossum','105':'Tegu','108':'Tegu','126':'Tegu'}

#measurements are correct, no need to flip
humeral_lengths = {'85_R':56.750, '85_L':56.620, '88_R':54.015, '88_L':53.454, '92_R':55.358, '101_R':59.293,'101_L':60.039,'105_R':46.588,'108_R':42.134,'108_L':42.824,'126_R':43.666,'126_L':43.296}
humeral_scalefactor = {'85_R':66.639, '85_L':66.639, '88_R':66.639, '88_L':66.639, '92_R':66.639, '101_R':66.639,'101_L':66.639,'105_R':43.5175,'108_R':43.5175,'108_L':43.5175,'126_R':43.5175,'126_L':43.5175}
humeral_volumes = {'85_R':1963.100, '85_L':1734.652, '88_R':1947.153, '88_L':1485.409, '92_R':1408.360, '101_R':2228.326,'101_L':2072.598,'105_R':947.092,'108_R':1345.199,'108_L':1325.958,'126_R':1064.988,'126_L':1061.491}
humeral_volumes_cuberoot = {key:humeral_volumes[key]**(1/3) for key in humeral_volumes}

orangeYellowSeafoam = ['#54100F','#9C4622','#DC8528','#FFD793','#BCE1D0','#3CA9A6','#08676A','#012C2D']
redYellowBlue = ['#451021','#A62547','#E47961','#F8D4AC','#BDE2CF','#559FBA','#1C5796','#012C2D']
teguColor = orangeYellowSeafoam[5]
opossumColor = orangeYellowSeafoam[3]
plt.rcParams.update({
    'grid.linewidth':  0,
    'grid.color':  'lightgrey',
    'savefig.facecolor': (0.0, 0.0, 0.0, 0.0), 
    'savefig.transparent': True,
})


ma85R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_right_triceps_lat-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma88L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_left_teresmaj_pec-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma88R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_trilong_pec-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma92R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv92_biceps_3pec_delt-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma101L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101_11Apr-Phil-2020-04-13-diff/maya/momentArms.csv', 
                     index_col=0)
ma105R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm105-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma126 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm126-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma126L = ma126.filter(regex='_l_.*_jnt', axis=1)
ma126R = ma126.filter(regex='_r_.*_jnt', axis=1)
ma85L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_left_biceps_teres_lat-Phil-2020-06-09/maya/MomentArms.csv', 
                     index_col=0)
ma101R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101right_biceps_triceps-Phil-2020-06-08/maya/MA.csv', 
                     index_col=0)
ma108 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm108-Phil-2020-06-09/maya/MomentArms.csv', 
                     index_col=0)
ma108L = ma108.filter(regex='_l_.*_jnt', axis=1)
ma108R = ma108.filter(regex='_r_.*_jnt', axis=1)

ma_dfs = {'ma_85_R':ma85R, 'ma_85_L':ma85L, 'ma_88_L':ma88L, 'ma_88_R':ma88R, 'ma_92_R':ma92R, 'ma_101_L':ma101L, 'ma_101_R':ma101R, 'ma_105_R':ma105R, 'ma_108_L':ma108L, 'ma_108_R':ma108R, 'ma_126_L':ma126L, 'ma_126_R':ma126R}

def filterMaByJcs(maDict, jcsDict):
    newDict = maDict.copy() 
    for (key, value) in maDict.items():
        newDict[key] = newDict[key][newDict[key].index.isin(jcsDict['jcs'+'_'+key.split('_',1)[-1]].index)]
    return newDict

ma_dfs = filterMaByJcs(ma_dfs, jcs_dfs)



jcs_dfs = generateNewCols(jcs_dfs, mode='jcs')
jcs_dfs = renameAndDrop(jcs_dfs)
jcs_concat = pd.concat(jcs_dfs.values())


#flip Y and Z for elbow to match latest CS convention: +Extension - Flexion, +ulnar dev -radial dev
# 7/28 discovered wrong flipping for certain tegus; all opossums fine and just need the standard elbow Y,Z flip, but
# don't flip 105 at all
# flip elbowz elbowy for everyone else
# additionally flip glenoidx glenoidz for 108R
jcs_concat.loc[(jcs_concat['animal']!='105'),['elbow.ty','elbow.tz','elbow.ry','elbow.rz']] *= -1
jcs_concat.loc[(jcs_concat['animal']=='108')&(jcs_concat['side']=='R'),['glenoid.tx','glenoid.tz','glenoid.rx','glenoid.rz']] *= -1

rob_df = jcs_concat.copy()
#flip left and right labels for Rob's version
rob_df.replace({'side':'R'}, 'Left', inplace=True)
rob_df.replace({'side':'L'}, 'Right', inplace=True)
rob_df.to_csv('/Users/phil/Downloads/ex_vivo_rob.csv')

         
raw_muscle_names = getMuscleNames(ma_dfs)
alias_map = getAliases(aliases, raw_muscle_names)
reverse_alias_map = reverseAliasMap(alias_map)
ma_dfs = renameMusclesWithMap(ma_dfs, reverse_alias_map)
ma_dfs = generateNewCols(ma_dfs, mode='mma')
#merge moment arm data
ma_concat = pd.concat(ma_dfs.values())


all_data = pd.merge(jcs_concat, ma_concat)

#scale moment arms and translations
re_t = '(.t[xyz])'
t_columns = [col for col in all_data.columns if re.search( re_t, col)]
ma_columns = [col for col in all_data.columns if col[-2:] == 'ma']
t_ma_all_data = all_data[t_columns+ma_columns]
scaled_t_ma_all_data = t_ma_all_data.divide(all_data['humeralVolCubeRoot'],axis=0)
scaled_t_ma_all_data = t_ma_all_data.divide(all_data['humeralLength'],axis=0).multiply(all_data['humeralLengthFactor'],axis=0)

scaled_t_ma_all_data = scaled_t_ma_all_data.add_prefix('scaled_')

#cosine-correct rotations
jcs_alldata = all_data.filter(like='.r')
##old wrong
# jcs_alldata['radRy'] = np.radians(jcs_alldata['Ry'])
# jcs_alldata['cosRy'] = np.cos(jcs_alldata['radRy'])
# jcs_alldata['RxcosRy'] = jcs_alldata['cosRy'].multiply(jcs_alldata['Rx'])
# all_data['cRx'] = jcs_alldata['RxcosRy']

jcs_alldata.loc[:,'elbow.radRy'] = np.radians(jcs_alldata['elbow.ry'])
jcs_alldata.loc[:,'glenoid.radRy'] = np.radians(jcs_alldata['glenoid.ry'])
jcs_alldata.loc[:,'acromion.radRy'] = np.radians(jcs_alldata['acromion.ry'])


jcs_alldata.loc[:,'elbow.cosRy'] = np.cos(jcs_alldata['elbow.radRy'])
jcs_alldata.loc[:,'glenoid.cosRy'] = np.cos(jcs_alldata['glenoid.radRy'])
jcs_alldata.loc[:,'acromion.cosRy'] = np.cos(jcs_alldata['acromion.radRy'])


jcs_alldata.loc[:,'elbow.RzcosRy'] = jcs_alldata['elbow.cosRy'].multiply(jcs_alldata['elbow.rz'])
jcs_alldata.loc[:,'glenoid.RzcosRy'] = jcs_alldata['glenoid.cosRy'].multiply(jcs_alldata['glenoid.rz'])
jcs_alldata.loc[:,'acromion.RzcosRy'] = jcs_alldata['acromion.cosRy'].multiply(jcs_alldata['acromion.rz'])


all_data.loc[:,'glenoid.crz'] = jcs_alldata['glenoid.RzcosRy']
all_data.loc[:,'elbow.crz'] = jcs_alldata['elbow.RzcosRy']
all_data.loc[:,'acromion.crz'] = jcs_alldata['acromion.RzcosRy']


# df = all_data.join(scaled_t_ma_all_data)
df = all_data


#reverse left-side translations (labeled right-side in data imports due to XROMM mirroring). This assumes axes are correctly flipped and now follow protocol
new_t_columns = [col for col in df.columns if re.search( re_t, col)]
df.loc[df['side']=='R',new_t_columns] *= -1




rob85R = dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/SEP85L', 'right')
rob85L = dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/SEP85R', 'left')
rob101R= dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/SEP101L', 'right')
rob101L= dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/SEP101R', 'left')
rob92R= dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/SEP92L', 'right')
rob88R= dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/SEP88L', 'right')
rob88L= dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/SEP88R', 'left')

rob105R = dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/105L', 'right', mmToCm=True)
rob108L = dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/108R', 'left', mmToCm=True)
rob108R = dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/108L', 'right', mmToCm=True)
rob126L = dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/126R', 'left', mmToCm=True)
rob126R = dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/126L', 'right', mmToCm=True)

dfOS = df.copy()

for rob_df, animal, side in zip([rob85R,rob85L,rob101R,rob101L, rob92R, rob88R, rob88L],['85','85','101','101','92','88','88'],['R','L','R','L','R','R','L']):
    dfOS = mergeRobIntoPhil(dfOS, rob_df, animal, side)
for rob_df, animal, side in zip([rob105R,rob108L,rob108R,rob126L, rob126R],['105','108','108','126','126'],['R','L','R','L','R']):
    dfOS = mergeRobIntoPhil(dfOS, rob_df, animal, side)
        
    
dfOS['model_lat_avg_glen_Xma'] = dfOS.loc[:,['model_lat_crn_glen_Xma','model_lat_int_glen_Xma','model_lat_cdl_glen_Xma']].mean(axis=1)
dfOS['model_lat_avg_glen_Yma'] = dfOS.loc[:,['model_lat_crn_glen_Yma','model_lat_int_glen_Yma','model_lat_cdl_glen_Yma']].mean(axis=1)
dfOS['model_lat_avg_glen_Zma'] = dfOS.loc[:,['model_lat_crn_glen_Zma','model_lat_int_glen_Zma','model_lat_cdl_glen_Zma']].mean(axis=1)

dfOS['model_pec_avg_glen_Xma'] = dfOS.loc[:,['model_pec_crn_glen_Xma','model_pec_int_glen_Xma','model_pec_cdl_glen_Xma']].mean(axis=1)
dfOS['model_pec_avg_glen_Yma'] = dfOS.loc[:,['model_pec_crn_glen_Yma','model_pec_int_glen_Yma','model_pec_cdl_glen_Yma']].mean(axis=1)
dfOS['model_pec_avg_glen_Zma'] = dfOS.loc[:,['model_pec_crn_glen_Zma','model_pec_int_glen_Zma','model_pec_cdl_glen_Zma']].mean(axis=1)

dfOS['model_isp_glen_Xma'].fillna(dfOS['model_ssp_glen_Xma'], inplace=True)
dfOS['model_isp_glen_Yma'].fillna(dfOS['model_ssp_glen_Yma'], inplace=True)
dfOS['model_isp_glen_Zma'].fillna(dfOS['model_ssp_glen_Zma'], inplace=True)
dfOS['model_isp_length'].fillna(dfOS['model_ssp_length'], inplace=True)



# swap x and z axes in opossum glenoid translation ("compression-distraction")
new_glenoid_tx = np.where(df['species']=='Opossum', df['glenoid.tz'], df['glenoid.tx'])
new_glenoid_tz = np.where(df['species']=='Opossum', df['glenoid.tx'], df['glenoid.tz'])
df['glenoid.tx'] = new_glenoid_tx
df['glenoid.tz'] = new_glenoid_tz
dfOS['glenoid.tx'] = new_glenoid_tx
dfOS['glenoid.tz'] = new_glenoid_tz

# manually drop elbow outliers by visual inspection
i101L = df.loc[((df['animal']=='101')&(df['side']=='L')&((df['frame'] ==464)|(df['frame'] ==309)))].index
i101R = df.loc[(df['animal']=='101')&(df['side']=='R')&((df['frame'] ==770)|(df['frame'] ==436))].index
i92R = df.loc[(df['animal']=='92')&(df['side']=='R')&((df['frame'] ==273)|(df['frame'] ==274))].index
i88R = df.loc[((df['animal']=='88')&(df['side']=='R')&((df['frame'] ==153)))].index

i88L = df.loc[((df['animal']=='88')&(df['side']=='L')&((df['frame'] ==193)|(df['frame'] ==614)|(df['frame'] ==297)))].index
i85L = df.loc[((df['animal']=='85')&(df['side']=='L')&((df['frame'] ==585)|(df['frame'] ==653)|(df['frame'] ==555)|(df['frame'] ==526)|(df['frame'] ==697)))].index
i85R = df.loc[((df['animal']=='85')&(df['side']=='R')&((df['frame'] ==38)|(df['frame'] ==551)|(df['frame'] ==654)|(df['frame'] ==653)|(df['frame'] ==547)|(df['frame'] ==585)))].index
i126L = df.loc[((df['animal']=='126')&(df['side']=='L')&((df['frame'] ==125)|(df['frame'] ==81)))].index
i126R = df.loc[((df['animal']=='126')&(df['side']=='R')&((df['frame'] ==342)|(df['frame'] ==241)))].index
i108R = df.loc[((df['animal']=='108')&(df['side']=='R')&((df['frame'] ==506)|(df['frame'] ==610)))].index
i108L = df.loc[((df['animal']=='108')&(df['side']=='L')&((df['frame'] ==249)|(df['frame'] ==345)))].index

df = df.drop([*i101L,*i101R,*i92R,*i88L,*i88R,*i85L,*i85R, *i126L,*i126R,*i108R,*i108L])

i101L_all = df.loc[((df['animal']=='101')&(df['side']=='L'))].index

df_elbow = df.drop([*i101L_all])

z_D_elbow = np.abs(stats.zscore(df_elbow.loc[df_elbow['species']=='Opossum'].filter(regex='.*.t[xyz]|.*.r[xyz]')))
D_outliers = df_elbow.loc[df_elbow['species']=='Opossum'].iloc[np.where(z_D_elbow>4)[0]].index
df_elbow = df_elbow.drop(D_outliers)
dfOS_elbow = dfOS.drop([*i101L,*i88L,*i85L,*i126L,*i108R,*i108L,*D_outliers])



dfS = df[df['species']=='Tegu']
dfD = df[df['species']=='Opossum']
dfOS_S = dfOS[dfOS['species']=='Tegu']
dfOS_D = dfOS[dfOS['species']=='Opossum']

dfS_elbow = df_elbow[df_elbow['species']=='Tegu']
dfD_elbow = df_elbow[df_elbow['species']=='Opossum']
dfOS_S_elbow = dfOS_elbow[dfOS_elbow['species']=='Tegu']
dfOS_D_elbow = dfOS_elbow[dfOS_elbow['species']=='Opossum']

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

### Get experimental data ranges

In [8]:
def roundUpToNearest(value, nearest):
    modulo = value%nearest
    if modulo == 0:
        result = value
    elif value<0:
        result = value-modulo
    else:          
        result = value-value%nearest+nearest
    return result

#calculate ranges
def getRotationRanges(df1, df2, joint, toNearest=None):
    maxrx = math.ceil(max([max(df1[joint+'.rx']),max(df2[joint+'.rx'])])/10)*10
    maxcrz = math.ceil(max([max(df1[joint+'.crz']),max(df2[joint+'.crz'])])/10)*10
    minrx = math.floor(min([min(df1[joint+'.rx']),min(df2[joint+'.rx'])])/10)*10
    mincrz = math.floor(min([min(df1[joint+'.crz']),min(df2[joint+'.crz'])])/10)*10
    maxry = math.ceil(max([max(df1[joint+'.ry']),max(df2[joint+'.ry'])])/10)*10
    minry = math.floor(min([min(df1[joint+'.ry']),min(df2[joint+'.ry'])])/10)*10
    maxrz = math.ceil(max([max(df1[joint+'.rz']),max(df2[joint+'.rz'])])/10)*10
    minrz = math.floor(min([min(df1[joint+'.rz']),min(df2[joint+'.rz'])])/10)*10
    result = {'maxrx':maxrx, 'maxcrz':maxcrz, 'minrx':minrx, 'mincrz':mincrz, 'maxry':maxry, 'minry':minry, 'maxrz':maxrz, 'minrz':minrz}
    if toNearest:
        result = {key: roundUpToNearest(val, toNearest) for key,val in result.items() }
    return result

def getTranslationRanges(df1, df2, joint, toNearest=None):
    maxtx = math.ceil(max([max(df1[joint+'.tx']),max(df2[joint+'.tx'])])/.1)*.1
    mintx = math.floor(min([min(df1[joint+'.tx']),min(df2[joint+'.tx'])])/.1)*.1
    maxty = math.ceil(max([max(df1[joint+'.ty']),max(df2[joint+'.ty'])])/.1)*.1
    minty = math.floor(min([min(df1[joint+'.ty']),min(df2[joint+'.ty'])])/.1)*.1
    maxtz = math.ceil(max([max(df1[joint+'.tz']),max(df2[joint+'.tz'])])/.1)*.1
    mintz = math.floor(min([min(df1[joint+'.tz']),min(df2[joint+'.tz'])])/.1)*.1
    result = {'maxtx':maxtx, 'mintx':mintx, 'maxty':maxty, 'minty':minty, 'maxtz':maxtz, 'mintz':mintz}
    if toNearest:
        result = {key: round(roundUpToNearest(val, toNearest),2) for key,val in result.items() }
    return result

scap_ranges = getRotationRanges(dfS, dfD, 'acromion',30)
glenoid_ranges = getRotationRanges(dfS, dfD, 'glenoid',30)
elbow_ranges = getRotationRanges(dfS_elbow, dfD_elbow, 'elbow',30)
shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }


scap_ranges_t = getTranslationRanges(dfS, dfD, 'acromion', .5)
glenoid_ranges_t = getTranslationRanges(dfS, dfD, 'glenoid', .5)
elbow_ranges_t = getTranslationRanges(dfS_elbow, dfD_elbow, 'elbow', .5)


#find min, max frames
sMaxrx_glenoid = dfS[dfS['glenoid.rx']==max(dfS['glenoid.rx'])]
sMinrx_glenoid = dfS[dfS['glenoid.rx']==min(dfS['glenoid.rx'])]
sMaxry_glenoid = dfS[dfS['glenoid.ry']==max(dfS['glenoid.ry'])]
sMinry_glenoid = dfS[dfS['glenoid.ry']==min(dfS['glenoid.ry'])]
sMaxcrz_glenoid = dfS[dfS['glenoid.crz']==max(dfS['glenoid.crz'])]
sMincrz_glenoid = dfS[dfS['glenoid.crz']==min(dfS['glenoid.crz'])]
sMaxrz_glenoid = dfS[dfS['glenoid.rz']==max(dfS['glenoid.rz'])]
sMinrz_glenoid = dfS[dfS['glenoid.rz']==min(dfS['glenoid.rz'])]

sMaxrx_elbow = dfS_elbow[dfS_elbow['elbow.rx']==max(dfS_elbow['elbow.rx'])]
sMinrx_elbow = dfS_elbow[dfS_elbow['elbow.rx']==min(dfS_elbow['elbow.rx'])]
sMaxry_elbow = dfS_elbow[dfS_elbow['elbow.ry']==max(dfS_elbow['elbow.ry'])]
sMinry_elbow = dfS_elbow[dfS_elbow['elbow.ry']==min(dfS_elbow['elbow.ry'])]
sMaxcrz_elbow = dfS_elbow[dfS_elbow['elbow.crz']==max(dfS_elbow['elbow.crz'])]
sMincrz_elbow = dfS_elbow[dfS_elbow['elbow.crz']==min(dfS_elbow['elbow.crz'])]
sMaxrz_elbow = dfS_elbow[dfS_elbow['elbow.rz']==max(dfS_elbow['elbow.rz'])]
sMinrz_elbow = dfS_elbow[dfS_elbow['elbow.rz']==min(dfS_elbow['elbow.rz'])]

dMaxrx_glenoid = dfD[dfD['glenoid.rx']==max(dfD['glenoid.rx'])]
dMinrx_glenoid = dfD[dfD['glenoid.rx']==min(dfD['glenoid.rx'])]
dMaxry_glenoid = dfD[dfD['glenoid.ry']==max(dfD['glenoid.ry'])]
dMinry_glenoid = dfD[dfD['glenoid.ry']==min(dfD['glenoid.ry'])]
dMaxcrz_glenoid = dfD[dfD['glenoid.crz']==max(dfD['glenoid.crz'])]
dMincrz_glenoid = dfD[dfD['glenoid.crz']==min(dfD['glenoid.crz'])]
dMaxrz_glenoid = dfD[dfD['glenoid.rz']==max(dfD['glenoid.rz'])]
dMinrz_glenoid = dfD[dfD['glenoid.rz']==min(dfD['glenoid.rz'])]

dMaxrx_elbow = dfD_elbow[dfD_elbow['elbow.rx']==max(dfD_elbow['elbow.rx'])]
dMinrx_elbow = dfD_elbow[dfD_elbow['elbow.rx']==min(dfD_elbow['elbow.rx'])]
dMaxry_elbow = dfD_elbow[dfD_elbow['elbow.ry']==max(dfD_elbow['elbow.ry'])]
dMinry_elbow = dfD_elbow[dfD_elbow['elbow.ry']==min(dfD_elbow['elbow.ry'])]
dMaxcrz_elbow = dfD_elbow[dfD_elbow['elbow.crz']==max(dfD_elbow['elbow.crz'])]
dMincrz_elbow = dfD_elbow[dfD_elbow['elbow.crz']==min(dfD_elbow['elbow.crz'])]
dMaxrz_elbow = dfD_elbow[dfD_elbow['elbow.rz']==max(dfD_elbow['elbow.rz'])]
dMinrz_elbow = dfD_elbow[dfD_elbow['elbow.rz']==min(dfD_elbow['elbow.rz'])]

### Report ranges

In [9]:
def getMinMax(df, joint, roundto=2):
    result = {'rx':{},'ry':{},'crz':{},'rz':{},'tx':{},'ty':{},'tz':{}}
    for dof in result.keys():
        result[dof]['min'] = df[joint+'.'+dof].min().round(roundto)
        result[dof]['max'] = df[joint+'.'+dof].max().round(roundto)
        result[dof]['range'] = abs(df[joint+'.'+dof].min()-df[joint+'.'+dof].max()).round(roundto)
    return(result)

getMinMax(dfS, 'glenoid',2)
        

{'rx': {'min': -96.16, 'max': 55.4, 'range': 151.56},
 'ry': {'min': -68.23, 'max': 80.01, 'range': 148.23},
 'crz': {'min': -75.9, 'max': 61.02, 'range': 136.93},
 'rz': {'min': -89.97, 'max': 64.99, 'range': 154.96},
 'tx': {'min': -0.26, 'max': 0.38, 'range': 0.64},
 'ty': {'min': -0.53, 'max': 0.32, 'range': 0.85},
 'tz': {'min': -0.3, 'max': 0.19, 'range': 0.49}}

### Calculate alpha shapes

In [ ]:

alphaVertScap_S,alphaEdgeScap_S,alphaTriScap_S = alpha_shape_3D(dfS.loc[:,['acromion.rx','acromion.ry','acromion.crz']].to_numpy(),20)
alphaVertScap_D,alphaEdgeScap_D,alphaTriScap_D = alpha_shape_3D(dfD.loc[:,['acromion.rx','acromion.ry','acromion.crz']].to_numpy(),20)
alphaVertShoulder_S,alphaEdgeShoulder_S,alphaTriShoulder_S = alpha_shape_3D(dfS.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),20)
alphaVertShoulder_D,alphaEdgeShoulder_D,alphaTriShoulder_D = alpha_shape_3D(dfD.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),20)
alphaVertElbow_S,alphaEdgeElbow_S,alphaTriElbow_S = alpha_shape_3D(dfS_elbow.loc[:,['elbow.rx','elbow.ry','elbow.crz']].to_numpy(),20)
alphaVertElbow_D,alphaEdgeElbow_D,alphaTriElbow_D = alpha_shape_3D(dfD_elbow.loc[:,['elbow.rx','elbow.ry','elbow.crz']].to_numpy(),20)



alphaVertScap_S_translation,alphaEdgeScap_S_translation,alphaTriScap_S_translation = alpha_shape_3D(dfS.loc[:,['acromion.tx','acromion.ty','acromion.tz']].to_numpy(),20)
alphaVertScap_D_translation,alphaEdgeScap_D_translation,alphaTriScap_D_translation = alpha_shape_3D(dfD.loc[:,['acromion.tx','acromion.ty','acromion.tz']].to_numpy(),20)
alphaVertShoulder_S_translation,alphaEdgeShoulder_S_translation,alphaTriShoulder_S_translation = alpha_shape_3D(dfS.loc[:,['glenoid.tx','glenoid.ty','glenoid.tz']].to_numpy(),20)
alphaVertShoulder_D_translation,alphaEdgeShoulder_D_translation,alphaTriShoulder_D_translation = alpha_shape_3D(dfD.loc[:,['glenoid.tx','glenoid.ty','glenoid.tz']].to_numpy(),20)
alphaVertElbow_S_translation,alphaEdgeElbow_S_translation,alphaTriElbow_S_translation = alpha_shape_3D(dfS_elbow.loc[:,['elbow.tx','elbow.ty','elbow.tz']].to_numpy(),20)
alphaVertElbow_D_translation,alphaEdgeElbow_D_translation,alphaTriElbow_D_translation = alpha_shape_3D(dfD_elbow.loc[:,['elbow.tx','elbow.ty','elbow.tz']].to_numpy(),20)


In [263]:
def getRange(dfOS, musclename):
    result = {'X':None,'Y':None,'Z':None}
    for axis in ['X','Y','Z']:
        dfPhil = dfOS[musclename+'_'+axis.upper()+'ma']
        dfRob = dfOS['model_'+musclename+'_glen_'+axis.upper()+'ma']
        maxPhil = (dfPhil.max(),abs(dfPhil.min()))
        maxRob = (dfRob.max(),abs(dfRob.min()))
        maxBoth = max(maxPhil,maxRob)
        result[axis] = max(maxBoth)
    return result
def makeValidationPlot(dfP, dfR, musclename, species, ylim, xstart,xend, savedir):
#     plt.close('all')
    lineweight=2
    fig, ax = plt.subplots(3,1,figsize=(12,4), sharey='row', sharex='col')
    fontsize =12
    hoffset = 1.4
    dottedline = (0, (1, 1))
    ax[0].plot(dfP[musclename+'_Xma'], linestyle='solid', color='red', linewidth=lineweight)
    ax[0].plot(dfR['model_'+musclename+'_glen_Xma'], linestyle=dottedline, color='red', linewidth=lineweight)
    ax[1].plot(dfP[musclename+'_Yma'], linestyle='solid', color='green', linewidth=lineweight)
    ax[1].plot(dfR['model_'+musclename+'_glen_Yma'], linestyle=dottedline, color='green', linewidth=lineweight)
    ax[2].plot(dfP[musclename+'_Zma'], linestyle='solid', color='blue', linewidth=lineweight)
    ax[2].plot(dfR['model_'+musclename+'_glen_Zma'], linestyle=dottedline, color='blue', linewidth=lineweight)
    for axis in ax:
        axis.set_xlim(xstart,xend)
        axis.set_ylim(-ylim, ylim)
#     xrange = max(max(dfP[musclename+'_Xma'].max(),abs(dfP[musclename+'_Xma'].min())),max(dfR['model_'+musclename+'_glen_Xma'].max(),abs(dfR['model_'+musclename+'_glen_Xma'].min())))
#     yrange = max(max(dfP[musclename+'_Yma'].max(),abs(dfP[musclename+'_Yma'].min())),max(dfR['model_'+musclename+'_glen_Yma'].max(),abs(dfR['model_'+musclename+'_glen_Yma'].min())))
#     zrange = max(max(dfP[musclename+'_Zma'].max(),abs(dfP[musclename+'_Zma'].min())),max(dfR['model_'+musclename+'_glen_Zma'].max(),abs(dfR['model_'+musclename+'_glen_Zma'].min())))

    plt.savefig(os.path.join(savedir,musclename+'_'+species+'.png'), dpi=300, transparent=True)
    return ax

pec_crn_range = getRange(dfOS, 'pec_crn')
dac_range = getRange(dfOS, 'dac')
dsc_range = getRange(dfOS, 'dsc')
trs_range = getRange(dfOS, 'trs')
bic_range = getRange(dfOS, 'bic')
dac_range

{'X': 0.606208265, 'Y': 1.198867679, 'Z': 1.554459929}

## Validation
### Figure: validation comparison

In [ ]:
plt.close('all')
lineweight=0.75
rangeLoc = (0,500)
rangeLoc_S = (0,400)

fig, ax = plt.subplots(19,3,figsize=(18,18), sharey='all', sharex='col')
fontsize =12
hoffset = 1.4
dottedline = (0, (1, 1))

dfOS_85R = dfOS.loc[(dfOS['animal']=='85')&(dfOS['side']=='R')].reset_index()
dfOS_88R = dfOS.loc[(dfOS['animal']=='88')&(dfOS['side']=='R')].reset_index()
dfOS_101R = dfOS.loc[(dfOS['animal']=='101')&(dfOS['side']=='R')].reset_index()
dfOS_126R = dfOS.loc[(dfOS['animal']=='126')&(dfOS['side']=='R')].reset_index()
dfOS_126L = dfOS.loc[(dfOS['animal']=='126')&(dfOS['side']=='L')].reset_index()
dfOS_108R = dfOS.loc[(dfOS['animal']=='108')&(dfOS['side']=='R')].reset_index()
dfOS_108L = dfOS.loc[(dfOS['animal']=='108')&(dfOS['side']=='L')].reset_index()

for i in [3, 7, 11, 15]:
    for j in [0,1]:
        ax[i,j].set_visible(False)
for i in range(19):
    ax[i,2].set_visible(False)
    
# ax[0,0].set_title('Tegu',fontproperties=roboto_bold, fontsize='16')
# ax[0,1].set_title('Opossum',fontproperties=roboto_bold, fontsize='16')

ax[0,0].plot(dfOS_126L['pec_crn_Xma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle='solid', color='red', linewidth=lineweight)
ax[0,0].plot(dfOS_126L['model_pec_crn_glen_Xma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle=dottedline, color='red', linewidth=lineweight)
ax[1,0].plot(dfOS_126L['pec_crn_Yma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle='solid', color='green', linewidth=lineweight)
ax[1,0].plot(dfOS_126L['model_pec_crn_glen_Yma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle=dottedline, color='green', linewidth=lineweight)
ax[2,0].plot(dfOS_126L['pec_crn_Zma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle='solid', color='blue', linewidth=lineweight)
ax[2,0].plot(dfOS_126L['model_pec_crn_glen_Zma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle=dottedline, color='blue', linewidth=lineweight)

ax[4,0].plot(onBone108L['dac_Xma'].reset_index(drop=True), linestyle='solid', color='red', linewidth=lineweight)
ax[4,0].plot(dfOS_108L['model_dac_glen_Xma'], linestyle=dottedline, color='red', linewidth=lineweight)
ax[5,0].plot(onBone108L['dac_Yma'].reset_index(drop=True), linestyle='solid', color='green', linewidth=lineweight)
ax[5,0].plot(dfOS_108L['model_dac_glen_Yma'], linestyle=dottedline, color='green', linewidth=lineweight)
ax[6,0].plot(onBone108L['dac_Zma'].reset_index(drop=True), linestyle='solid', color='blue', linewidth=lineweight)
ax[6,0].plot(dfOS_108L['model_dac_glen_Zma'], linestyle=dottedline, color='blue', linewidth=lineweight)

ax[8,0].plot(onBone108L['dsc_Xma'].reset_index(drop=True), linestyle='solid', color='red', linewidth=lineweight)
ax[8,0].plot(dfOS_108L['model_dsc_glen_Xma'], linestyle=dottedline, color='red', linewidth=lineweight)
ax[9,0].plot(onBone108L['dsc_Yma'].reset_index(drop=True), linestyle='solid', color='green', linewidth=lineweight)
ax[9,0].plot(dfOS_108L['model_dsc_glen_Yma'], linestyle=dottedline, color='green', linewidth=lineweight)
ax[10,0].plot(onBone108L['dsc_Zma'].reset_index(drop=True), linestyle='solid', color='blue', linewidth=lineweight)
ax[10,0].plot(dfOS_108L['model_dsc_glen_Zma'], linestyle=dottedline, color='blue', linewidth=lineweight)

ax[12,0].plot(dfOS_126R['trs_Xma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle='solid', color='red', linewidth=lineweight)
ax[12,0].plot(dfOS_126R['model_trs_glen_Xma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle=dottedline, color='red', linewidth=lineweight)
ax[13,0].plot(dfOS_126R['trs_Yma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle='solid', color='green', linewidth=lineweight)
ax[13,0].plot(dfOS_126R['model_trs_glen_Yma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle=dottedline, color='green', linewidth=lineweight)
ax[14,0].plot(dfOS_126R['trs_Zma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle='solid', color='blue', linewidth=lineweight)
ax[14,0].plot(dfOS_126R['model_trs_glen_Zma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle=dottedline, color='blue', linewidth=lineweight)

ax[16,0].plot(dfOS_126L['bic_Xma'].reset_index(drop=True).iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle='solid', color='red', linewidth=lineweight)
ax[16,0].plot(dfOS_126L['model_bic_glen_Xma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle=dottedline, color='red', linewidth=lineweight)
ax[17,0].plot(dfOS_126L['bic_Yma'].reset_index(drop=True).iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle='solid', color='green', linewidth=lineweight)
ax[17,0].plot(dfOS_126L['model_bic_glen_Yma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle=dottedline, color='green', linewidth=lineweight)
ax[18,0].plot(dfOS_126L['bic_Zma'].reset_index(drop=True).iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle='solid', color='blue', linewidth=lineweight)
ax[18,0].plot(dfOS_126L['model_bic_glen_Zma'].iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), linestyle=dottedline, color='blue', linewidth=lineweight)




ax[0,1].plot(dfOS_85R['pec_int_Xma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='red', linewidth=lineweight)
ax[0,1].plot(dfOS_85R['model_pec_int_glen_Xma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='red', linewidth=lineweight)
ax[1,1].plot(dfOS_85R['pec_int_Yma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='green', linewidth=lineweight)
ax[1,1].plot(dfOS_85R['model_pec_int_glen_Yma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='green', linewidth=lineweight)
ax[1,1].annotate('m. pectoralis', (hoffset, 0.5),  xycoords='axes fraction', va='center', ha='center',annotation_clip=False, fontproperties = roboto_bold_italic, fontsize=fontsize)
ax[2,1].plot(dfOS_85R['pec_int_Zma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='blue', linewidth=lineweight)
ax[2,1].plot(dfOS_85R['model_pec_int_glen_Zma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='blue', linewidth=lineweight)

ax[4,1].plot(dfOS_101R['dac_Xma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='red', linewidth=lineweight)
ax[4,1].plot(dfOS_101R['model_dac_glen_Xma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='red', linewidth=lineweight)
ax[5,1].plot(dfOS_101R['dac_Yma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='green', linewidth=lineweight)
ax[5,1].plot(dfOS_101R['model_dac_glen_Yma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='green', linewidth=lineweight)
ax[5,1].annotate('m. deltoideus acromialis', (hoffset, 0.5),  xycoords='axes fraction', va='center', ha='center',annotation_clip=False, fontproperties = roboto_bold_italic, fontsize=fontsize)
ax[6,1].plot(dfOS_101R['dac_Zma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='blue', linewidth=lineweight)
ax[6,1].plot(dfOS_101R['model_dac_glen_Zma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='blue', linewidth=lineweight)

ax[8,1].plot(onBone101R['dsc_Xma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='red', linewidth=lineweight)
ax[8,1].plot(dfOS_101R['model_dsc_glen_Xma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='red', linewidth=lineweight)
ax[9,1].plot(onBone101R['dsc_Yma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='green', linewidth=lineweight)
ax[9,1].plot(dfOS_101R['model_dsc_glen_Yma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='green', linewidth=lineweight)
ax[9,1].annotate('m. deltoideus scapularis', (hoffset, 0.5),  xycoords='axes fraction', va='center', ha='center',annotation_clip=False, fontproperties = roboto_bold_italic, fontsize=fontsize)
ax[10,1].plot(onBone101R['dsc_Zma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='blue', linewidth=lineweight)
ax[10,1].plot(dfOS_101R['model_dsc_glen_Zma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='blue', linewidth=lineweight)

ax[12,1].plot(onBone88R['trs_Xma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='red', linewidth=lineweight)
ax[12,1].plot(dfOS_88R['model_trs_glen_Xma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='red', linewidth=lineweight)
ax[13,1].plot(onBone88R['trs_Yma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='green', linewidth=lineweight)
ax[13,1].plot(dfOS_88R['model_trs_glen_Yma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='green', linewidth=lineweight)
ax[13,1].annotate('m. triceps scapularis', (hoffset, 0.5),  xycoords='axes fraction', va='center', ha='center',annotation_clip=False, fontproperties = roboto_bold_italic, fontsize=fontsize)
ax[14,1].plot(onBone88R['trs_Zma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='blue', linewidth=lineweight)
ax[14,1].plot(dfOS_88R['model_trs_glen_Zma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='blue', linewidth=lineweight)

ax[16,1].plot(onBone101Rdb['bic_Xma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='red', linewidth=lineweight)
ax[16,1].plot(dfOS_101R['model_bic_glen_Xma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='red', linewidth=lineweight)
ax[17,1].plot(onBone101Rdb['bic_Yma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='green', linewidth=lineweight)
ax[17,1].plot(dfOS_101R['model_bic_glen_Yma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='green', linewidth=lineweight)
ax[17,1].annotate('m. biceps brachii', (hoffset, 0.5),  xycoords='axes fraction', va='center', ha='center',annotation_clip=False, fontproperties = roboto_bold_italic, fontsize=fontsize)
ax[18,1].plot(onBone101Rdb['bic_Zma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='blue', linewidth=lineweight)
ax[18,1].plot(dfOS_101R['model_bic_glen_Zma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='blue', linewidth=lineweight)

plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/validation.png', dpi=600, transparent=True)


In [278]:
rangeLoc = (0,500)
rangeLoc_S = (0,400)
savedir = '/Users/phil/Desktop/rotoscoping/Figures/ch3plots/validation'

dfOS_85R = dfOS.loc[(dfOS['animal']=='85')&(dfOS['side']=='R')].reset_index()
dfOS_88R = dfOS.loc[(dfOS['animal']=='88')&(dfOS['side']=='R')].reset_index()
dfOS_101R = dfOS.loc[(dfOS['animal']=='101')&(dfOS['side']=='R')].reset_index()
dfOS_126R = dfOS.loc[(dfOS['animal']=='126')&(dfOS['side']=='R')].reset_index()
dfOS_126L = dfOS.loc[(dfOS['animal']=='126')&(dfOS['side']=='L')].reset_index()
dfOS_108R = dfOS.loc[(dfOS['animal']=='108')&(dfOS['side']=='R')].reset_index()
dfOS_108L = dfOS.loc[(dfOS['animal']=='108')&(dfOS['side']=='L')].reset_index()

def getRange(dfP, dfR, musclename, axis):
    dfPhil = dfP[musclename+'_'+axis.upper()+'ma']
    dfRob = dfR['model_'+musclename+'_glen_'+axis.upper()+'ma']
    maxPhil = (dfPhil.max(),abs(dfPhil.min()))
    maxRob = (dfRob.max(),abs(dfRob.min()))
    maxBoth = max(maxPhil,maxRob)
    return maxBoth
def makeValidationPlot(dfP, dfR, musclename, species, ylim, xend, savedir):
#     plt.close('all')
    lineweight=2
    fig, ax = plt.subplots(3,1,figsize=(12,4), sharey='row', sharex='col')
    fontsize =12
    hoffset = 1.4
    dottedline = (0, (1, 1))
    ax[0].plot(dfP[musclename+'_Xma'], linestyle='solid', color='red', linewidth=lineweight)
    ax[0].plot(dfR['model_'+musclename+'_glen_Xma'], linestyle=dottedline, color='red', linewidth=lineweight)
    ax[1].plot(dfP[musclename+'_Yma'], linestyle='solid', color='green', linewidth=lineweight)
    ax[1].plot(dfR['model_'+musclename+'_glen_Yma'], linestyle=dottedline, color='green', linewidth=lineweight)
    ax[2].plot(dfP[musclename+'_Zma'], linestyle='solid', color='blue', linewidth=lineweight)
    ax[2].plot(dfR['model_'+musclename+'_glen_Zma'], linestyle=dottedline, color='blue', linewidth=lineweight)
    for axis in ax:
        axis.set_xlim(0,xend)
        axis.set_ylim(-ylim, ylim)
#     xrange = max(max(dfP[musclename+'_Xma'].max(),abs(dfP[musclename+'_Xma'].min())),max(dfR['model_'+musclename+'_glen_Xma'].max(),abs(dfR['model_'+musclename+'_glen_Xma'].min())))
#     yrange = max(max(dfP[musclename+'_Yma'].max(),abs(dfP[musclename+'_Yma'].min())),max(dfR['model_'+musclename+'_glen_Yma'].max(),abs(dfR['model_'+musclename+'_glen_Yma'].min())))
#     zrange = max(max(dfP[musclename+'_Zma'].max(),abs(dfP[musclename+'_Zma'].min())),max(dfR['model_'+musclename+'_glen_Zma'].max(),abs(dfR['model_'+musclename+'_glen_Zma'].min())))

    plt.savefig(os.path.join(savedir,musclename+'_'+species+'.png'), dpi=300, transparent=True)
    return ax




makeValidationPlot(dfOS_126L.iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), dfOS_126L.iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), 'pec_crn','tegu', 2.5, 500, savedir)
makeValidationPlot(onBone108L.reset_index(drop=True), dfOS_108L, 'dac','tegu', 2.5, 500, savedir)
makeValidationPlot(onBone108L.reset_index(drop=True), dfOS_108L, 'dsc','tegu', 2.5, 500, savedir)
makeValidationPlot(dfOS_126R.iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), dfOS_126R.iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), 'trs','tegu', 2.5, 500, savedir)
makeValidationPlot(dfOS_126L.iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), dfOS_126L.iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80), 'bic','tegu', 2.5, 500, savedir)

makeValidationPlot(dfOS_85R.iloc[rangeLoc[0]:rangeLoc[1]], dfOS_85R.iloc[rangeLoc[0]:rangeLoc[1]]*10, 'pec_crn','opossum', 2.5, 500, savedir)
makeValidationPlot(dfOS_101R.iloc[rangeLoc[0]:rangeLoc[1]], dfOS_101R.iloc[rangeLoc[0]:rangeLoc[1]]*10, 'dac','opossum', 2.5, 500, savedir)
makeValidationPlot(onBone101R.reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], dfOS_101R.iloc[rangeLoc[0]:rangeLoc[1]]*10, 'dsc','opossum', 2.5, 500, savedir)
makeValidationPlot(dfOS_88R.reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], dfOS_88R.iloc[rangeLoc[0]:rangeLoc[1]]*10, 'trs','opossum', 2.5, 500, savedir)
makeValidationPlot(onBone101Rdb.reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], dfOS_101R.iloc[rangeLoc[0]:rangeLoc[1]]*10, 'bic','opossum', 2.5, 500, savedir)



# ax[0,1].plot(dfOS_85R['pec_int_Xma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='red', linewidth=lineweight)
# ax[0,1].plot(dfOS_85R['model_pec_int_glen_Xma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='red', linewidth=lineweight)
# ax[1,1].plot(dfOS_85R['pec_int_Yma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='green', linewidth=lineweight)
# ax[1,1].plot(dfOS_85R['model_pec_int_glen_Yma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='green', linewidth=lineweight)
# ax[1,1].annotate('m. pectoralis', (hoffset, 0.5),  xycoords='axes fraction', va='center', ha='center',annotation_clip=False, fontproperties = roboto_bold_italic, fontsize=fontsize)
# ax[2,1].plot(dfOS_85R['pec_int_Zma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='blue', linewidth=lineweight)
# ax[2,1].plot(dfOS_85R['model_pec_int_glen_Zma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='blue', linewidth=lineweight)

# ax[4,1].plot(dfOS_101R['dac_Xma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='red', linewidth=lineweight)
# ax[4,1].plot(dfOS_101R['model_dac_glen_Xma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='red', linewidth=lineweight)
# ax[5,1].plot(dfOS_101R['dac_Yma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='green', linewidth=lineweight)
# ax[5,1].plot(dfOS_101R['model_dac_glen_Yma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='green', linewidth=lineweight)
# ax[5,1].annotate('m. deltoideus acromialis', (hoffset, 0.5),  xycoords='axes fraction', va='center', ha='center',annotation_clip=False, fontproperties = roboto_bold_italic, fontsize=fontsize)
# ax[6,1].plot(dfOS_101R['dac_Zma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='blue', linewidth=lineweight)
# ax[6,1].plot(dfOS_101R['model_dac_glen_Zma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='blue', linewidth=lineweight)

# ax[8,1].plot(onBone101R['dsc_Xma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='red', linewidth=lineweight)
# ax[8,1].plot(dfOS_101R['model_dsc_glen_Xma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='red', linewidth=lineweight)
# ax[9,1].plot(onBone101R['dsc_Yma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='green', linewidth=lineweight)
# ax[9,1].plot(dfOS_101R['model_dsc_glen_Yma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='green', linewidth=lineweight)
# ax[9,1].annotate('m. deltoideus scapularis', (hoffset, 0.5),  xycoords='axes fraction', va='center', ha='center',annotation_clip=False, fontproperties = roboto_bold_italic, fontsize=fontsize)
# ax[10,1].plot(onBone101R['dsc_Zma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='blue', linewidth=lineweight)
# ax[10,1].plot(dfOS_101R['model_dsc_glen_Zma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='blue', linewidth=lineweight)

# ax[12,1].plot(onBone88R['trs_Xma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='red', linewidth=lineweight)
# ax[12,1].plot(dfOS_88R['model_trs_glen_Xma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='red', linewidth=lineweight)
# ax[13,1].plot(onBone88R['trs_Yma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='green', linewidth=lineweight)
# ax[13,1].plot(dfOS_88R['model_trs_glen_Yma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='green', linewidth=lineweight)
# ax[13,1].annotate('m. triceps scapularis', (hoffset, 0.5),  xycoords='axes fraction', va='center', ha='center',annotation_clip=False, fontproperties = roboto_bold_italic, fontsize=fontsize)
# ax[14,1].plot(onBone88R['trs_Zma'].iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='blue', linewidth=lineweight)
# ax[14,1].plot(dfOS_88R['model_trs_glen_Zma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='blue', linewidth=lineweight)

# ax[16,1].plot(onBone101Rdb['bic_Xma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='red', linewidth=lineweight)
# ax[16,1].plot(dfOS_101R['model_bic_glen_Xma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='red', linewidth=lineweight)
# ax[17,1].plot(onBone101Rdb['bic_Yma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='green', linewidth=lineweight)
# ax[17,1].plot(dfOS_101R['model_bic_glen_Yma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='green', linewidth=lineweight)
# ax[17,1].annotate('m. biceps brachii', (hoffset, 0.5),  xycoords='axes fraction', va='center', ha='center',annotation_clip=False, fontproperties = roboto_bold_italic, fontsize=fontsize)
# ax[18,1].plot(onBone101Rdb['bic_Zma'].reset_index(drop=True).iloc[rangeLoc[0]:rangeLoc[1]], linestyle='solid', color='blue', linewidth=lineweight)
# ax[18,1].plot(dfOS_101R['model_bic_glen_Zma'].iloc[rangeLoc[0]:rangeLoc[1]]*10, linestyle=dottedline, color='blue', linewidth=lineweight)

# plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/validation.png', dpi=600, transparent=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>], dtype=object)

#### 88R comparison

In [194]:
plt.close('all')
dfOS_126L['bic_Zma'].reset_index(drop=True).iloc[rangeLoc_S[0]:rangeLoc_S[1]].drop(80).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [21]:
validation_dictlist = [
    {    'index':1
         'animal':'85',
         'side':'R', 
         'phil':'pec_int',
         'rob':'pec_int',
         'phil_df':dfOS,
         'rob_df':dfOS
         'nicename':'Pectoralis (intermediate)',
    },
        {'animal':'85',
         'side':'R', 
         'phil':'lat_cdl', 
         'rob':'lat_cdl',
         'nicename':'Latissimus (caudal, bone insertion)',
    },    { 'animal':'101',
         'side':'R', 
         'phil':'dac',
         'rob':,
         'nicename':,
    },    {'animal':'101',
         'side':'R', 
         'phil':'dsc',
         'rob':,
         'nicename':,
    },    { 'animal':'88',
         'side':'R', 
         'phil':'trs',
         'rob':'trs',
         'nicename':'Triceps (scapular, bone origin)',
    },
     {    'animal':'101',
         'side':'R', 
         'phil':'bic','bic', fig_title='Biceps, bone origin')
         'rob':,
         'nicename':,
    }
    
    
    dsc', fig_title='Deltoid (scapular)')
    'dac', fig_title='Deltoid (acromial)')
]
[i for i in enumerate(meta_88R.items())]

[(0, ('animal', '88')),
 (1, ('side', 'R')),
 (2,
  ('folder',
   '/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/88R'))]

In [184]:
abc = onBone101Rdb.reset_index(drop=True)
abc.loc[abc['bic_Yma']==abc['bic_Yma'].min()]

,dsc_Xma,dsc_Yma,dsc_Zma,bic_Xma,bic_Yma,bic_Zma,animal,side
153,-0.130321,0.079247,0.097798,0.050453,-0.916711,-0.067935,101,R


In [38]:
meta_88R = {
                'animal':'88',
                'side':'R', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/88R'
           }
onBone88R = dropMMA0s(pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/88R/88RonBone.csv', 
                     index_col=0).rename(columns=reverse_alias_map).assign(animal=meta_88R['animal'],side=meta_88R['side']))
comparisons_88R = findComparisons(dfD, meta_88R['animal'], meta_88R['side'])

##trs, dac and dsc redone on bone

makeMMA3plot(meta_88R['folder'], dfD, rob88R, meta_88R['animal'], meta_88R['side'], 'pec_crn', 'pec_int', fig_title='Pectoralis (intermediate)')
makeMMA3plot(meta_88R['folder'], onBone88R, rob88R, meta_88R['animal'], meta_88R['side'], 'trs', fig_title='Triceps (scapular, bone origin)')
makeMMA3plot(meta_88R['folder'], dfD, rob88R, meta_88R['animal'], meta_88R['side'], 'trs', fig_title='Triceps (scapular, fleshy origin)',plot_suffix='_fsh')

['pec_crn', 'trs', 'dsc', 'dac']


KeyError: 'None of [RangeIndex(start=0, stop=707, step=1)] are in the [index]'

#### 85R comparisons

In [833]:
new85Rlat = pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/85R/newlat85R.csv', 
                     index_col=0)
new85Rtrs = pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/85R/newtrs85R.csv', 
                     index_col=0)
meta_85R = {
                'animal':'85',
                'side':'R', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/85R'
           }
onBone85R = dropMMA0s(pd.concat([new85Rlat,new85Rtrs],axis=1).rename(columns=reverse_alias_map).assign(animal=meta_85R['animal'],side=meta_85R['side']))

comparisons_85R = findComparisons(dfD, meta_85R['animal'], meta_85R['side'])

makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'pec_crn',  fig_title='Pectoralis (cranial)')
makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'pec_int',  fig_title='Pectoralis (intermediate)')
makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'pec_cdl',  fig_title='Pectoralis (caudal)')

makeMMA3plot(meta_85R['folder'], onBone85R, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_crn',  fig_title='Latissimus (cranial, bone insertion)')
makeMMA3plot(meta_85R['folder'], onBone85R, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_cdl',  fig_title='Latissimus (caudal, bone insertion)')

makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_crn',  fig_title='Latissimus (cranial, fleshy insertion)',plot_suffix='_fsh')
makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_cdl',  fig_title='Latissimus (caudal, fleshy insertion)',plot_suffix='_fsh')

makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'trs',  fig_title='Triceps (scapular, fleshy origin)',plot_suffix='_fsh')
makeMMA3plot(meta_85R['folder'], onBone85R, rob85R, meta_85R['animal'], meta_85R['side'], 'trs',  fig_title='Triceps (scapular, bone origin)')

['lat_crn', 'lat_cdl', 'pec_crn', 'pec_int', 'pec_cdl', 'trs', 'dsc', 'dac']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>], dtype=object)

In [1699]:
corr = signal.correlate(dfD.loc[(dfD['animal']=='101')&(dfD['side']=='R')]['pec_crn_Yma'],rob101R['model_pec_crn_glen_Yma'])
np.max(corr)
corr /= np.max(corr)

plt.close('all')
fig,ax = plt.subplots()
ax.plot(corr)
# lags = signal.correlation_lags(len(sig), len(sig_noise))
# dfD, rob101R, meta_101R['animal'], meta_101R['side'], 'pec_crn', fig_title='Pectoralis (cranial)')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### 101R comparisons

In [41]:
meta_101R = {
                'animal':'101',
                'side':'R', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/101R'
           }

onBone101Rdb = dropMMA0s(pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/101R/101Ronbone_dscbic.csv', 
                     index_col=0).rename(columns=reverse_alias_map).assign(animal=meta_101R['animal'],side=meta_101R['side']))
onBone101R = dropMMA0s(pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/101R/101Ronbone.csv', 
                     index_col=0).rename(columns=reverse_alias_map).assign(animal=meta_101R['animal'],side=meta_101R['side']))
comparisons_101R = findComparisons(dfD, meta_101R['animal'], meta_101R['side'])


makeMMA3plot(meta_101R['folder'], dfOS, meta_101R['animal'], meta_101R['side'], 'pec_crn', fig_title='Pectoralis (cranial)')
makeMMA3plot(meta_101R['folder'], dfOS, meta_101R['animal'], meta_101R['side'], 'pec_cdl', fig_title='Pectoralis (caudal)')
makeMMA3plotFromMerged(meta_101R['folder'], onBone101R, rob101R, meta_101R['animal'], meta_101R['side'], 'trs', fig_title='Triceps (scapular, bone origin)', plot_suffix='')
makeMMA3plot(meta_101R['folder'], dfOS, meta_101R['animal'], meta_101R['side'], 'trs', fig_title='Triceps (scapular, fleshy origin)', plot_suffix='_fsh')
makeMMA3plotFromMerged(meta_101R['folder'], onBone101R, rob101R, meta_101R['animal'], meta_101R['side'], 'dsc', fig_title='Deltoid (scapular)')
makeMMA3plot(meta_101R['folder'], dfOS, meta_101R['animal'], meta_101R['side'], 'dac', fig_title='Deltoid (acromial)')
makeMMA3plot(meta_101R['folder'], dfOS, meta_101R['animal'], meta_101R['side'], 'bic', fig_title='Biceps, fleshy origin', plot_suffix='_fsh')
makeMMA3plotFromMerged(meta_101R['folder'], onBone101Rdb, rob101R, meta_101R['animal'], meta_101R['side'], 'bic', fig_title='Biceps, bone origin')
makeMMA3plotFromMerged(meta_101R['folder'], onBone101Rdb, rob101R, meta_101R['animal'], meta_101R['side'], 'bic', fig_title='Biceps, bone origin and insertion',plot_suffix='_bb')



['pec_crn', 'pec_int', 'pec_cdl', 'trs', 'dsc', 'dac', 'bic']


AttributeError: 'str' object has no attribute 'loc'

In [275]:
rob92R

,model_bicr_length,model_bic_length,model_bra_length,model_cbb_length,model_dac_length,model_dcl_length,model_dsc_length,model_isp_length,model_lat_crn_length,model_lat_int_length,...,model_bra_elbo_Yma,model_trl_elbo_Yma,model_trm_elbo_Yma,model_trs_elbo_Yma,model_bicr_elbo_Zma,model_bic_elbo_Zma,model_bra_elbo_Zma,model_trl_elbo_Zma,model_trm_elbo_Zma,model_trs_elbo_Zma
2933,0.709074,0.760170,0.400153,0.140466,0.134291,0.101742,0.476189,0.302532,0.556592,0.766313,...,0.026283,-0.044419,-0.061338,0.078365,-0.055584,-0.059624,-0.037676,0.009704,0.004865,0.087060
2934,0.708801,0.760937,0.400198,0.139830,0.129869,0.101742,0.472042,0.299848,0.550906,0.765333,...,0.026158,-0.041431,-0.057283,0.079614,-0.056460,-0.060477,-0.038369,0.009730,0.005088,0.085659
2935,0.707274,0.757450,0.402275,0.138464,0.123290,0.101742,0.467444,0.297852,0.544623,0.765170,...,0.027212,-0.047439,-0.063673,0.073312,-0.056810,-0.060314,-0.038323,0.015420,0.012681,0.091266
2936,0.717722,0.770669,0.399951,0.144516,0.131952,0.101742,0.473882,0.303221,0.543694,0.764273,...,0.024162,-0.029133,-0.042922,0.084461,-0.055232,-0.059785,-0.037975,0.014409,0.011415,0.080676
2937,0.712611,0.766549,0.398272,0.141848,0.122751,0.101742,0.465796,0.298975,0.538037,0.767067,...,0.025513,-0.032812,-0.046983,0.081320,-0.056160,-0.060127,-0.038525,0.013951,0.011595,0.083601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3725,0.790839,0.837198,0.393140,0.179793,0.121735,0.101742,0.414896,0.307331,0.313302,0.609056,...,0.023134,-0.007824,-0.012586,0.086844,-0.055835,-0.060935,-0.039615,0.020201,0.022144,0.076700
3726,0.807203,0.857571,0.398543,0.186406,0.128087,0.101742,0.412464,0.311603,0.321711,0.632956,...,0.024169,-0.023072,-0.034202,0.085773,-0.055705,-0.060341,-0.038646,0.015501,0.013693,0.078676
3727,0.808225,0.857918,0.398767,0.185255,0.124224,0.101742,0.411075,0.314341,0.321089,0.616853,...,0.023168,-0.017625,-0.026955,0.087405,-0.055320,-0.060419,-0.038613,0.017575,0.016487,0.076922
3728,0.810232,0.859901,0.398542,0.186796,0.128585,0.101742,0.410730,0.311686,0.319509,0.603691,...,0.023087,-0.017589,-0.026713,0.087853,-0.055722,-0.060833,-0.038890,0.015701,0.013758,0.076218


#### 92R comparisons

In [272]:
meta_92R = {
                'animal':'92',
                'side':'R', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/92R'
           }
# onBone92R = dropMMA0s(pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/92R/92RonBone.csv', 
#                      index_col=0).rename(columns=reverse_alias_map).assign(animal=meta_92R['animal'],side=meta_92R['side']))
comparisons_92R = findComparisons(dfD, meta_92R['animal'], meta_92R['side'])

# makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'pec_crn', fig_title='Pectoralis (cranial)')
# makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'pec_int', fig_title='Pectoralis (intermediate)')
# makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'pec_cdl', fig_title='Pectoralis (caudal)')
# makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'dsc', fig_title='Deltoid (scapular)')
makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'dac', fig_title='Deltoid (acromial)')
# makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'bic', fig_title='Biceps (fleshy origin)', plot_suffix='_fsh')
# # makeMMA3plot(meta_92R['folder'], onBone92R, rob92R, meta_92R['animal'], meta_92R['side'], 'bic', fig_title='Biceps (bone origin)')



['pec_crn', 'pec_int', 'pec_cdl', 'dsc', 'dac', 'bic']


KeyError: 'None of [RangeIndex(start=0, stop=795, step=1)] are in the [index]'

In [42]:
fig, ax = plt.subplots(3,1,figsize=(16,8), sharey='all', sharex='all')
rob_muscle = 'pec_crn'
phil_muscle = 'pec_crn'

plt.suptitle(fig_title if fig_title else phil_muscle)
ax[0].plot(phil_df[phil_muscle+'_Xma'], linestyle='solid', color='red')
ax[0].plot(rob_df['model_'+rob_muscle+'_glen_Xma'], linestyle='dotted', color='red')
ax[1].plot(phil_df[phil_muscle+'_Yma'], linestyle='solid', color='green')
ax[1].plot(rob_df['model_'+rob_muscle+'_glen_Yma'], linestyle='dotted', color='green')
ax[2].plot(phil_df[phil_muscle+'_Zma'], linestyle='solid', color='blue')
ax[2].plot(rob_df['model_'+rob_muscle+'_glen_Zma'], linestyle='dotted', color='blue')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'fig_title' is not defined

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            391, 392, 393, 394, 395, 396, 397, 398, 399, 400],
           dtype='int64', name='frame', length=397)

,bic_Xma,bic_Yma,bic_Zma,trs_Xma,trs_Yma,trs_Zma,pec_crn_Xma,pec_crn_Yma,pec_crn_Zma,pec_cdl_Xma,pec_cdl_Yma,pec_cdl_Zma,dsc_Xma,dsc_Yma,dsc_Zma,dac_Xma,dac_Yma,dac_Zma,animal,side
frame,,,,,,,,,,,,,,,,,,,,
1,0.142382,-0.512631,-0.864181,-0.018809,0.428269,-0.052161,0.417270,-1.022282,-0.170944,0.752445,-0.719881,-0.161269,-0.206848,-0.297094,-0.020422,-0.471409,-0.556703,-0.030795,105,R
2,0.148745,-0.520437,-0.838388,-0.016865,0.413726,-0.049546,0.418496,-1.027371,-0.156553,0.752921,-0.727627,-0.151337,-0.215648,-0.304292,-0.016271,-0.474412,-0.566077,-0.023779,105,R
3,0.148991,-0.525497,-0.860786,-0.017806,0.412513,-0.060374,0.425198,-1.023407,-0.164783,0.755337,-0.725627,-0.167205,-0.212203,-0.293395,-0.010163,-0.467676,-0.563988,-0.012903,105,R
4,0.135206,-0.446779,-0.910592,-0.007162,0.554044,-0.004461,0.456078,-0.928528,-0.280829,0.761456,-0.683247,-0.453107,-0.253476,-0.015823,0.146411,-0.425208,-0.424554,0.238371,105,R
5,0.112330,-0.382050,-0.626565,0.036662,0.456864,0.094315,0.393370,-0.612043,-0.346338,0.700619,-0.482946,-0.706229,-0.446289,0.552117,0.413873,-0.533959,-0.007020,0.593451,105,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,0.108533,-0.656273,-0.336408,0.049488,0.476514,0.026528,0.027039,-1.085067,0.314349,0.285337,-1.048261,0.220708,-0.561567,0.211812,0.116713,-0.820779,0.139361,0.221257,105,R
397,0.105354,-0.620886,-0.340996,0.055688,0.509596,0.028322,-0.003773,-1.066897,0.328293,0.245936,-1.032476,0.225006,-0.567586,0.264487,0.129761,-0.826450,0.189227,0.248960,105,R
398,0.154829,-0.743844,-0.465751,0.029027,0.433108,-0.011065,0.172726,-1.144500,0.191454,0.463860,-1.052986,0.160146,-0.475671,-0.020288,0.028510,-0.752432,-0.062478,0.050422,105,R


#### 105R comparisons

In [412]:
meta_105R = {
                'animal':'105',
                'side':'R', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/105R'
           }
plt.close('all')

new105R = pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/105R/105Ronbone.csv', 
                     index_col=0)
onBone105R = dropMMA0s(new105R.rename(columns=reverse_alias_map).assign(animal=meta_105R['animal'],side=meta_105R['side']))
frames = dfOS.loc[(dfOS['animal']=='105')&(dfOS['side']=='R')]['frame']
onBone105R = onBone105R.loc[pd.Index(frames)]

comparisons_105R = findComparisons(dfS, meta_105R['animal'], meta_105R['side'])

makeMMA3plot(meta_105R['folder'],onBone105R, rob105R.reset_index(), meta_105R['animal'], meta_105R['side'], 'pec_crn',  fig_title='Pectoralis (cranial)')
makeMMA3plot(meta_105R['folder'],onBone105R, rob105R.reset_index(), meta_105R['animal'], meta_105R['side'], 'pec_cdl',  fig_title='Pectoralis (caudal)')
makeMMA3plot(meta_105R['folder'],onBone105R, rob105R.reset_index(), meta_105R['animal'], meta_105R['side'], 'trs',  fig_title='Triceps (scapular)')
makeMMA3plot(meta_105R['folder'],onBone105R, rob105R.reset_index(), meta_105R['animal'], meta_105R['side'], 'dsc',  fig_title='Deltoid (scapular)')
makeMMA3plot(meta_105R['folder'],onBone105R, rob105R.reset_index(), meta_105R['animal'], meta_105R['side'], 'dac',  fig_title='Deltoid (acromial)')
makeMMA3plot(meta_105R['folder'],onBone105R, rob105R.reset_index(), meta_105R['animal'], meta_105R['side'], 'bic',  fig_title='Biceps')


# makeMMA3plot(meta_85R['folder'], onBone85R, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_crn',  fig_title='Latissimus (cranial, bone insertion)')
# makeMMA3plot(meta_85R['folder'], onBone85R, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_cdl',  fig_title='Latissimus (caudal, bone insertion)')

# makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_crn',  fig_title='Latissimus (cranial, fleshy insertion)',plot_suffix='_fsh')
# makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_cdl',  fig_title='Latissimus (caudal, fleshy insertion)',plot_suffix='_fsh')

# makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'trs',  fig_title='Triceps (scapular, fleshy origin)',plot_suffix='_fsh')
# makeMMA3plot(meta_85R['folder'], onBone85R, rob85R, meta_85R['animal'], meta_85R['side'], 'trs',  fig_title='Triceps (scapular, bone origin)')

['pec_crn', 'pec_cdl', 'trs', 'dsc', 'dac', 'bic']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>], dtype=object)

#### 108L comparisons

In [150]:
plt.close('all')
meta_108L = {
                'animal':'108',
                'side':'L', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/108L'
           }

new108L = pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/108L/108Lonbone.csv', 
                     index_col=0)
onBone108L = dropMMA0s(new108L.rename(columns=reverse_alias_map).assign(animal=meta_108L['animal'],side=meta_108L['side']))
frames = dfOS.loc[(dfOS['animal']=='108')&(dfOS['side']=='L')]['frame']
onBone108L = onBone108L.loc[pd.Index(frames)]

comparisons_108L = findComparisons(dfS, meta_108L['animal'], meta_108L['side'])

makeMMA3plotFromMerged(meta_108L['folder'], dfOS, meta_108L['animal'], meta_108L['side'], 'pec_crn',  fig_title='Pectoralis (cranial)')
makeMMA3plotFromMerged(meta_108L['folder'], dfOS, meta_108L['animal'], meta_108L['side'], 'pec_cdl',  fig_title='Pectoralis (caudal)')
makeMMA3plot(meta_108L['folder'], onBone108L, rob108L.reset_index(), meta_108L['animal'], meta_108L['side'], 'trs',  fig_title='Triceps (scapular)')

makeMMA3plot(meta_108L['folder'], onBone108L,  rob108L.reset_index(),meta_108L['animal'], meta_108L['side'], 'dsc',  fig_title='Deltoid (scapular)')

makeMMA3plot(meta_108L['folder'], onBone108L, rob108L.reset_index(), meta_108L['animal'], meta_108L['side'], 'dac',  fig_title='Deltoid (acromial)')


makeMMA3plotFromMerged(meta_108L['folder'], dfOS, meta_108L['animal'], meta_108L['side'], 'bic',  fig_title='Biceps')


['pec_crn', 'pec_cdl', 'trs', 'dsc', 'dac', 'bic']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>], dtype=object)

#### 108R comparisons

In [138]:
meta_108R = {
                'animal':'108',
                'side':'R', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/108R'
           }

new108R = pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/108R/108Ronbone.csv', 
                     index_col=0)
onBone108R = dropMMA0s(new108R.rename(columns=reverse_alias_map).assign(animal=meta_108R['animal'],side=meta_108R['side']))
frames = dfOS.loc[(dfOS['animal']=='108')&(dfOS['side']=='R')]['frame']
onBone108R = onBone108R.loc[pd.Index(frames)]


plt.close('all')
comparisons_108R = findComparisons(dfS, meta_108R['animal'], meta_108R['side'])

makeMMA3plotFromMerged(meta_108R['folder'], dfOS, meta_108R['animal'], meta_108R['side'], 'pec_crn',  fig_title='Pectoralis (cranial)')
makeMMA3plotFromMerged(meta_108R['folder'], dfOS, meta_108R['animal'], meta_108R['side'], 'pec_cdl',  fig_title='Pectoralis (caudal)')
makeMMA3plot(meta_108R['folder'], onBone108R,  rob108R.reset_index(),meta_108R['animal'], meta_108R['side'], 'trs',  fig_title='Triceps (scapular)')

makeMMA3plotFromMerged(meta_108R['folder'], dfOS, meta_108R['animal'], meta_108R['side'], 'dsc',  fig_title='Deltoid (scapular)')

makeMMA3plotFromMerged(meta_108R['folder'], dfOS, meta_108R['animal'], meta_108R['side'], 'dac',  fig_title='Deltoid (acromial)')

makeMMA3plot(meta_108R['folder'], onBone108R,  rob108R.reset_index(),meta_108R['animal'], meta_108R['side'], 'bic',  fig_title='Biceps (scapular)')




['pec_crn', 'pec_cdl', 'trs', 'dsc', 'dac', 'bic']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>], dtype=object)

#### 126R comparisons

In [ ]:
# new85Rlat = pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/85R/newlat85R.csv', 
#                      index_col=0)
# new85Rtrs = pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/85R/newtrs85R.csv', 
#                      index_col=0)

plt.close('all')
meta_126R = {
                'animal':'126',
                'side':'R', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/126R'
           }
# onBone105R = dropMMA0s(pd.concat([new85Rlat,new85Rtrs],axis=1).rename(columns=reverse_alias_map).assign(animal=meta_85R['animal'],side=meta_85R['side']))

comparisons_126R = findComparisons(dfS, meta_126R['animal'], meta_126R['side'])

makeMMA3plotFromMerged(meta_126R['folder'], dfOS, meta_126R['animal'], meta_126R['side'], 'pec_crn',  fig_title='Pectoralis (cranial)')
makeMMA3plotFromMerged(meta_126R['folder'], dfOS, meta_126R['animal'], meta_126R['side'], 'pec_cdl',  fig_title='Pectoralis (caudal)')
makeMMA3plotFromMerged(meta_126R['folder'], dfOS, meta_126R['animal'], meta_126R['side'], 'trs',  fig_title='Triceps (scapular)')
makeMMA3plotFromMerged(meta_126R['folder'], dfOS, meta_126R['animal'], meta_126R['side'], 'dsc',  fig_title='Deltoid (scapular)')
makeMMA3plotFromMerged(meta_126R['folder'], dfOS, meta_126R['animal'], meta_126R['side'], 'dac',  fig_title='Deltoid (acromial)')
makeMMA3plotFromMerged(meta_126R['folder'], dfOS, meta_126R['animal'], meta_126R['side'], 'bic',  fig_title='Biceps')



#### 126L comparisons

In [160]:
# new85Rlat = pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/85R/newlat85R.csv', 
#                      index_col=0)
# new85Rtrs = pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/85R/newtrs85R.csv', 
#                      index_col=0)
plt.close('all')
meta_126L = {
                'animal':'126',
                'side':'L', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/126L'
           }
# onBone105R = dropMMA0s(pd.concat([new85Rlat,new85Rtrs],axis=1).rename(columns=reverse_alias_map).assign(animal=meta_85R['animal'],side=meta_85R['side']))

comparisons_126L = findComparisons(dfS, meta_126L['animal'], meta_126L['side'])

makeMMA3plotFromMerged(meta_126L['folder'], dfOS, meta_126L['animal'], meta_126L['side'], 'pec_crn',  fig_title='Pectoralis (cranial)')
makeMMA3plotFromMerged(meta_126L['folder'], dfOS, meta_126L['animal'], meta_126L['side'], 'pec_cdl',  fig_title='Pectoralis (caudal)')
makeMMA3plotFromMerged(meta_126L['folder'], dfOS, meta_126L['animal'], meta_126L['side'], 'trs',  fig_title='Triceps (scapular)')
makeMMA3plotFromMerged(meta_126L['folder'], dfOS, meta_126L['animal'], meta_126L['side'], 'dsc',  fig_title='Deltoid (scapular)')
makeMMA3plotFromMerged(meta_126L['folder'], dfOS, meta_126L['animal'], meta_126L['side'], 'dac',  fig_title='Deltoid (acromial)')
makeMMA3plotFromMerged(meta_126L['folder'], dfOS, meta_126L['animal'], meta_126L['side'], 'bic',  fig_title='Biceps')



['pec_crn', 'pec_cdl', 'trs', 'dsc', 'dac', 'bic']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>], dtype=object)

In [348]:

# def makeMMA3plot(save_dir, phil_df, rob_df, animal, side, phil_muscle, rob_muscle=None, fig_title=None, plot_suffix=''):
#     phil_df = phil_df.loc[(phil_df['animal']==animal)&(phil_df['side']==side)].reset_index()
#     rob_df = rob_df.reset_index().loc[phil_df.index]
#     fig, ax = plt.subplots(3,1,figsize=(16,8), sharey='all', sharex='all')
#     rob_muscle = rob_muscle if rob_muscle else phil_muscle
#     plt.suptitle(fig_title if fig_title else phil_muscle)
#     ax[0].plot(phil_df[phil_muscle+'_Xma'], linestyle='solid', color='red')
#     ax[0].plot(rob_df['model_'+rob_muscle+'_glen_Xma'], linestyle='dotted', color='red')
#     ax[1].plot(phil_df[phil_muscle+'_Yma'], linestyle='solid', color='green')
#     ax[1].plot(rob_df['model_'+rob_muscle+'_glen_Yma'], linestyle='dotted', color='green')
#     ax[2].plot(phil_df[phil_muscle+'_Zma'], linestyle='solid', color='blue')
#     ax[2].plot(rob_df['model_'+rob_muscle+'_glen_Zma'], linestyle='dotted', color='blue')
#     plt.tight_layout()
#     plt.savefig(os.path.join(save_dir,phil_muscle+plot_suffix+'.png'), dpi=150, transparent=True)
#     return ax

def makeMMA3plotFromMerged(save_dir, phil_df, animal, side, phil_muscle, rob_muscle=None, fig_title=None, plot_suffix=''):
    df = phil_df.loc[(phil_df['animal']==animal)&(phil_df['side']==side)]
    fig, ax = plt.subplots(3,1,figsize=(16,8), sharey='all', sharex='all')
    rob_muscle = rob_muscle if rob_muscle else phil_muscle
    plt.suptitle(fig_title if fig_title else phil_muscle)
    ax[0].plot(df[phil_muscle+'_Xma'], linestyle='solid', color='red')
    ax[0].plot(df['model_'+rob_muscle+'_glen_Xma'], linestyle='dotted', color='red')
    ax[1].plot(df[phil_muscle+'_Yma'], linestyle='solid', color='green')
    ax[1].plot(df['model_'+rob_muscle+'_glen_Yma'], linestyle='dotted', color='green')
    ax[2].plot(df[phil_muscle+'_Zma'], linestyle='solid', color='blue')
    ax[2].plot(df['model_'+rob_muscle+'_glen_Zma'], linestyle='dotted', color='blue')
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir,phil_muscle+plot_suffix+'.png'), dpi=150, transparent=True)
    return ax



makeMMA3plotFromMerged('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/test',dfOS, '108','R','dac')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>], dtype=object)

In [367]:
dfOS126L=dfOS.loc[(dfOS['animal']=='126')&(dfOS['side']=='L')]
# dfOS126L
dfOS126L.loc[dfOS126L['trs_Yma']==dfOS126L['trs_Yma'].min()]['frame']

6802    81
Name: frame, dtype: int64

In [240]:
rside = dfS.loc[(dfS['animal']=='108')&(dfS['side']=='R')]
lside = dfS.loc[(dfS['animal']=='108')&(dfS['side']=='L')]
lside['pec_crn_Zma']

5922    0.383305
5923    0.578302
5924    0.776948
5925    0.699834
5926    0.332545
          ...   
6317    0.764546
6318    0.910417
6319    0.810314
6320    0.447163
6321    0.054010
Name: pec_crn_Zma, Length: 400, dtype: float64

In [344]:
rob108R

,model_bicr_length,model_bicr2_length,model_bic_length,model_brar_length,model_bra_length,model_cbb_length,model_cbl_length,model_dac_length,model_dcl_length,model_dsc_length,...,model_trm_elbo_Yma,model_trs_elbo_Yma,model_bicr_elbo_Zma,model_bicr2_elbo_Zma,model_bic_elbo_Zma,model_brar_elbo_Zma,model_bra_elbo_Zma,model_trl_elbo_Zma,model_trm_elbo_Zma,model_trs_elbo_Zma
6322,5.844176,5.801231,5.751485,2.400858,2.323761,2.052258,4.626585,1.319160,2.585024,2.742779,...,-0.255402,-0.255402,0.484129,0.441180,0.342188,-0.050573,-0.077038,0.012796,0.012796,0.012796
6323,5.808602,5.745098,5.661579,2.400234,2.298571,2.113119,4.662938,1.271257,2.529592,2.658944,...,-0.309768,-0.309768,0.482368,0.525306,0.322013,-0.019040,-0.014591,-0.085328,-0.085328,-0.085328
6324,5.942118,5.886577,5.828961,2.387154,2.289637,2.048879,4.659665,1.304166,2.552623,2.745799,...,-0.143250,-0.143250,0.466900,0.387362,0.366406,-0.116476,-0.137474,0.064800,0.064800,0.064800
6325,5.425705,5.432838,5.329594,2.348871,2.355713,1.496255,4.167864,1.343867,2.343802,3.333876,...,-0.130718,-0.130718,-0.376268,0.458615,0.371229,-0.148015,-0.115984,0.165185,0.165185,0.165185
6326,4.837726,5.011617,4.949483,2.297327,2.418273,1.204569,3.910440,1.260934,2.141317,3.445789,...,-0.093194,-0.093194,-0.323044,0.514052,-0.460861,-0.169369,-0.152293,0.297063,0.297063,0.297063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6717,4.935327,4.869090,4.778772,2.358966,2.319706,1.264040,3.796424,0.711657,1.452722,2.640418,...,-0.170127,-0.170127,0.496014,0.426913,0.367060,-0.127301,-0.113748,0.095743,0.095743,0.095743
6718,5.000154,4.990634,4.897151,2.349721,2.313331,1.313801,3.849419,0.777617,1.455602,2.705228,...,-0.122878,-0.122878,-0.394176,0.430550,0.378337,-0.151299,-0.135204,0.130279,0.130279,0.130279
6719,5.135449,5.255717,5.088902,2.331545,2.310933,1.471906,4.020121,0.861500,1.551275,2.755174,...,-0.043770,-0.043770,-0.386033,0.435680,0.364530,-0.200208,-0.140269,0.196746,0.196746,0.196746
6720,5.361519,5.516965,5.346515,2.324010,2.280547,1.670924,4.273058,0.917682,1.709078,2.740249,...,-0.010887,-0.010887,-0.398057,0.414422,0.376128,-0.219090,-0.161784,0.191054,0.191054,0.191054


In [247]:
rob108L = dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/108R', 'left')
rob108L['model_pec_crn_glen_Zma']

0     -1.236250
1     -1.110091
2     -0.400332
3      2.069416
4      5.319904
         ...   
395   -3.775061
396   -3.155047
397   -1.417017
398    2.704211
399    7.751887
Name: model_pec_crn_glen_Zma, Length: 400, dtype: float64

In [237]:
index108L = dfS.loc[(dfS['animal']=='108')&(dfS['side']=='L')].index

In [233]:
rob_muscle='pec_crn'
phil_df = rside
rob_df = rob108L
plt.close('all')
rside = dfS.loc[(dfS['animal']=='108')&(dfS['side']=='R')].reset_index()
lside = dfS.loc[(dfS['animal']=='108')&(dfS['side']=='L')].reset_index()
fig, ax = plt.subplots(3,1,figsize=(16,8), sharey='all', sharex='all')
rob_muscle = rob_muscle if rob_muscle else phil_muscle

ax[0].plot(phil_df[phil_muscle+'_Xma'], linestyle='solid', color='red')
ax[0].plot(rob_df['model_'+rob_muscle+'_glen_Xma'], linestyle='dotted', color='red')

ax[1].plot(phil_df[phil_muscle+'_Yma'], linestyle='solid', color='green')
ax[1].plot(rob_df['model_'+rob_muscle+'_glen_Yma'], linestyle='dotted', color='green')

ax[2].plot(phil_df[phil_muscle+'_Zma'], linestyle='solid', color='blue')
ax[2].plot(rob_df['model_'+rob_muscle+'_glen_Zma'], linestyle='dotted', color='blue')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

- Rob from Phil is always the reciprocal/inverse of Phil from Rob
- Offset between proximal JCS is always the same
- Can get offset from proximal JCS of any frame as long as we use the correct multiplication order
- Correct order for our purposes (A rotation to match target B rotation) is Mat_B * Mat_A.inverse()
- This is different from the relative rotation (B rotation from A rotation), which is Mat_A.inverse() * Mat_B
- get Rob from Phil JCS rotations by getting zyx order Eulers from MTransformationMatrix of robFromPhil
- get Phil from Rob JCS rotations by getting xyz order Eulers from MTransformationMatrix of philFromRob
    - xyz rob from phil = [4.700399999999999, 11.080600000000002, -30.509199999999996] zyx rob from phil = [9.737539736905026, 7.089631542779325, -31.570056869286752]
    - zyx phil from rob = [-9.737539736905022, -7.089631542779322, 31.570056869286752] xyz phil from rob = [-4.700399999999999, -11.0806, 30.509199999999996]
- all good so far, am able to use these matrices to snap rob jcs to phil jcs and vice versa. But unable to match JCS locator data. 
- eventually able to recreate JCS locator data by going step by step following JAxesDual source code, end up with same values as negative of robDistFromRobProx25_rot.as_euler('XYZ',degrees=True). ONLY WORKS FOR KEYED FRAMES! for interpolated frames, channel editor gives different value from script
- def checkFrame():
    oldMatrix = xform('robJCS', ws=1, q=1, m=1)
    oldMatrix1 = getAttr('robJCS.worldMatrix')
    axesDurChild = dt.Matrix(xform('robJCS', ws=1, q=1, m=1)).transpose()
    axesDurParent = dt.Matrix(xform('robJCSProx', ws=1, q=1, m=1)).transpose()
    outChild = axesDurParent.transpose()*axesDurChild
    newMatrix = outChild.transpose()
    test = dt.TransformationMatrix(newMatrix)
    testDeg = [math.degrees(rad) for rad in test.euler]
    return testDeg
- need to use 'XYZ' to specify intrinsic rotations when importing eulers from Maya output, and need to flip sign of rotations or else transpose the resulting matrices.
- errors before: didn't know how to bring in eulers to scipy, possibly frame 1 keyed JCS, also interpolated frames



### Check relationship between proretraction and crancaudal translation of glenohumeral
pretty strong correlation if we throw out 108, also tz~ry

In [ ]:
dfSforCorr = dfS.filter(regex='glenoid')
dfSforCorr = dfSforCorr.dropna(axis=1, how='all')
corr = dfSforCorr.corr()
corr.style.background_gradient(cmap='bwr')


In [2460]:
print("Tegu glenoid max rx",dfS['glenoid.rx'].max())





print("Opossum glenoid max rx",alpha_d_glenoid['volumes']['20'].values)

Tegu glenoid max rx 55.39564647
Opossum glenoid max rx [442580.2398631696]


In [2460]:
print("Tegu glenoid max rx",dfS['glenoid.rx'].max())





print("Opossum glenoid max rx",alpha_d_glenoid['volumes']['20'].values)

Tegu glenoid max rx 55.39564647
Opossum glenoid max rx [442580.2398631696]


In [157]:
def Rgb6DOF(df, species, joint, xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True):
    df = df.copy()
    dfspecies = df.filter(items=['species'])
    if not normalizeAcrossSpecies: 
        df = df[df['species']==species[0]]
    df = df.replace(np.NaN, 0)
    df = df.filter(like=joint+'.t')
    tX = df.filter(like=joint+'.tx')
    tY = df.filter(like=joint+'.ty')
    tZ = df.filter(like=joint+'.tz')
    pos_tX = np.where(tX > 0, tX, 0)
    pos_tY = np.where(tY > 0, tY, 0)
    pos_tZ = np.where(tZ > 0, tZ, 0)
    neg_tX = np.where(tX < 0, abs(tX), 0)
    neg_tY = np.where(tY < 0, abs(tY), 0)
    neg_tZ = np.where(tZ < 0, abs(tZ), 0)
    df=df.drop([tX.columns[0],tY.columns[0],tZ.columns[0]], axis=1)
    tMax = df.max().max()
    df=df/tMax
    print(pos_tX)
    df['tX'] = pos_tX if xPos else neg_tX
    df['tY'] = pos_tY if yPos else neg_tY
    df['tZ'] = pos_tZ if zPos else neg_tZ
    totalXYZ = df[['tX','tY','tZ']].sum(axis=1)
    df['scale'] = preprocessing.MinMaxScaler().fit_transform(totalXYZ.values.reshape(-1, 1))
    if normalizeAcrossSpecies:
        df = dfspecies.join(df)
        df = df[df['species']==species]

    return(df[['tX','tY','tZ','scale']])

dfS_6dofPos = Rgb6DOF(df, 'Tegu', 'glenoid', xPos=True, yPos=True, zPos=True)
dfD_6dofPos = Rgb6DOF(df, 'Opossum', 'glenoid', xPos=True, yPos=True, zPos=True)
dfS_6dofNeg = Rgb6DOF(df, 'Tegu', 'glenoid', xPos=False, yPos=False, zPos=False)
dfD_6dofNeg = Rgb6DOF(df, 'Opossum', 'glenoid', xPos=False, yPos=False, zPos=False)


startView = viewXZ


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = glenoid_ranges.values()


plt.close('all')


fig = plt.figure(figsize=[12,12], constrained_layout=True)
ax0 = fig.add_subplot(221, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(222, projection='3d', proj_type = 'ortho')
ax2 = fig.add_subplot(223, projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(224, projection='3d', proj_type = 'ortho')



format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=startView)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=startView)
format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=startView)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=startView)

addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax2, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax3, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=dfS_6dofPos['scale']*25, c=dfS_6dofPos[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax1.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=dfD_6dofPos['scale']*25, c=dfD_6dofPos[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax2.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=dfS_6dofNeg['scale']*25, c=dfS_6dofNeg[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax3.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=dfD_6dofNeg['scale']*25, c=dfD_6dofNeg[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')

connection4 = fig.canvas.mpl_connect('motion_notify_event', on_move4)

plt.show()



[[0.        ]
 [0.        ]
 [0.02450193]
 ...
 [0.09374336]
 [0.18028628]
 [0.15707111]]
[[0.        ]
 [0.        ]
 [0.02450193]
 ...
 [0.09374336]
 [0.18028628]
 [0.15707111]]
[[0.        ]
 [0.        ]
 [0.02450193]
 ...
 [0.09374336]
 [0.18028628]
 [0.15707111]]
[[0.        ]
 [0.        ]
 [0.02450193]
 ...
 [0.09374336]
 [0.18028628]
 [0.15707111]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: RGBA values should be within 0-1 range

### In vivo ROM vs ex vivo ROM

### Import in vivo data

In [12]:
df_invivo = pd.read_csv('./invivo_for_exvivo_comparison.csv',index_col=None).filter(regex='uid|frame|species|individual|condition|glenoid.rotate|elbow.rotate|run')
df_invivo.columns = ['percentage','frame','species','individual','condition','glenoid.rx', 'glenoid.ry', 'glenoid.rz', 'elbow.rx', 'elbow.ry', 'elbow.rz']
df_invivo['percentage'] = df_invivo['percentage'].str.rsplit('_',1).str[-1]
glenoid_radRy = np.radians(df_invivo['glenoid.ry'])
elbow_radRy = np.radians(df_invivo['glenoid.ry'])
glenoid_cosRy = np.cos(glenoid_radRy)
elbow_cosRy = np.cos(elbow_radRy)
df_invivo['glenoid.crz'] = glenoid_cosRy.multiply(df_invivo['glenoid.rz'])
df_invivo['elbow.crz'] = elbow_cosRy.multiply(df_invivo['elbow.rz'])

df_invivo_stand = df_invivo.loc[df_invivo['condition']=='stand']
df_invivo_walk = df_invivo.loc[df_invivo['condition']!='stand']

df_invivo_S = df_invivo_walk.loc[df_invivo_walk['species']=='Tegu']
df_invivo_D = df_invivo_walk.loc[df_invivo_walk['species']=='Opossum']


In [13]:

startView = viewXY



glenoid_ranges = getRotationRanges(dfS, dfD, 'glenoid',30)
elbow_ranges = getRotationRanges(dfS, dfD, 'elbow',30)
comp_ranges = {key: max(glenoid_ranges[key], elbow_ranges[key]) for key in glenoid_ranges}
maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = comp_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'


fig = plt.figure(figsize=[12,12], constrained_layout=True)
ax0 = fig.add_subplot(3,4,1, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(3,4,2, projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(3,4,5, projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(3,4,6, projection='3d', proj_type = 'ortho')

ax4 = fig.add_subplot(3,4,9, projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(3,4,10, projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(3,4,3, projection='3d', proj_type = 'ortho')
ax7 = fig.add_subplot(3,4,4, projection='3d', proj_type = 'ortho')

ax8 = fig.add_subplot(3,4,7, projection='3d', proj_type = 'ortho')
ax9 = fig.add_subplot(3,4,8, projection='3d', proj_type = 'ortho')

ax10 = fig.add_subplot(3,4,11, projection='3d', proj_type = 'ortho')
ax11 = fig.add_subplot(3,4,12, projection='3d', proj_type = 'ortho')

format3dPlot(ax0, 'Tegu glenoid', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
format3dPlot(ax1, 'Opossum glenoid', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)
format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax6, 'Tegu elbow', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')
format3dPlot(ax7, 'Opossum elbow', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')
format3dPlot(ax8, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY, joint='elbow')
format3dPlot(ax9, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY, joint='elbow')
format3dPlot(ax10, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')
format3dPlot(ax11, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')

addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)


ax0.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax1.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)
ax2.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax3.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)
ax4.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax5.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)

ax6.plot_trisurf(dfS_elbow['elbow.rx'],dfS_elbow['elbow.ry'],alphaTriElbow_S,dfS_elbow['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax7.plot_trisurf(dfD_elbow['elbow.rx'],dfD_elbow['elbow.ry'],alphaTriElbow_D,dfD_elbow['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)
ax8.plot_trisurf(dfS_elbow['elbow.rx'],dfS_elbow['elbow.ry'],alphaTriElbow_S,dfS_elbow['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax9.plot_trisurf(dfD_elbow['elbow.rx'],dfD_elbow['elbow.ry'],alphaTriElbow_D,dfD_elbow['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)
ax10.plot_trisurf(dfS_elbow['elbow.rx'],dfS_elbow['elbow.ry'],alphaTriElbow_S,dfS_elbow['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax11.plot_trisurf(dfD_elbow['elbow.rx'],dfD_elbow['elbow.ry'],alphaTriElbow_D,dfD_elbow['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)

good_tegu = df_invivo_S

# .loc[((df_invivo_S['individual']=='Tegu 2')&(df_invivo_S['condition']=='hi'))]


df_invivo_S_filt = good_tegu
df_invivo_D_filt = df_invivo_D.loc[(df_invivo_D['condition']=='hi')]

#.loc[(df_invivo_S['individual']!='Tegu 3')&(df_invivo_S['condition']!='lo')]
# df_invivo_S_filt = df_invivo_S.loc[(df_invivo_S['percentage'].astype('int')>40)]
# df_invivo_D_filt = df_invivo_D.loc[(df_invivo_D['individual']!='Opossum 3')&(df_invivo_D['condition']!='lo')]


# .loc[(df_invivo_S['individual']=='Tegu 3')&(df_invivo_S['condition']=='lo')]


# df_invivo_D = df_invivo_D.loc[(df_invivo_D['individual']!='Opossum 3')&(df_invivo_D['condition']!='lo')]


ax0.scatter(df_invivo_S_filt['glenoid.rx'],df_invivo_S_filt['glenoid.ry'],df_invivo_S_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax1.scatter(df_invivo_D_filt['glenoid.rx'],df_invivo_D_filt['glenoid.ry'],df_invivo_D_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax2.scatter(df_invivo_S_filt['glenoid.rx'],df_invivo_S_filt['glenoid.ry'],df_invivo_S_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax3.scatter(df_invivo_D_filt['glenoid.rx'],df_invivo_D_filt['glenoid.ry'],df_invivo_D_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax4.scatter(df_invivo_S_filt['glenoid.rx'],df_invivo_S_filt['glenoid.ry'],df_invivo_S_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax5.scatter(df_invivo_D_filt['glenoid.rx'],df_invivo_D_filt['glenoid.ry'],df_invivo_D_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)

ax6.scatter(df_invivo_S_filt['elbow.rx'],df_invivo_S_filt['elbow.ry'],df_invivo_S_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax7.scatter(df_invivo_D_filt['elbow.rx'],df_invivo_D_filt['elbow.ry'],df_invivo_D_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax8.scatter(df_invivo_S_filt['elbow.rx'],df_invivo_S_filt['elbow.ry'],df_invivo_S_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax9.scatter(df_invivo_D_filt['elbow.rx'],df_invivo_D_filt['elbow.ry'],df_invivo_D_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax10.scatter(df_invivo_S_filt['elbow.rx'],df_invivo_S_filt['elbow.ry'],df_invivo_S_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax11.scatter(df_invivo_D_filt['elbow.rx'],df_invivo_D_filt['elbow.ry'],df_invivo_D_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)

# connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Supp. figure: bad wraps

In [214]:
alphaVertShoulder_S_crit,alphaEdgeShoulder_S_crit,alphaTriShoulder_S_crit = alpha_shape_3D(dfS.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),8)
alphaVertShoulder_D_crit,alphaEdgeShoulder_D_crit,alphaTriShoulder_D_crit = alpha_shape_3D(dfD.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),9)

alphaVertShoulder_S_10,alphaEdgeShoulder_S_10,alphaTriShoulder_S_10 = alpha_shape_3D(dfS.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),10)
alphaVertShoulder_D_10,alphaEdgeShoulder_D_10,alphaTriShoulder_D_10 = alpha_shape_3D(dfD.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),10)

alphaVertShoulder_S_20,alphaEdgeShoulder_S_20,alphaTriShoulder_S_20 = alpha_shape_3D(dfS.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),20)
alphaVertShoulder_D_20,alphaEdgeShoulder_D_20,alphaTriShoulder_D_20 = alpha_shape_3D(dfD.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),20)

alphaVertShoulder_S_30,alphaEdgeShoulder_S_30,alphaTriShoulder_S_30 = alpha_shape_3D(dfS.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),30)
alphaVertShoulder_D_30,alphaEdgeShoulder_D_30,alphaTriShoulder_D_30 = alpha_shape_3D(dfD.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),30)

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'

maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()
fig = plt.figure(figsize=[10,10], constrained_layout=True)
gs2x2 = gridspec.GridSpec(2,2, figure=fig)
gs1x2_crit = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs2x2[0,0])
gs1x2_10 = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs2x2[0,1])
gs1x2_20 = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs2x2[1,0])
gs1x2_30 = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs2x2[1,1])

ax_crit_S = fig.add_subplot(gs1x2_crit[0], projection='3d', proj_type = 'ortho')
ax_crit_D = fig.add_subplot(gs1x2_crit[1], projection='3d', proj_type = 'ortho')
ax_10_S = fig.add_subplot(gs1x2_10[0], projection='3d', proj_type = 'ortho')
ax_10_D = fig.add_subplot(gs1x2_10[1], projection='3d', proj_type = 'ortho')
ax_20_S = fig.add_subplot(gs1x2_20[0], projection='3d', proj_type = 'ortho')
ax_20_D = fig.add_subplot(gs1x2_20[1], projection='3d', proj_type = 'ortho')
ax_30_S = fig.add_subplot(gs1x2_30[0], projection='3d', proj_type = 'ortho')
ax_30_D = fig.add_subplot(gs1x2_30[1], projection='3d', proj_type = 'ortho')

for ax, tri,alphaName in zip([ax_crit_S, ax_10_S, ax_20_S, ax_30_S],[alphaTriShoulder_S_crit, alphaTriShoulder_S_10, alphaTriShoulder_S_20, alphaTriShoulder_S_30],['Tegu critical (8)','Tegu 10','Tegu 20','Tegu 30']):
    format3dPlot(ax, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
    addCosGrid(ax, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
    ax.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],tri,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
    ax.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=0.25, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
    ax.set_title(alphaName, fontproperties=roboto_bold, fontsize=12)
for ax, tri,alphaName in zip([ax_crit_D, ax_10_D, ax_20_D, ax_30_D],[alphaTriShoulder_D_crit, alphaTriShoulder_D_10, alphaTriShoulder_D_20, alphaTriShoulder_D_30],['Opossum critical (9)','Opossum 10','Opossum 20','Opossum 30']):
    format3dPlot(ax, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
    addCosGrid(ax, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
    ax.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],tri,dfD['glenoid.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
    ax.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=0.25, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
    ax.set_title(alphaName, fontproperties=roboto_bold, fontsize=12)
    
for axis in [ax_crit_S, ax_crit_D, ax_10_S, ax_10_D, ax_20_S, ax_20_D, ax_30_S, ax_30_D]:
    axis.axis('off')
    axis.dist = 6
plt.show()

plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/supp_badwraps.png', dpi=300, transparent=True)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### DONT USE: FIGURE: ROM envelopes: combined 

In [ ]:
# scatterSize=0.15
# scatterAlpha=0.1
# scatterTeal = '#3CA9A6'
# scatterGold = '#FFCA4E'

# shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }
# combined_ranges = {key: (shoulderElbow_ranges[key] if (abs(shoulderElbow_ranges[key])>abs(scap_ranges[key])) else scap_ranges[key])  for key in shoulderElbow_ranges.keys() }


# shoulderElbow_ranges_translation = {key: (elbow_ranges_t[key] if (abs(elbow_ranges_t[key])>abs(glenoid_ranges_t[key])) else glenoid_ranges_t[key])  for key in elbow_ranges_t.keys() }
# combined_ranges_translation = {key: (shoulderElbow_ranges_translation[key] if (abs(shoulderElbow_ranges_translation[key])>abs(scap_ranges_t[key])) else scap_ranges_t[key])  for key in shoulderElbow_ranges_translation.keys() }


# maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = combined_ranges.values()

# maxtx, mintx, maxty, minty, maxtz, mintz = combined_ranges_translation.values()


# plt.close('all')
# plt.rcParams['grid.linewidth'] = 0
# plt.rcParams['grid.color'] = 'lightgrey'

# fig = plt.figure(figsize=[18,10], constrained_layout=True)

# gs3col = gridspec.GridSpec(1,3, figure=fig)
# gs3row_scap = gridspec.GridSpecFromSubplotSpec(3, 1, subplot_spec=gs3col[0])
# gs3row_glen = gridspec.GridSpecFromSubplotSpec(3, 1, subplot_spec=gs3col[1])
# gs3row_elb = gridspec.GridSpecFromSubplotSpec(3, 1, subplot_spec=gs3col[2])

# gs2col_scap_YZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_scap[0])
# gs2col_scap_YX = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_scap[1])
# gs2col_scap_XZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_scap[2])

# gs2col_glen_YZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_glen[0])
# gs2col_glen_YX = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_glen[1])
# gs2col_glen_XZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_glen[2])

# gs2col_elb_YZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_elb[0])
# gs2col_elb_YX = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_elb[1])
# gs2col_elb_XZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_elb[2])

# ax_scap_YZ_S = fig.add_subplot(gs2col_scap_YZ[0], projection='3d', proj_type = 'ortho')
# ax_scap_YX_S = fig.add_subplot(gs2col_scap_YX[0], projection='3d', proj_type = 'ortho')
# ax_scap_XZ_S = fig.add_subplot(gs2col_scap_XZ[0], projection='3d', proj_type = 'ortho')

# ax_scap_YZ_D = fig.add_subplot(gs2col_scap_YZ[1], projection='3d', proj_type = 'ortho')
# ax_scap_YX_D = fig.add_subplot(gs2col_scap_YX[1], projection='3d', proj_type = 'ortho')
# ax_scap_XZ_D = fig.add_subplot(gs2col_scap_XZ[1], projection='3d', proj_type = 'ortho')

# ax_glen_YZ_S = fig.add_subplot(gs2col_glen_YZ[0], projection='3d', proj_type = 'ortho')
# ax_glen_YX_S = fig.add_subplot(gs2col_glen_YX[0], projection='3d', proj_type = 'ortho')
# ax_glen_XZ_S = fig.add_subplot(gs2col_glen_XZ[0], projection='3d', proj_type = 'ortho')

# ax_glen_YZ_D = fig.add_subplot(gs2col_glen_YZ[1], projection='3d', proj_type = 'ortho')
# ax_glen_YX_D = fig.add_subplot(gs2col_glen_YX[1], projection='3d', proj_type = 'ortho')
# ax_glen_XZ_D = fig.add_subplot(gs2col_glen_XZ[1], projection='3d', proj_type = 'ortho')

# ax_elb_YZ_S = fig.add_subplot(gs2col_elb_YZ[0], projection='3d', proj_type = 'ortho')
# ax_elb_YX_S = fig.add_subplot(gs2col_elb_YX[0], projection='3d', proj_type = 'ortho')
# ax_elb_XZ_S = fig.add_subplot(gs2col_elb_XZ[0], projection='3d', proj_type = 'ortho')

# ax_elb_YZ_D = fig.add_subplot(gs2col_elb_YZ[1], projection='3d', proj_type = 'ortho')
# ax_elb_YX_D = fig.add_subplot(gs2col_elb_YX[1], projection='3d', proj_type = 'ortho')
# ax_elb_XZ_D = fig.add_subplot(gs2col_elb_XZ[1], projection='3d', proj_type = 'ortho')


# # tax_scap_YZ_S = fig.add_subplot(gs_t[0,0], projection='3d', proj_type = 'ortho')
# # tax_scap_YX_S = fig.add_subplot(gs_t[0,2], projection='3d', proj_type = 'ortho')
# # tax_scap_XZ_S = fig.add_subplot(gs_t[0,1], projection='3d', proj_type = 'ortho')

# # tax_scap_YZ_D = fig.add_subplot(gs_t[1,0], projection='3d', proj_type = 'ortho')
# # tax_scap_YX_D = fig.add_subplot(gs_t[1,2], projection='3d', proj_type = 'ortho')
# # tax_scap_XZ_D = fig.add_subplot(gs_t[1,1], projection='3d', proj_type = 'ortho')

# # tax_glen_YZ_S = fig.add_subplot(gs_t[2,0], projection='3d', proj_type = 'ortho')
# # tax_glen_YX_S = fig.add_subplot(gs_t[2,2], projection='3d', proj_type = 'ortho')
# # tax_glen_XZ_S = fig.add_subplot(gs_t[2,1], projection='3d', proj_type = 'ortho')

# # tax_glen_YZ_D = fig.add_subplot(gs_t[3,0], projection='3d', proj_type = 'ortho')
# # tax_glen_YX_D = fig.add_subplot(gs_t[3,2], projection='3d', proj_type = 'ortho')
# # tax_glen_XZ_D = fig.add_subplot(gs_t[3,1], projection='3d', proj_type = 'ortho')

# # tax_elb_YZ_S = fig.add_subplot(gs_t[4,0], projection='3d', proj_type = 'ortho')
# # tax_elb_YX_S = fig.add_subplot(gs_t[4,2], projection='3d', proj_type = 'ortho')
# # tax_elb_XZ_S = fig.add_subplot(gs_t[4,1], projection='3d', proj_type = 'ortho')

# # tax_elb_YZ_D = fig.add_subplot(gs_t[5,0], projection='3d', proj_type = 'ortho')
# # tax_elb_YX_D = fig.add_subplot(gs_t[5,2], projection='3d', proj_type = 'ortho')
# # tax_elb_XZ_D = fig.add_subplot(gs_t[5,1], projection='3d', proj_type = 'ortho')

# # format3dPlot(tax_scap_YZ_S, '', (mintx, maxtx), (minty, maxty), (mintz, maxtz), view=viewYZ, mode='translations',joint='scap', tickInterval=0.5)


# format3dPlot(ax_scap_YZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='scap')
# format3dPlot(ax_scap_YZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='scap')
# format3dPlot(ax_glen_YZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='glenoid')
# format3dPlot(ax_glen_YZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='glenoid')
# format3dPlot(ax_elb_YZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')
# format3dPlot(ax_elb_YZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')

# format3dPlot(ax_scap_YX_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='scap')
# format3dPlot(ax_scap_YX_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='scap')
# format3dPlot(ax_glen_YX_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='glenoid')
# format3dPlot(ax_glen_YX_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='glenoid')
# format3dPlot(ax_elb_YX_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='elbow')
# format3dPlot(ax_elb_YX_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='elbow')

# format3dPlot(ax_scap_XZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='scap')
# format3dPlot(ax_scap_XZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='scap')
# format3dPlot(ax_glen_XZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='glenoid')
# format3dPlot(ax_glen_XZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='glenoid')
# format3dPlot(ax_elb_XZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')
# format3dPlot(ax_elb_XZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')


# addCosGrid(ax_scap_YZ_S, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
# addCosGrid(ax_scap_YZ_D, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
# addCosGrid(ax_glen_YZ_S, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
# addCosGrid(ax_glen_YZ_D, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
# addCosGrid(ax_elb_YZ_S, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
# addCosGrid(ax_elb_YZ_D, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)


# ax_scap_YZ_S.plot_trisurf(dfS['acromion.rx'],dfS['acromion.ry'],alphaTriScap_S,dfS['acromion.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_scap_YX_S.plot_trisurf(dfS['acromion.rx'],dfS['acromion.ry'],alphaTriScap_S,dfS['acromion.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_scap_XZ_S.plot_trisurf(dfS['acromion.rx'],dfS['acromion.ry'],alphaTriScap_S,dfS['acromion.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_scap_YZ_D.plot_trisurf(dfD['acromion.rx'],dfD['acromion.ry'],alphaTriScap_D,dfD['acromion.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_scap_YX_D.plot_trisurf(dfD['acromion.rx'],dfD['acromion.ry'],alphaTriScap_D,dfD['acromion.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_scap_XZ_D.plot_trisurf(dfD['acromion.rx'],dfD['acromion.ry'],alphaTriScap_D,dfD['acromion.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)

# ax_glen_YZ_S.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_glen_YX_S.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_glen_XZ_S.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_glen_YZ_D.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_glen_YX_D.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_glen_XZ_D.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)

# ax_elb_YZ_S.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_elb_YX_S.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_elb_XZ_S.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_elb_YZ_D.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_elb_YX_D.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_elb_XZ_D.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)

# ax_scap_YZ_S.set_title('Tegu')
# ax_glen_YZ_S.set_title('Tegu')
# ax_elb_YZ_S.set_title('Tegu')
# ax_scap_YZ_D.set_title('Opossum')
# ax_glen_YZ_D.set_title('Opossum')
# ax_elb_YZ_D.set_title('Opossum')

# ax_scap_YZ_S.scatter(dfS['acromion.rx'],dfS['acromion.ry'],dfS['acromion.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_scap_YX_S.scatter(dfS['acromion.rx'],dfS['acromion.ry'],dfS['acromion.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_scap_XZ_S.scatter(dfS['acromion.rx'],dfS['acromion.ry'],dfS['acromion.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_scap_YZ_D.scatter(dfD['acromion.rx'],dfD['acromion.ry'],dfD['acromion.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_scap_YX_D.scatter(dfD['acromion.rx'],dfD['acromion.ry'],dfD['acromion.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_scap_XZ_D.scatter(dfD['acromion.rx'],dfD['acromion.ry'],dfD['acromion.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)

# ax_glen_YZ_S.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_glen_YX_S.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_glen_XZ_S.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_glen_YZ_D.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_glen_YX_D.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_glen_XZ_D.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)

# ax_elb_YZ_S.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_elb_YX_S.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_elb_XZ_S.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_elb_YZ_D.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_elb_YX_D.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_elb_XZ_D.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)



# # connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# # def animate(i):
# #     ax0.view_init(elev=45, azim=i/10)
# #     ax1.view_init(elev=45, azim=i/10)
# #     return fig

# # both_ani = animation.FuncAnimation(fig, animate, 
# #                                frames=3600, interval=5, blit=True)

# # plt.show()

# # plt.savefig('both3Q.png', dpi=150, transparent=True)


In [26]:
dfOS.loc[dfOS['model_dac_glen_Zma']==dfOS['model_dac_glen_Zma'].max()]

,acromion.crz,acromion.rx,acromion.ry,acromion.rz,acromion.tx,acromion.ty,acromion.tz,animal,bic_Xma,bic_Yma,...,tmj_Zma,trs_Xma,trs_Yma,trs_Zma,model_lat_avg_glen_Xma,model_lat_avg_glen_Yma,model_lat_avg_glen_Zma,model_pec_avg_glen_Xma,model_pec_avg_glen_Yma,model_pec_avg_glen_Zma
6553,0.650979,23.630376,8.411742,0.658058,-0.990696,-0.384413,-0.016932,108,0.117518,-0.490276,...,NaN,0.129612,0.418011,0.256823,0.160055,0.588027,-0.568225,-0.026227,-0.106942,0.183657


### CH4 figure: invivo vs exvivo

In [471]:
smallScatter = 0.25
bigScatter = 0.75



maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'

fig = plt.figure(figsize=[15,7.5], constrained_layout=True)
ax0 = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(122, projection='3d', proj_type = 'ortho')


format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)



# plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch4plots/rotations_base.eps', dpi=300, transparent=True)


ax0.scatter(df_invivo_S_filt['glenoid.rx'],df_invivo_S_filt['glenoid.ry'],df_invivo_S_filt['glenoid.crz'], s=bigScatter, depthshade=True, color='purple', zorder=9, alpha=1)
ax1.scatter(df_invivo_D_filt['glenoid.rx'],df_invivo_D_filt['glenoid.ry'],df_invivo_D_filt['glenoid.crz'], s=bigScatter, depthshade=True, color='purple', zorder=9, alpha=1)
# plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch4plots/rotations_normal.png', dpi=300, transparent=True)
# ax0.scatter(df_invivo_S_filt_con['glenoid.rx'],df_invivo_S_filt_con['glenoid.ry'],df_invivo_S_filt_con['glenoid.crz'], s=bigScatter, depthshade=True, color='red', zorder=0, alpha=1)
# ax1.scatter(df_invivo_D_filt_con['glenoid.rx'],df_invivo_D_filt_con['glenoid.ry'],df_invivo_D_filt_con['glenoid.crz'], s=bigScatter, depthshade=True, color='red', zorder=0, alpha=1)
# plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch4plots/rotations_constrained.png', dpi=300, transparent=True)


# addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
# addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)


ax0.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax1.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)



# df_invivo_S_filt = df_invivo_S.loc[(df_invivo_S['condition']=='lo')]
# df_invivo_D_filt = df_invivo_D.loc[(df_invivo_D['condition']=='hi')]

# df_invivo_S_filt_con = df_invivo_S.loc[(df_invivo_S['condition']=='hi')]
# df_invivo_D_filt_con = df_invivo_D.loc[(df_invivo_D['condition']=='lo')]


# plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch4plots/rotations_alpha.png', dpi=300, transparent=True)


#centroids
meanSx, meanSy, meanScrz = df_invivo_S['glenoid.rx'].mean(), df_invivo_S['glenoid.ry'].mean(), df_invivo_S['glenoid.crz'].mean()
meanDx, meanDy, meanDcrz = df_invivo_D['glenoid.rx'].mean(), df_invivo_D['glenoid.ry'].mean(), df_invivo_D['glenoid.crz'].mean()
# ax0.plot(np.repeat(meanSx,abs(mincrz-meanScrz)),np.repeat(maxry,abs(mincrz-meanScrz)),np.linspace(mincrz,meanScrz,int(abs(mincrz-meanScrz))),'r--',alpha=1, linewidth=1)
# ax0.plot(np.repeat(maxrx,abs(mincrz-meanScrz)),np.repeat(meanSy,abs(mincrz-meanScrz)),np.linspace(mincrz,meanScrz,int(abs(mincrz-meanScrz))),'r--',alpha=1, linewidth=1)
# ax0.plot(np.repeat(maxrx,abs(minry-meanSy)),np.repeat(minry,abs(minry-meanSy)),np.repeat(meanScrz,abs(minry-meanSy)),'r--',alpha=1, linewidth=1)


# ax0.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
# ax1.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

# for axis in [ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7]:
#     axis.axis('off')

connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### FIGURE: ROM envelopes: glenoid big 3q
rob's data cuts off at + - 90 degrees from neutral, which is 0,0,-20 for tegu and 0, -40, -80 for opossum

In [271]:
smallScatter = 0.25
bigScatter = 0.75



maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'


fig = plt.figure(figsize=[18,10], constrained_layout=True)
gs = gridspec.GridSpec(6, 6, figure=fig)

ax0 = fig.add_subplot(gs[0:2,2], projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(gs[0:2,3], projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(gs[2:4,2], projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(gs[2:4,3], projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(gs[4:6,2], projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(gs[4:6,3], projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(gs[0:6,0:2], projection='3d', proj_type = 'ortho')

ax7 = fig.add_subplot(gs[0:6,4:6], projection='3d', proj_type = 'ortho')

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)

format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)

format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
format3dPlot(ax7, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)

plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/rotations_base.eps', dpi=300, transparent=True)




addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)



ax0.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax1.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

ax2.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax3.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)
ax4.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax5.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

ax6.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax7.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)




ax0.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax1.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax2.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax3.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax4.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax5.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

ax6.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=bigScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax7.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=bigScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

# for axis in [ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7]:
#     axis.axis('off')

connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()

# plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/glenoid_rotations.png', dpi=300, transparent=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### FIGURE: ROM envelopes:elbow big 3q

In [228]:
smallScatter = 0.25
bigScatter = 0.75

shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'

fig = plt.figure(figsize=[18,10], constrained_layout=True)
gs = gridspec.GridSpec(6, 6, figure=fig)

ax0 = fig.add_subplot(gs[0:2,2], projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(gs[0:2,3], projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(gs[2:4,2], projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(gs[2:4,3], projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(gs[4:6,2], projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(gs[4:6,3], projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(gs[0:6,0:2], projection='3d', proj_type = 'ortho')

ax7 = fig.add_subplot(gs[0:6,4:6], projection='3d', proj_type = 'ortho')

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')

format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY, joint='elbow')
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY, joint='elbow')

format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos, joint='elbow')
format3dPlot(ax7, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos, joint='elbow')

plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/rotations_base_elbow.eps', dpi=300, transparent=True)


addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)



ax0.plot_trisurf(dfS_elbow['elbow.rx'],dfS_elbow['elbow.ry'],alphaTriElbow_S,dfS_elbow['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax1.plot_trisurf(dfD_elbow['elbow.rx'],dfD_elbow['elbow.ry'],alphaTriElbow_D,dfD_elbow['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

ax2.plot_trisurf(dfS_elbow['elbow.rx'],dfS_elbow['elbow.ry'],alphaTriElbow_S,dfS_elbow['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax3.plot_trisurf(dfD_elbow['elbow.rx'],dfD_elbow['elbow.ry'],alphaTriElbow_D,dfD_elbow['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)
ax4.plot_trisurf(dfS_elbow['elbow.rx'],dfS_elbow['elbow.ry'],alphaTriElbow_S,dfS_elbow['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax5.plot_trisurf(dfD_elbow['elbow.rx'],dfD_elbow['elbow.ry'],alphaTriElbow_D,dfD_elbow['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

ax6.plot_trisurf(dfS_elbow['elbow.rx'],dfS_elbow['elbow.ry'],alphaTriElbow_S,dfS_elbow['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax7.plot_trisurf(dfD_elbow['elbow.rx'],dfD_elbow['elbow.ry'],alphaTriElbow_D,dfD_elbow['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)




ax0.scatter(dfS_elbow['elbow.rx'],dfS_elbow['elbow.ry'],dfS_elbow['elbow.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax1.scatter(dfD_elbow['elbow.rx'],dfD_elbow['elbow.ry'],dfD_elbow['elbow.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax2.scatter(dfS_elbow['elbow.rx'],dfS_elbow['elbow.ry'],dfS_elbow['elbow.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax3.scatter(dfD_elbow['elbow.rx'],dfD_elbow['elbow.ry'],dfD_elbow['elbow.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax4.scatter(dfS_elbow['elbow.rx'],dfS_elbow['elbow.ry'],dfS_elbow['elbow.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax5.scatter(dfD_elbow['elbow.rx'],dfD_elbow['elbow.ry'],dfD_elbow['elbow.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

ax6.scatter(dfS_elbow['elbow.rx'],dfS_elbow['elbow.ry'],dfS_elbow['elbow.crz'], s=bigScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax7.scatter(dfD_elbow['elbow.rx'],dfD_elbow['elbow.ry'],dfD_elbow['elbow.crz'], s=bigScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

# for axis in [ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7]:
#     axis.axis('off')

connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()

plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/elbow_rotations.png', dpi=300, transparent=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### FIGURE: translation plots

In [12]:
# def Rgb6DOF(df, species, joint, xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True):
#     df = df.copy()
#     dfspecies = df.filter(items=['species'])
#     if not normalizeAcrossSpecies: 
#         df = df[df['species']==species]
#     df = df.replace(np.NaN, 0)
#     df = df.filter(like=joint+'.t')

# #     print(df.max())
#     tX = df.filter(like=joint+'.tx')
#     tY = df.filter(like=joint+'.ty')
#     tZ = df.filter(like=joint+'.tz')
#     pos_tX = np.where(tX > 0, tX, 0)
#     pos_tY = np.where(tY > 0, tY, 0)
#     pos_tZ = np.where(tZ > 0, tZ, 0)
#     neg_tX = np.where(tX < 0, abs(tX), 0)
#     neg_tY = np.where(tY < 0, abs(tY), 0)
#     neg_tZ = np.where(tZ < 0, abs(tZ), 0)

#     df['tX'] = pos_tX if xPos else neg_tX
#     df['tY'] = pos_tY if yPos else neg_tY
#     df['tZ'] = pos_tZ if zPos else neg_tZ
#     tMax = max(df.max().max(),abs(df.min().min()))

#     df/=tMax

#     #     df=df.drop([tX.columns[0],tY.columns[0],tZ.columns[0]], axis=1)

#     totalXYZ = df[['tX','tY','tZ']].sum(axis=1)
#     df['scale'] = preprocessing.MinMaxScaler().fit_transform(totalXYZ.values.reshape(-1, 1))
#     if normalizeAcrossSpecies:
#         df = dfspecies.join(df)
#         df = df[df['species']==species]

#     return(df[['tX','tY','tZ','scale']])

# dfS_6dofPos = Rgb6DOF(df, 'Tegu', 'glenoid', xPos=True, yPos=True, zPos=True)
# dfD_6dofPos = Rgb6DOF(df, 'Opossum', 'glenoid', xPos=True, yPos=True, zPos=True)
# dfS_6dofNeg = Rgb6DOF(df, 'Tegu', 'glenoid', xPos=False, yPos=False, zPos=False)
# dfD_6dofNeg = Rgb6DOF(df, 'Opossum', 'glenoid', xPos=False, yPos=False, zPos=False)


dfS_transDiverge_scap = divergingTranslation(df, 'Tegu', 'glenoid', normalizeAcrossSpecies=True)
dfD_transDiverge_scap = divergingTranslation(df, 'Opossum', 'glenoid', normalizeAcrossSpecies=True)

smallScatter = 0.25
bigScatter = 0.75
scale=10


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'


fig = plt.figure(figsize=[12,12], constrained_layout=True)
ax0 = fig.add_subplot(321, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(322, projection='3d', proj_type = 'ortho')
ax2 = fig.add_subplot(323, projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(324, projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(325, projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(326, projection='3d', proj_type = 'ortho')

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)

addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax2, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax3, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax4, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax5, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=abs(dfS_transDiverge_scap.xma)*scale, c=dfS_transDiverge_scap.xma, cmap='PuOr', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
ax1.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(dfD_transDiverge_scap.xma)*scale, c=dfD_transDiverge_scap.xma, cmap='PuOr', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
ax2.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=abs(dfS_transDiverge_scap.yma)*scale, c=dfS_transDiverge_scap.yma, cmap='PuOr', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
ax3.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(dfD_transDiverge_scap.yma)*scale, c=dfD_transDiverge_scap.yma, cmap='PuOr', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
ax4.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=abs(dfS_transDiverge_scap.zma)*scale, c=dfS_transDiverge_scap.zma, cmap='PuOr', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
ax5.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(dfD_transDiverge_scap.zma)*scale, c=dfD_transDiverge_scap.zma, cmap='PuOr', vmin=-1, vmax=1, depthshade=False, edgecolors='None')

ax0.annotate('X', (1.275, 0.55), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold')
ax0.annotate('+ Compression  ', (1.275, 0.475), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='#6f559c')
ax0.annotate('  Distraction -', (1.275, 0.475), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='#de8226')
ax2.annotate('Y', (1.275, 0.55), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold')
ax2.annotate('+ Posterior  ', (1.275, 0.475), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='#6f559c')
ax2.annotate('  Anterior -', (1.275, 0.475), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='#de8226')
ax4.annotate('Z', (1.275, 0.55), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold')
ax4.annotate('+ Dorsal tegu/Medial opossum  ', (1.275, 0.475), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='#6f559c')
ax4.annotate('  Ventral tegu/Lateral opossum -', (1.275, 0.475), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='#de8226')

for axis in [ax0, ax1, ax2, ax3, ax4, ax5]:
    axis.dist=13


# plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/rotations_base.eps', dpi=300, transparent=True)


# ax0.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=dfS_6dofPos['scale']*25, c=dfS_6dofPos[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
# ax1.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=dfD_6dofPos['scale']*25, c=dfD_6dofPos[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
# ax2.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=dfS_6dofNeg['scale']*25, c=dfS_6dofNeg[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
# ax3.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=dfD_6dofNeg['scale']*25, c=dfD_6dofNeg[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')

# for axis in [ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7]:
#     axis.axis('off')

# connection4 = fig.canvas.mpl_connect('motion_notify_event', on_move4)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)


plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/6dof.png', dpi=300, transparent=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [321]:
dfD[(dfD['glenoid.crz']<-110)&(dfD['glenoid.rx']<-45)][['animal','side','frame']]

,animal,side,frame
1505,85,L,697


In [ ]:
sorted(dfD_6dofNeg[['tX']].values, reverse=True)

In [ ]:
# manually drop elbow outliers by visual inspection
i101L = df.loc[((df['animal']=='101')&(df['side']=='L'))].index
i101R = df.loc[(df['animal']=='101')&(df['side']=='R')(&(df['frame'] ==770)|&(df['frame'] ==436))].index
i88L = df.loc[((df['animal']=='88')&(df['side']=='L')&((df['frame'] ==193)|(df['frame'] ==614)))].index
i85L = df.loc[((df['animal']=='85')&(df['side']=='L')&((df['frame'] ==585)|(df['frame'] ==653)))].index
i85R = df.loc[((df['animal']=='85')&(df['side']=='R')&(df['frame'] ==38))].index

i126L = df.loc[((df['animal']=='126')&(df['side']=='L')&(df['frame'] ==125))].index
i126R = df.loc[((df['animal']=='126')&(df['side']=='R')&(df['frame'] ==342))].index

i108R = df.loc[((df['animal']=='108')&(df['side']=='R')&(df['frame'] ==506))].index
i108L = df.loc[((df['animal']=='108')&(df['side']=='L')&((df['frame'] ==249)|(df['frame'] ==345)))].index
df_elbow = df.drop([*i101L,*i88L,*i85L,*i126L,*i108R,*i108L])
dfOS_elbow = dfOS.drop([*i101L,*i88L,*i85L,*i126L,*i108R,*i108L])


### test plots these dont save

In [ ]:


startView = viewXY
shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'

dfS_robCrop = dfS.loc[(dfS['glenoid.rx'].between(-90,90))&(dfS['glenoid.ry'].between(-90,90))&(dfS['glenoid.rz'].between(-110,70))]
dfD_robCrop = dfD.loc[(dfD['glenoid.rx'].between(-90,90))&(dfD['glenoid.ry'].between(-130,50))&(dfD['glenoid.rz'].between(-170,10))]

fig = plt.figure(figsize=[18,10], constrained_layout=True)
gs = gridspec.GridSpec(6, 6, figure=fig)

ax0 = fig.add_subplot(gs[0:2,2], projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(gs[0:2,3], projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(gs[2:4,2], projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(gs[2:4,3], projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(gs[4:6,2], projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(gs[4:6,3], projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(gs[0:6,0:2], projection='3d', proj_type = 'ortho')

ax7 = fig.add_subplot(gs[0:6,4:6], projection='3d', proj_type = 'ortho')

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)

format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)

format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
format3dPlot(ax7, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)





addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)

# ax0.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTrishoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax1.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTrishoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax2.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTrishoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax3.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTrishoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)
# ax4.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTrishoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax5.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTrishoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax6.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTrishoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax7.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTrishoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)




ax0.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax1.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax2.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax3.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax4.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax5.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

ax6.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax7.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)



connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()

# plt.savefig('both3Q.png', dpi=150, transparent=True)


In [ ]:
dfD.loc[dfD['glenoid.rx']<-50].filter(like='glenoid.cr')

In [ ]:


startView = viewXY
shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'

dfS_robCrop = dfS.loc[(dfS['elbow.rx'].between(-90,90))&(dfS['elbow.ry'].between(-90,90))&(dfS['elbow.rz'].between(-110,70))]
dfD_robCrop = dfD.loc[(dfD['elbow.rx'].between(-90,90))&(dfD['elbow.ry'].between(-130,50))&(dfD['elbow.rz'].between(-170,10))]

fig = plt.figure(figsize=[18,10], constrained_layout=True)
gs = gridspec.GridSpec(6, 6, figure=fig)

ax0 = fig.add_subplot(gs[0:2,2], projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(gs[0:2,3], projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(gs[2:4,2], projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(gs[2:4,3], projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(gs[4:6,2], projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(gs[4:6,3], projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(gs[0:6,0:2], projection='3d', proj_type = 'ortho')

ax7 = fig.add_subplot(gs[0:6,4:6], projection='3d', proj_type = 'ortho')

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)

format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)

format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
format3dPlot(ax7, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)





addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)

# ax0.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax1.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax2.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax3.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)
# ax4.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax5.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax6.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax7.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)




ax0.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax1.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax2.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax3.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax4.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax5.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

ax6.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax7.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)



connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()

# plt.savefig('both3Q.png', dpi=150, transparent=True)


In [ ]:


startView = viewXY
shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'

dfS_robCrop = dfS.loc[(dfS['elbow.rx'].between(-90,90))&(dfS['elbow.ry'].between(-90,90))&(dfS['elbow.rz'].between(-110,70))]
dfD_robCrop = dfD.loc[(dfD['elbow.rx'].between(-90,90))&(dfD['elbow.ry'].between(-130,50))&(dfD['elbow.rz'].between(-170,10))]

fig = plt.figure(figsize=[18,10], constrained_layout=True)
gs = gridspec.GridSpec(6, 6, figure=fig)

ax0 = fig.add_subplot(gs[0:2,2], projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(gs[0:2,3], projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(gs[2:4,2], projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(gs[2:4,3], projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(gs[4:6,2], projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(gs[4:6,3], projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(gs[0:6,0:2], projection='3d', proj_type = 'ortho')

ax7 = fig.add_subplot(gs[0:6,4:6], projection='3d', proj_type = 'ortho')

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)

format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)

format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
format3dPlot(ax7, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)





addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)

# ax0.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax1.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax2.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax3.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)
# ax4.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax5.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax6.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax7.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)


dfDno101L = dfD.loc[~(dfD['animal']=='101')|~(dfD['side']=='L')]

ax0.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax1.scatter(dfDno101L['elbow.rx'],dfDno101L['elbow.ry'],dfDno101L['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax2.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax3.scatter(dfDno101L['elbow.rx'],dfDno101L['elbow.ry'],dfDno101L['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax4.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax5.scatter(dfDno101L['elbow.rx'],dfDno101L['elbow.ry'],dfDno101L['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

ax6.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax7.scatter(dfDno101L['elbow.rx'],dfDno101L['elbow.ry'],dfDno101L['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)



connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()

# plt.savefig('both3Q.png', dpi=150, transparent=True)


### R shit here

In [2372]:
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()
from rpy2.robjects.conversion import localconverter
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.robjects.lib.dplyr import DataFrame
from rpy2.robjects import rl

alphashape3d = importr('alphashape3d')
tidyverse = importr('tidyverse')
ro.r('''
        getCriticalAlpha <- function(components, alphas){
          criticalIndex <- 1
          for (i in 1:length(components)){
            comp = as.numeric(components[[i]])
            if (any(comp != 1)){
              criticalIndex <- i
            }
          }
          criticalIndex <- criticalIndex + 1
          print(alphas[[criticalIndex]])
          return(list(criticalIndex, alphas[[criticalIndex]]))
        }
        ''')
getCriticalAlpha = ro.r['getCriticalAlpha']
with localconverter(ro.default_converter + pandas2ri.converter):
    rdf_S_glenoid = ro.r['as.matrix'](ro.conversion.py2rpy(dfS[['glenoid.rx','glenoid.ry','glenoid.crz']].copy()))
    rdf_D_glenoid = ro.r['as.matrix'](ro.conversion.py2rpy(dfD[['glenoid.rx','glenoid.ry','glenoid.crz']].copy()))
    rdf_S_elbow = ro.r['as.matrix'](ro.conversion.py2rpy(dfS_elbow[['elbow.rx','elbow.ry','elbow.crz']].copy()))
    rdf_D_elbow = ro.r['as.matrix'](ro.conversion.py2rpy(dfD_elbow[['elbow.rx','elbow.ry','elbow.crz']].copy()))
#     rdf_S_model = ro.r['as.matrix'](ro.conversion.py2rpy(df_M_S[['Rx','Ry','cRz']].copy()))
#     rdf_D_model = ro.r['as.matrix'](ro.conversion.py2rpy(df_M_D[['Rx','Ry','cRz']].copy()))
    
subTenAlphas = np.linspace(0.1,9.9, 99)
supTenAlphas = np.linspace(10, 100, 91)
alphas = np.concatenate((subTenAlphas,supTenAlphas))
alphaCols = [str(round(alpha,3)).replace('.','_') for alpha in alphas]


alphas_sens = np.arange(1,101)
alphaCols_sens = [str(round(alpha,3)).replace('.','_') for alpha in alphas_sens]


def getAlphaObjects(rdf, alphas):
    try:
        alphaCols = [str(round(alpha,3)).replace('.','_') for alpha in alphas]
        print("...calculating alphas...")
        alphaShapes = alphashape3d.ashape3d(rdf, alphas, pert=True)
        print("...calculating components...")
        components = alphashape3d.components_ashape3d(alphaShapes, indexAlpha="all")
        print("...calculating volumes...")
        volumes = alphashape3d.volume_ashape3d(alphaShapes, indexAlpha="all")
    #     print("...calculating normals...")
    #     normals = alphashape3d.surfaceNormals(alphaShapes, indexAlpha="all")
        print("...calculating critical alpha...")
        [crit_index, crit_val] = getCriticalAlpha(components, alphas)
        shapes = np.array(alphaShapes, dtype=object)
        tetras_df = pd.DataFrame(alphaShapes[0]).rename(columns={**{0:'index1',1:'index2',2:'index3',3:'index4',4:'intervals'},**dict(zip(range(5,len(alphas)+5),alphaCols))})
        triangles_df = pd.DataFrame(alphaShapes[1]).rename(columns={**{0:'index1',1:'index2',2:'index3',3:'on_convex_hull',4:'attached',5:'intervals1',6:'intervals2',7:'intervals3'},**dict(zip(range(8,len(alphas)+8),alphaCols))})
        edges_df = pd.DataFrame(alphaShapes[2]).rename(columns={**{0:'index1',1:'index2',2:'on_convex_hull',3:'attached',4:'intervals1',5:'intervals2',6:'intervals3'},**dict(zip(range(7,len(alphas)+7),alphaCols))})
        vertices_df = pd.DataFrame(alphaShapes[3]).rename(columns={**{0:'index',1:'on_convex_hull',2:'intervals1',3:'intervals2'},**dict(zip(range(4,len(alphas)+4),alphaCols))})
        vertices_df['x'],vertices_df['y'],vertices_df['z'] = alphaShapes[4].T[0],alphaShapes[4].T[1],alphaShapes[4].T[2]
        components_df = pd.DataFrame(np.array(components, dtype=object).T).rename(columns=dict(zip(range(0,len(alphas)+1),alphaCols)))
        volumes_df = pd.DataFrame(np.array(volumes, dtype=object)).transpose().rename(columns=dict(zip(range(0,len(alphas)+1),alphaCols)))
    #     normals_df = pd.DataFrame(np.array(normals, dtype=object), columns=['normal','center'], index=alphaCols)
        return {'tetrahedrons':tetras_df, 'triangles': triangles_df, 'edges': edges_df, 'vertices': vertices_df,
                'components': components_df, 'volumes': volumes_df, 'crit_val':crit_val, 'crit_index':crit_index }
    except rpy2.rinterface_lib.embedded.RRuntimeError:
        return {}

alpha_s_glenoid = getAlphaObjects(rdf_S_glenoid, alphas_sens)
alpha_d_glenoid = getAlphaObjects(rdf_D_glenoid, alphas_sens)
alpha_s_elbow = getAlphaObjects(rdf_S_elbow, alphas_sens)
alpha_d_elbow = getAlphaObjects(rdf_D_elbow, alphas_sens)

# %pip install kneed
from kneed import DataGenerator, KneeLocator

def autoAlphaChooser(dfs_with_volumes, names, starting_alpha, sParam=10.0, curve='concave', direction='increasing'):
    volumes_df = pd.concat(dfs_with_volumes, axis=1).astype('float')
    volumes_df.columns=names
    alphas = [float(alphaCol.replace('_','.')) for alphaCol in volumes_df.index]
    maxVol = volumes_df.iloc[-1].max()
    minVol = volumes_df.iloc[-1].min()
    volumes_df.set_index(pd.Index(alphas), inplace=True)
    volumes_df.rename(columns={"index":"alpha"}, inplace=True)
    maxCol = volumes_df.columns[list(volumes_df.iloc[-1].values).index(maxVol)]
    minCol = volumes_df.columns[list(volumes_df.iloc[-1].values).index(minVol)]
    ratio = maxCol+' : '+minCol
    volumes_df[ratio] = np.divide(volumes_df.loc[:,maxCol],volumes_df.loc[:,minCol])
    volumes_df[ratio].fillna(0, inplace=True)
    ratio_series = volumes_df.loc[starting_alpha:,ratio].copy() if starting_alpha else volumes_df.loc[:,ratio].copy()
    kneedle = KneeLocator(list(ratio_series.index), ratio_series, S=sParam, curve=curve, direction=direction)
    return([volumes_df, kneedle])
    
# plt.close("all")
# volumes_df, auto_alpha = autoAlphaChooser([alpha_s_glenoid['volumes'].T,alpha_d_glenoid['volumes'].T,alpha_s_elbow['volumes'].T,alpha_d_elbow['volumes'].T],['Tegu glenoid','Opossum glenoid','Tegu elbow','Opossum elbow'],15, 10.0, 'convex', 'decreasing')
# fig, ax = plt.subplots()
# volumes_df.loc[:,['Tegu glenoid','Opossum glenoid','Tegu elbow','Opossum elbow']].plot(ax=ax, color=['C0','C1','#A0CBE8','#F1CE63'])
# volumes_df.loc[:,[volumes_df.columns[-1]]].plot(secondary_y=True, style='g',ax=ax)
# # ax.vlines(auto_alpha.knee, volumes_df.min().min(), volumes_df.max().max(), color='r', label='knee')
# # auto_alpha.knee

# ax.vlines(20, 0, maxMax, color='red')
# ax.annotate(text='20', xy =(((20-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='center',size=8)

...calculating alphas...
...calculating components...
...calculating volumes...
...calculating critical alpha...
[1] 8
...calculating alphas...
...calculating components...
...calculating volumes...
...calculating critical alpha...
[1] 9
...calculating alphas...
...calculating components...
...calculating volumes...
...calculating critical alpha...
[1] 9
...calculating alphas...
...calculating components...
...calculating volumes...
...calculating critical alpha...
[1] 6


#### alpha volumes at alpha 20

In [2454]:
print("Tegu glenoid",alpha_s_glenoid['volumes']['20'].values)
print("Opossum glenoid",alpha_d_glenoid['volumes']['20'].values)
print("Tegu elbow",alpha_s_elbow['volumes']['20'].values)
print("Opossum elbow",alpha_d_elbow['volumes']['20'].values)

Tegu glenoid [811968.4796175994]
Opossum glenoid [442580.2398631696]
Tegu elbow [569328.7952580728]
Opossum elbow [233326.94973002883]


### Downsampling

In [2374]:
def downsample(df, species, factor=1, random=False):
    full = df.loc[df.species == species].copy()
    full.loc[:,'uid'] = full.index
    full.loc[:,'animalside'] = full.animal+full.side
    if random:
        frames_to_pick = np.sort(np.random.choice(list(full.index), len(full)//factor, replace=False))
        return(full.loc[frames_to_pick])
    for animalside in full.animalside.unique():
        as_df = full[full.animalside == animalside]
        min_frame, max_frame = as_df.frame.min(), as_df.frame.max()
        candidate_range = np.arange(min_frame, max_frame+1)
        frames_to_pick = list(candidate_range[::int(factor)])
        full.drop(full[(full['animalside']==animalside)&(~full['frame'].isin(frames_to_pick))].index, inplace=True)
    return(full)

factors_to_test = np.arange(1,11)
df_S_1to10 = [downsample(df, 'Tegu', factor) for factor in factors_to_test]
df_D_1to10 = [downsample(df, 'Opossum', factor) for factor in factors_to_test]

df_S_1to10_rand = [downsample(df, 'Tegu', factor, random=True) for factor in factors_to_test]
df_D_1to10_rand = [downsample(df, 'Opossum', factor, random=True) for factor in factors_to_test]

In [ ]:
with localconverter(ro.default_converter + pandas2ri.converter):
    rdf_S_glenoid_1to10 = [ro.r['as.matrix'](ro.conversion.py2rpy(dfS[['glenoid.rx','glenoid.ry','glenoid.crz']].copy())) for dfS in df_S_1to10]
    rdf_D_glenoid_1to10 = [ro.r['as.matrix'](ro.conversion.py2rpy(dfD[['glenoid.rx','glenoid.ry','glenoid.crz']].copy())) for dfD in df_D_1to10]
    rdf_S_glenoid_1to10_rand = [ro.r['as.matrix'](ro.conversion.py2rpy(dfS[['glenoid.rx','glenoid.ry','glenoid.crz']].copy())) for dfS in df_S_1to10_rand]
    rdf_D_glenoid_1to10_rand = [ro.r['as.matrix'](ro.conversion.py2rpy(dfD[['glenoid.rx','glenoid.ry','glenoid.crz']].copy())) for dfD in df_D_1to10_rand]

alphas_S_1to10 = [getAlphaObjects(rdf_S_glenoid, alphas_sens) for rdf_S_glenoid in rdf_S_glenoid_1to10]
alphas_D_1to10 = [getAlphaObjects(rdf_D_glenoid, alphas_sens) for rdf_D_glenoid in rdf_D_glenoid_1to10]
alphas_S_1to10_rand = [getAlphaObjects(rdf_S_glenoid_rand, alphas_sens) for rdf_S_glenoid_rand in rdf_S_glenoid_1to10_rand]
alphas_D_1to10_rand = [getAlphaObjects(rdf_D_glenoid_rand, alphas_sens) for rdf_D_glenoid_rand in rdf_D_glenoid_1to10_rand]



import csv
for i, alphaobj in enumerate(alphas_S_1to10):
    name = 'alpha_S_'+str(i)
    with open('./alphas/'+name+'.csv', 'w') as f:
        for key in alphaobj.keys():
            f.write("%s,%s\n"%(key,alphaobj[key]))
for i, alphaobj in enumerate(alphas_D_1to10):
    name = 'alpha_D_'+str(i)
    with open('./alphas/'+name+'.csv', 'w') as f:
        for key in alphaobj.keys():
            f.write("%s,%s\n"%(key,alphaobj[key]))
for i, alphaobj in enumerate(alphas_S_1to10_rand):
    name = 'alpha_S_rand_'+str(i)
    with open('./alphas/'+name+'.csv', 'w') as f:
        for key in alphaobj.keys():
            f.write("%s,%s\n"%(key,alphaobj[key]))
for i, alphaobj in enumerate(alphas_D_1to10_rand):
    name = 'alpha_D_rand_'+str(i)
    with open('./alphas/'+name+'.csv', 'w') as f:
        for key in alphaobj.keys():
            f.write("%s,%s\n"%(key,alphaobj[key]))

#### results from alpha and downsampling sensititivty analaysis

In [2455]:
criticals_S_uniform = [8,9,10,10,13,12,np.nan,np.nan,np.nan,np.nan]
criticals_D_uniform = [9,9,12,11,11,14,11,14,10,13]
criticals_S_random = [8,10,10,14,11,20,14,16,13,np.nan]
criticals_D_random = [9,9,10,8,11,8,11,12,10,12]
pd.set_option('display.max_rows', 20)


In [2453]:
pd.set_option('display.max_rows', 20)

volumes_df, auto_alpha = autoAlphaChooser([alphas_S_1to10[0]['volumes'].T,alphas_D_1to10[0]['volumes'].T],['Tegu uniform 1/8','Opossum uniform 1/8'],18, 10.0, 'convex', 'decreasing')
volumes_df

,Tegu uniform 1/8,Opossum uniform 1/8,Tegu uniform 1/8 : Opossum uniform 1/8
1.0,0.000000e+00,36.618028,0.000000
2.0,2.860997e+01,4344.098849,0.006586
3.0,1.012789e+03,37720.885072,0.026850
4.0,8.981658e+03,101462.505204,0.088522
5.0,3.761149e+04,160973.638924,0.233650
6.0,1.033477e+05,208300.146173,0.496148
7.0,1.965819e+05,245971.750362,0.799205
8.0,2.978965e+05,275642.962236,1.080733
9.0,4.010782e+05,305170.802033,1.314275
10.0,5.028714e+05,327503.676969,1.535468


In [2411]:
plt.close("all")
fig, ax = plt.subplots(3,2, sharey='all', figsize=[15,12])

volumes_df, auto_alpha = autoAlphaChooser([alphas_S_1to10[0]['volumes'].T,alphas_D_1to10[0]['volumes'].T],['Tegu uniform 1/8','Opossum uniform 1/8'],18, 10.0, 'convex', 'decreasing')
volumes_df.loc[:,['Tegu uniform 1/8','Opossum uniform 1/8']].plot(ax=ax[0,0])
volumes_df.loc[:,volumes_df.columns[-1]].plot(secondary_y=True, style='g',ax=ax[0,0], label='ratio')
ax[0,0].vlines(20, volumes_df.min().min(), volumes_df.max().max(), color='r')
ax[0,0].title.set_text('1 in 8 (no subsampling)')
x_bounds = ax[0,0].get_xlim()
ax[0,0].legend(fontsize=10, loc=4)
# ax[0,0].annotate(text=int(auto_alpha.knee), xy =(((auto_alpha.knee-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='center',size=8)
ax[0,0].annotate(text='U,R 20', xy =(((20-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='center',size=8)


yMax = [volumes_df.max().max()]

for v_i, a_i, denom in zip(np.arange(1,6),[(0,1),(1,0),(1,1),(2,0),(2,1)],((np.arange(5)+2)*8).astype('str')):
    volumes_df, auto_alpha = autoAlphaChooser([alphas_S_1to10[v_i]['volumes'].T,alphas_D_1to10[v_i]['volumes'].T],['Tegu uniform 1/'+denom,'Opossum uniform 1/'+denom],18, 10.0, 'convex', 'decreasing')
    volumes_df_rand, auto_alpha_rand = autoAlphaChooser([alphas_S_1to10_rand[v_i]['volumes'].T,alphas_D_1to10_rand[v_i]['volumes'].T],['Tegu random 1/'+denom,'Opossum random 1/'+denom],18, 10.0, 'convex', 'decreasing')
    volumes_df.loc[:,['Tegu uniform 1/'+denom,'Opossum uniform 1/'+denom]].plot(ax=ax[a_i], color=['C0','C1'])
    volumes_df_rand.loc[:,['Tegu random 1/'+denom,'Opossum random 1/'+denom]].plot(ax=ax[a_i], color=['C0','C1'], linestyle='dotted')
    volumes_df.loc[:,[volumes_df.columns[-1]]].plot(secondary_y=True, style='g',ax=ax[a_i])
    volumes_df_rand.loc[:,[volumes_df_rand.columns[-1]]].plot(secondary_y=True, style='g',ax=ax[a_i],linestyle='dotted')
    x_bounds = ax[a_i].get_xlim()
    ax[a_i].legend(fontsize=10, loc=4)
    ax[a_i].title.set_text('1 in '+denom)
    yMax.append(volumes_df.max().max())

maxMax = max(yMax)

ax[0,0].set_ylabel('Mobility volume (°^3)')
ax[1,0].set_ylabel('Mobility volume (°^3)')
ax[2,0].set_ylabel('Mobility volume (°^3)')
ax[2,0].set_xlabel('Alpha')
ax[2,1].set_xlabel('Alpha')
ax[0,1].right_ax.set_ylabel('Tegu-opossum ratio')
ax[1,1].right_ax.set_ylabel('Tegu-opossum ratio')
ax[2,1].right_ax.set_ylabel('Tegu-opossum ratio')


ax[0,1].vlines(22, 0, maxMax, color='r')
ax[0,1].vlines(24, 0, maxMax, color='r', linestyle='dotted')
ax[0,1].annotate(text='U 22', xy =(((22-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[0,1].annotate(text='R 24', xy =(((24-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)


ax[1,0].vlines(24, 0, maxMax, color='r')
ax[1,0].vlines(26, 0, maxMax, color='r', linestyle='dotted')
ax[1,0].annotate(text='U 24', xy =(((24-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[1,0].annotate(text='R 26', xy =(((26-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[1,1].vlines(26, 0, maxMax, color='r')
ax[1,1].vlines(28, 0, maxMax, color='r', linestyle='dotted')
ax[1,1].annotate(text='U 26', xy =(((26-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[1,1].annotate(text='R 28', xy =(((28-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[2,0].vlines(27, 0, maxMax, color='r')
ax[2,0].vlines(30, 0, maxMax, color='r', linestyle='dotted')
ax[2,0].annotate(text='U 27', xy =(((27-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[2,0].annotate(text='R 30', xy =(((30-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[2,1].vlines(30, 0, maxMax, color='r')
ax[2,1].annotate(text='U, R 30', xy =(((30-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='center',size=8)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/usr/local/lib/python3.9/site-packages/pandas/plotting/_matplotlib/tools.py:400: MatplotlibDeprecationWarning: 
The is_first_col function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_first_col() instead.
  if ax.is_first_col():
/usr/local/lib/python3.9/site-packages/pandas/plotting/_matplotlib/tools.py:400: MatplotlibDeprecationWarning: 
The is_first_col function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_first_col() instead.
  if ax.is_first_col():
/usr/local/lib/python3.9/site-packages/pandas/plotting/_matplotlib/tools.py:400: MatplotlibDeprecationWarning: 
The is_first_col function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_first_col() instead.
  if ax.is_first_col():
/usr/local/lib/python3.9/site-packages/pandas/plotting/_matplotlib/tools.py:400: MatplotlibDeprecationWarning: 
The is_first_col f

Text(0.3117539026629936, 1.01, 'U, R 30')

In [ ]:
plt.close("all")
fig, ax = plt.subplots(3,2, sharey='all', figsize=[15,12])

volumes_df, auto_alpha = autoAlphaChooser([alphas_S_1to10[0]['volumes'].T,alphas_D_1to10[0]['volumes'].T],['Tegu uniform 1/8','Opossum uniform 1/8'],18, 10.0, 'convex', 'decreasing')
volumes_df.loc[:,['Tegu uniform 1/8','Opossum uniform 1/8']].plot(ax=ax[0,0])
volumes_df.loc[:,[volumes_df.columns[-1]]].plot(secondary_y=True, style='g',ax=ax[0,0])
ax[0,0].vlines(auto_alpha.knee, volumes_df.min().min(), volumes_df.max().max(), color='r')
ax[0,0].title.set_text('1 in 8 (no further downsampling)')
x_bounds = ax[0,0].get_xlim()
ax[0,0].legend(fontsize=10)
ax[0,0].annotate(text=int(auto_alpha.knee), xy =(((auto_alpha.knee-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='center',size=8)

yMax = [volumes_df.max().max()]

for v_i, a_i, denom in zip(np.arange(1,6),[(0,1),(1,0),(1,1),(2,0),(2,1)],((np.arange(5)+2)*8).astype('str')):
    volumes_df, auto_alpha = autoAlphaChooser([alphas_S_1to10[v_i]['volumes'].T,alphas_D_1to10[v_i]['volumes'].T],['Tegu uniform 1/'+denom,'Opossum uniform 1/'+denom],18, 10.0, 'convex', 'decreasing')
    volumes_df_rand, auto_alpha_rand = autoAlphaChooser([alphas_S_1to10_rand[v_i]['volumes'].T,alphas_D_1to10_rand[v_i]['volumes'].T],['Tegu random 1/'+denom,'Opossum random 1/'+denom],18, 10.0, 'convex', 'decreasing')
    volumes_df.loc[:,['Tegu uniform 1/'+denom,'Opossum uniform 1/'+denom]].plot(ax=ax[a_i], color=['C0','C1'])
    volumes_df_rand.loc[:,['Tegu random 1/'+denom,'Opossum random 1/'+denom]].plot(ax=ax[a_i], color=['C0','C1'], linestyle='dotted')
    volumes_df.loc[:,[volumes_df.columns[-1]]].plot(secondary_y=True, style='g',ax=ax[a_i])
    volumes_df_rand.loc[:,[volumes_df_rand.columns[-1]]].plot(secondary_y=True, style='#0D2913',ax=ax[a_i])
    x_bounds = ax[a_i].get_xlim()
    ax[a_i].legend(fontsize=10)
    ax[a_i].title.set_text('1 in '+denom)
    yMax.append(volumes_df.max().max())

maxMax = max(yMax)

ax[0,0].set_ylabel('Mobility volume (°^3)')
ax[1,0].set_ylabel('Mobility volume (°^3)')
ax[2,0].set_ylabel('Mobility volume (°^3)')
ax[2,0].set_xlabel('Alpha')
ax[2,1].set_xlabel('Alpha')
ax[0,1].right_ax.set_ylabel('Tegu-opossum ratio')
ax[1,1].right_ax.set_ylabel('Tegu-opossum ratio')
ax[2,1].right_ax.set_ylabel('Tegu-opossum ratio')


ax[0,1].vlines(22, 0, maxMax, color='r')
ax[0,1].annotate(text='U,R 22', xy =(((22-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='center',size=8)

ax[1,0].vlines(24, 0, maxMax, color='r')
ax[1,0].vlines(26, 0, maxMax, color='r', linestyle='dotted')
ax[1,0].annotate(text='U 24', xy =(((24-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[1,0].annotate(text='R 26', xy =(((26-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[1,1].vlines(26, 0, maxMax, color='r')
ax[1,1].vlines(28, 0, maxMax, color='r', linestyle='dotted')
ax[1,1].annotate(text='U 26', xy =(((26-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[1,1].annotate(text='R 28', xy =(((28-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[2,0].vlines(27, 0, maxMax, color='r')
ax[2,0].vlines(30, 0, maxMax, color='r', linestyle='dotted')
ax[2,0].annotate(text='U 27', xy =(((27-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[2,0].annotate(text='R 30', xy =(((30-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[2,1].vlines(30, 0, maxMax, color='r')
ax[2,1].vlines(37, 0, maxMax, color='r', linestyle='dotted')
ax[2,1].annotate(text='U 30', xy =(((30-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[2,1].annotate(text='R 37', xy =(((37-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)



## automate choosing alpha

In [ ]:
volumes_df = alpha_d['volumes'].T.join(alpha_s['volumes'].T, lsuffix='opossum', rsuffix='tegu').replace(0, np.nan)
volumes_df.index = pd.to_numeric([index.replace('_','.') for index in volumes_df.index])
volumes_df.columns=['opossum','tegu']
volumes_df['t-o ratio'] = volumes_df['tegu']/volumes_df['opossum']
volumes_df['ratio d1'] = np.gradient(volumes_df['t-o ratio'])
zero_crossings = [volumes_df.index[index] for index in np.where(np.diff(np.sign(volumes_df['ratio d1'])))[0]]
zero_crossing_rows = volumes_df.loc[zero_crossings].loc[(volumes_df['tegu'] >= volumes_df['tegu'].max()/2)&(volumes_df['opossum'] >= volumes_df['opossum'].max()/2)]
first_zero_crossing_row = zero_crossing_rows.iloc[0]
second_zero_crossing_row = zero_crossing_rows.iloc[1]



fig, ax = plt.subplots()
volumes_df.loc[:,['opossum','tegu']].plot(ax=ax, style=['blue','orange'])
volumes_df.loc[:,['t-o ratio']].plot(secondary_y=True, style='g',ax=ax, label='tegu-opossum ratio')
# volumes_df.loc[:,['ratio d1']].plot(secondary_y=True, style='g',ax=ax)
ax.vlines(second_zero_crossing_row.name, volumes_df.min().min(), volumes_df.max().max(), color='r', label='second 0 crossing of ratio d1')
ax.vlines(allEnclosed_d, volumes_df.min().min(), volumes_df.max().max(), color='blue', linestyle=':', label='opossum critical')
ax.vlines(allEnclosed_s, volumes_df.min().min(), volumes_df.max().max(), color='orange', linestyle=':', label='tegu critical')
ax.vlines(waterTight_d, volumes_df.min().min(), volumes_df.max().max(), color='blue', linestyle='dashed', label='opossum watertight')
ax.vlines(waterTight_s, volumes_df.min().min(), volumes_df.max().max(), color='orange', linestyle='dashed', label='tegu watertight')
ax.legend()

### color by moment arm

In [904]:
#color by momentarm

shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }
maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()
def RgbMa(df, species, muscleName, xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True):
    df = df.copy()
    dfspecies = df.filter(items=['species'])
    if not normalizeAcrossSpecies: 
        df = df[df['species']==species[0]]
    df = df.loc[:,[muscleName+'_Xma',muscleName+'_Yma',muscleName+'_Zma']]
    df = df.replace(np.NaN, 0)
    Xma = df.filter(like='Xma')
    Yma = df.filter(like='Yma')
    Zma = df.filter(like='Zma')
    df['posXma'] = np.where(Xma > 0, Xma, 0)
    df['posYma'] = np.where(Yma > 0, Yma, 0)
    df['posZma'] = np.where(Zma > 0, Zma, 0)
    df['negXma'] = np.where(Xma < 0, abs(Xma), 0)
    df['negYma'] = np.where(Yma < 0, abs(Yma), 0)
    df['negZma'] = np.where(Zma < 0, abs(Zma), 0)
    df=df.drop([Xma.columns[0],Yma.columns[0],Zma.columns[0]], axis=1)
    mmMax = df.max().max()
    df=df/mmMax
    df['Xma'] = df['posXma'] if xPos else df['negXma']
    df['Yma'] = df['posYma'] if yPos else df['negYma']
    df['Zma'] = df['posZma'] if zPos else df['negZma']
    totalXYZ = df[['Xma','Yma','Zma']].sum(axis=1)
    df['scale'] = preprocessing.MinMaxScaler().fit_transform(totalXYZ.values.reshape(-1, 1))
    if normalizeAcrossSpecies:
        df = dfspecies.join(df)
        df = df[df['species']==species]
    return(df[['Xma','Yma','Zma','scale']])




posS = RgbMa(df, 'Tegu', 'dac', xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True)
posD = RgbMa(df, 'Opossum', 'dac', xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True)
negS = RgbMa(df, 'Tegu', 'dac', xPos=False, yPos=False, zPos=False, normalizeAcrossSpecies=True)
negD = RgbMa(df, 'Opossum', 'dac', xPos=False, yPos=False, zPos=False, normalizeAcrossSpecies=True)

plt.close('all')


fig = plt.figure(figsize=[12,12], constrained_layout=True)
ax0 = fig.add_subplot(221, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(222, projection='3d', proj_type = 'ortho')
ax2 = fig.add_subplot(223, projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(224, projection='3d', proj_type = 'ortho')


# 'Tegu Stance R: Prn, G: Add, B: Ext'
# 'Opossum Stance R: Prn, G: Add, B: Ext'

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), viewXZ)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), viewXZ)
format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), viewXZ)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), viewXZ)

addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax2, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax3, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=posS['scale']*15, c=posS[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax1.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=posD['scale']*15, c=posD[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax2.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=posS['scale']*15, c=negS[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax3.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=posD['scale']*15, c=negD[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')

connection4 = fig.canvas.mpl_connect('motion_notify_event', on_move4)

plt.show()

#threshold, then pca?

# plt.savefig('dsc.png', dpi=150, transparent=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### CH4 figure: trajectories

In [1015]:


# tegu_vecs = [[[t1_l.at[50, plot_x],t1_h.at[50, plot_x]],[t1_l.at[50, plot_y],t1_h.at[50, plot_y]],[t1_l.at[50, plot_z],t1_h.at[50, plot_z]]],
#                 [[t2_l.at[50, plot_x],t2_h.at[50, plot_x]],[t2_l.at[50, plot_y],t2_h.at[50, plot_y]],[t2_l.at[50, plot_z],t2_h.at[50, plot_z]]],
#                 [[t3_l.at[50, plot_x],t3_h.at[50, plot_x]],[t3_l.at[50, plot_y],t3_h.at[50, plot_y]],[t3_l.at[50, plot_z],t3_h.at[50, plot_z]]],
#                 [[t4_l.at[50, plot_x],t4_h.at[50, plot_x]],[t4_l.at[50, plot_y],t4_h.at[50, plot_y]],[t4_l.at[50, plot_z],t4_h.at[50, plot_z]]],
#                ]
# opossum_vecs = [[[p1_l.at[50, plot_x],p1_h.at[50, plot_x]],[p1_l.at[50, plot_y],p1_h.at[50, plot_y]],[p1_l.at[50, plot_z],p1_h.at[50, plot_z]]],
#                 [[p2_l.at[50, plot_x],p2_h.at[50, plot_x]],[p2_l.at[50, plot_y],p2_h.at[50, plot_y]],[p2_l.at[50, plot_z],p2_h.at[50, plot_z]]],
#                 [[p3_l.at[50, plot_x],p3_h.at[50, plot_x]],[p3_l.at[50, plot_y],p3_h.at[50, plot_y]],[p3_l.at[50, plot_z],p3_h.at[50, plot_z]]],
#                 [[p4_l.at[50, plot_x],p4_h.at[50, plot_x]],[p4_l.at[50, plot_y],p4_h.at[50, plot_y]],[p4_l.at[50, plot_z],p4_h.at[50, plot_z]]],
#                ]

viewYZ = (0,0)
viewXZ = (0,90)
viewXY = (90,90)
view3Q = (45,45)
view3Qst = (-45,45)
view3Qsw = (135,45)

vX, vY = viewXY

plt.close('all')
plt.rcParams['grid.linewidth'] = 1
plt.rcParams['grid.color'] = 'lightgrey'
plt.rcParams['axes.facecolor'] = 'white'



fig = plt.figure(figsize=[10,10], constrained_layout=True)
ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')
format3dPlot(ax, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)

# ax.scatter(lo_df[plot_x],lo_df[plot_y],lo_df[plot_z], s=100, c=lo_df, marker='v', depthshade=False)
# ax.scatter(hi_df[plot_x],hi_df[plot_y],hi_df[plot_z], s=100, c=hi_df, marker='^', depthshade=False)
ax.grid(True)
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

# for i in range(0,4):
#     for j in range(0,2):
#         ax.text(tegu_vecs[i][0][j],tegu_vecs[i][1][j],tegu_vecs[i][2][j], f'T$^{str(i+1)}$', ha='center', va='bottom', fontsize='large', color='teal')
#         ax.text(opossum_vecs[i][0][j],opossum_vecs[i][1][j],opossum_vecs[i][2][j], f'O$^{str(i+1)}$', ha='center', va='bottom', fontsize='large', color='orange')
#     ax.plot(tegu_vecs[i][0],tegu_vecs[i][1],tegu_vecs[i][2], c='teal', linewidth=0.5, linestyle='dotted')
#     ax.plot(opossum_vecs[i][0],opossum_vecs[i][1],opossum_vecs[i][2], c='orange', linewidth=0.5, linestyle='dotted')

plt.suptitle('Sprawling Space')
ax.set_xlabel('Spine Bending')
ax.set_ylabel('Long-Axis Rotation')
ax.set_zlabel('Body Height')

plt.show()

# plt.savefig('both3Q.png', dpi=150, transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Figure: space plots

In [35]:
def divergingTranslation(df, species, joint, normalizeAcrossSpecies=True):
    df = df.copy()
    dfspecies = df.filter(items=['species'])
    if not normalizeAcrossSpecies: 
        df = df[df['species']==species]
    df = df.loc[:,[joint+'.tx',joint+'.ty',joint+'.tz']]
    df = df.replace(np.NaN, 0)
    tMax = df.max().max()
    tMin = df.min().min()
    MaxMinMax = max((abs(tMax),abs(tMin)))
    df /= MaxMinMax
    df.columns=['xma','yma','zma']
    if normalizeAcrossSpecies:
        df = dfspecies.join(df)
        df = df[df['species']==species]
    return(df)
def divergingMa(df, species, muscleName, normalizeAcrossSpecies=True):
    df = df.copy()
    dfspecies = df.filter(items=['species'])
    if not normalizeAcrossSpecies: 
        df = df[df['species']==species]
    df = df.loc[:,[muscleName+'_Xma',muscleName+'_Yma',muscleName+'_Zma']]
    df = df.replace(np.NaN, 0)
    mmMax = df.max().max()
    mmMin = df.min().min()
    MaxMinMax = max((abs(mmMax),abs(mmMin)))
    df /= MaxMinMax
    df.columns=['xma','yma','zma']
    if normalizeAcrossSpecies:
        df = dfspecies.join(df)
        df = df[df['species']==species]
    return(df)

def sort_by_keys(dictionary):
    sorted_keys = sorted(dictionary.keys())
    new_dict = {key:dictionary[key] for key in sorted_keys}
    return new_dict
def makeSubGridSpec(outer, i):
    gs = gridspec.GridSpecFromSubplotSpec(3, 2, subplot_spec=outer[i], wspace=0, hspace=0) 
    return gs

def plotDiverging(fig, gridspec, i, df, muscleName, niceName='', normalizeAcrossSpecies=True, scale=1):
    scale = scale*10
    niceName = niceName if niceName else muscleName
    muscleName = 'model_'+muscleName+'_glen'
    dfD = df.loc[df['species']=='Opossum']
    dfS = df.loc[df['species']=='Tegu']
    map_D = divergingMa(df, 'Opossum' ,muscleName, normalizeAcrossSpecies)
    map_S = divergingMa(df, 'Tegu' ,muscleName, normalizeAcrossSpecies)

    plt.rcParams['grid.linewidth'] = 0
    plt.rcParams['grid.color'] = 'lightgrey'
    plt.rcParams['axes.facecolor'] = 'white'
    maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()
    ax0 = fig.add_subplot(gridspec[i][0], projection='3d', proj_type = 'ortho')
    ax1 = fig.add_subplot(gridspec[i][1], projection='3d', proj_type = 'ortho')
    ax2 = fig.add_subplot(gridspec[i][2], projection='3d', proj_type = 'ortho')
    ax3 = fig.add_subplot(gridspec[i][3], projection='3d', proj_type = 'ortho')
    ax4 = fig.add_subplot(gridspec[i][4], projection='3d', proj_type = 'ortho')
    ax5 = fig.add_subplot(gridspec[i][5], projection='3d', proj_type = 'ortho')

    format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)

    addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
    addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
    addCosGrid(ax2, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
    addCosGrid(ax3, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
    addCosGrid(ax4, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)
    addCosGrid(ax5, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='grey', linewidths=0.1)

    # ax0.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'])
    ax1.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(map_D.xma)*scale, c=map_D.xma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
    # ax2.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'])
    ax3.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(map_D.yma)*scale, c=map_D.yma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
    # ax4.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'])
    ax5.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(map_D.zma)*scale, c=map_D.zma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
    for axis in [ax0, ax1, ax2, ax3, ax4, ax5]:
        axis.axis('off')
        axis.dist = 6
    if i in [0,1,2]:
        ax0.set_title('Tegu')
        ax1.set_title('Opossum')
    ax0.annotate(niceName, (1.9, 1.1), xycoords='axes fraction', va='center', ha='center',annotation_clip=False, weight='bold', fontsize='medium')
    ax0.annotate('X', (1.9, 0.65), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold')
    ax0.annotate('+ Pronation  ', (1.9, 0.5), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red')
    ax0.annotate('  Supination -', (1.9, 0.5), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue')
    ax2.annotate('Y', (1.9, 0.65), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold')
    ax2.annotate('+ Elevation  ', (1.9, 0.5), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red')
    ax2.annotate('  Depression -', (1.9, 0.5), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue')
    ax4.annotate('Z', (1.9, 0.65), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold')
    ax4.annotate('+ Protraction  ', (1.9, 0.5), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red')
    ax4.annotate('  Retraction -', (1.9, 0.5), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue')




def plot6gridMMAs(df, muscle_name_dict):
    plt.close('all')
    fig = plt.figure(figsize=[18,10], constrained_layout=True)
    outer = gridspec.GridSpec(2,3, figure=fig)
    indices =  range(len(muscle_name_dict))
    gs = [makeSubGridSpec(outer,i) for i in indices]                 
    for i, (k, v) in enumerate(muscle_name_dict.items(), 0):
        plotDiverging(fig, gs, i, df, k, v, normalizeAcrossSpecies=True, scale=0.1)
    

rob_muscles = list(set([col.rsplit('_',1)[0].split('_',1)[-1].rsplit('_',1)[0] for col in rob85R.columns if ('length' not in col)&('elbo' not in col)&('acro' not in col)]))
        
muscle_name_dict = { 'bic': 'm. biceps brachii',
                     'bicr': 'm. biceps brachii (radial insertion)',
                     'cbb': 'm. coracobrachialis brevis',
                     'cbl': 'm. coracobrachialis longus',
                     'dac': 'm. deltoideus acromialis',
                     'dsc': 'm. deltoideus scapularis',
                     'isp': 'm. infraspinatus',
                     'lat_avg': 'm. latissimus dorsi',
                     'lat_cdl': 'm. latissimus dorsi (caudal)',
                     'lat_crn': 'm. latissimus dorsi (cranial)',
                     'lat_int': 'm. latissimus (intermediate)',
                     'pec_avg': 'm. pectoralis',
                     'pec_cdl': 'm. pectoralis (caudal)',
                     'pec_crn': 'm. pectoralis (cranial)',
                     'pec_int': 'm. pectoralis (intermediate)',
                     'sbs': 'm. subscapularis',
                     'ssp': 'm. supraspinatus',
                     'tmj': 'm. teres major',
                     'tmn': 'm. teres minor',
                     'trs': 'm. triceps scapularis'}
muscle_name_dict_colnames = {'model_'+key+'_glen':muscle_name_dict[key] for key in muscle_name_dict.keys()}

In [476]:
def plotDivergingMini(fig, gridspec, i, df, muscleName, niceName='', normalizeAcrossSpecies=True, scale=1):
    scale = scale*10
    niceName = niceName if niceName else muscleName
    muscleName = 'model_'+muscleName+'_glen'
    dfD = df.loc[df['species']=='Opossum']
    dfS = df.loc[df['species']=='Tegu']
    map_D = divergingMa(df, 'Opossum' ,muscleName, normalizeAcrossSpecies)
    map_S = divergingMa(df, 'Tegu' ,muscleName, normalizeAcrossSpecies)
    plt.rcParams['grid.linewidth'] = 0
    plt.rcParams['grid.color'] = 'lightgrey'
    plt.rcParams['axes.facecolor'] = 'white'
    maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()
    ax0 = fig.add_subplot(gridspec[i][0])
    ax1 = fig.add_subplot(gridspec[i][1], projection='3d', proj_type = 'ortho')
    ax2 = fig.add_subplot(gridspec[i][2], projection='3d', proj_type = 'ortho')
    ax3 = fig.add_subplot(gridspec[i][3], projection='3d', proj_type = 'ortho')
    ax4 = fig.add_subplot(gridspec[i][4], projection='3d', proj_type = 'ortho')
    ax5 = fig.add_subplot(gridspec[i][5], projection='3d', proj_type = 'ortho')
    ax6 = fig.add_subplot(gridspec[i][6], projection='3d', proj_type = 'ortho')

    format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)


    addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)
    addCosGrid(ax2, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)
    addCosGrid(ax3, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)
    addCosGrid(ax4, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)
    addCosGrid(ax5, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)
    addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)


#     ax1.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'])
    ax2.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(map_D.xma)*scale, c=map_D.xma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
#     ax3.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'])
    ax4.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(map_D.yma)*scale, c=map_D.yma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
#     ax5.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'])
    ax6.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(map_D.zma)*scale, c=map_D.zma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
    for axis in [ax0, ax1, ax2, ax3, ax4, ax5, ax6]:
        axis.axis('off')
        axis.dist = 5
#     if i in [0,1,2]:
#         ax0.set_title('Tegu')
#         ax1.set_title('Opossum')
    ax0.annotate(niceName, (1, 0.5), xycoords='axes fraction', va='center', ha='right',annotation_clip=False, weight='bold', fontsize='medium', fontproperties=roboto_bold)
    if i==0:
        for axpair in [(ax1,ax2),(ax3,ax4),(ax5,ax6)]:
            axpair[0].annotate('Tegu', (0.5, 3), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', fontsize='small', fontproperties=roboto_regular)
            axpair[1].annotate('Opossum', (0.5, 3), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', fontsize='small', fontproperties=roboto_regular)

        ax1.annotate('X', (2, 2.5), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black',  fontproperties=roboto_black)
        ax1.annotate('+ Pronation  ', (2, 2), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red', fontproperties=roboto_bold_condensed)
        ax1.annotate('  Supination -', (2, 2), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue', fontproperties=roboto_bold_condensed)
        ax3.annotate('Y', (2, 2.5), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold', fontproperties=roboto_black)
        ax3.annotate('+ Elevation  ', (2, 2), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red' , fontproperties=roboto_bold_condensed)
        ax3.annotate('  Depression -', (2, 2), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue', fontproperties=roboto_bold_condensed)
        ax5.annotate('Z', (2, 2.5), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold', fontproperties=roboto_black)
        ax5.annotate('+ Protraction  ', (2, 2), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red' , fontproperties=roboto_bold_condensed)
        ax5.annotate('  Retraction -', (2, 2), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue', fontproperties=roboto_bold_condensed)


def plotMiniMMAs(df, muscle_name_dict):
    plt.close('all')
    fig = plt.figure(figsize=[15,12], constrained_layout=True)
    outer = gridspec.GridSpec(12,1, figure=fig, hspace=0.5)
    indices =  range(len(muscle_name_dict))
    gs = [gridspec.GridSpecFromSubplotSpec(1, 7, subplot_spec=outer[i])  for i in indices]                 
    for i, (k, v) in enumerate(muscle_name_dict.items(), 0):
        plotDivergingMini(fig, gs, i, df, k, v, normalizeAcrossSpecies=True, scale=0.1)

    fig.set_constrained_layout_pads(w_pad=2./72., h_pad=8./72.,
        hspace=0., wspace=0.)
plotMiniMMAs(dfOS, {k: muscle_name_dict[k] for k in ('pec_avg', 'lat_avg', 'dsc','dac','trs','bic','ssp', 'isp', 'tmn','sbs','cbb','tmj')})
plt.savefig(os.path.join('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/spacePlots/mmaTable.png'), dpi=300, transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-476-e72c38047992>:78: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  plt.savefig(os.path.join('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/spacePlots/mmaTable.png'), dpi=300, transparent=True)


In [37]:
def plotDivergingMini3up(fig, gridspec, i, df, muscleName, niceName='', normalizeAcrossSpecies=True, scale=1):
    length = len(gridspec)
    scale = scale*10
    niceName = niceName if niceName else muscleName
    muscleName = 'model_'+muscleName+'_glen'
    dfD = df.loc[df['species']=='Opossum']
    dfS = df.loc[df['species']=='Tegu']
    map_D = divergingMa(df, 'Opossum' ,muscleName, normalizeAcrossSpecies)
    map_S = divergingMa(df, 'Tegu' ,muscleName, normalizeAcrossSpecies)
    plt.rcParams['grid.linewidth'] = 0
    plt.rcParams['grid.color'] = 'lightgrey'
    plt.rcParams['axes.facecolor'] = 'white'
    maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()
    ax0 = fig.add_subplot(gridspec[i][3])
    ax1 = fig.add_subplot(gridspec[i][0], projection='3d', proj_type = 'ortho')
    ax2 = fig.add_subplot(gridspec[i][1], projection='3d', proj_type = 'ortho')
    ax3 = fig.add_subplot(gridspec[i][2], projection='3d', proj_type = 'ortho')
    ax4 = fig.add_subplot(gridspec[i][4], projection='3d', proj_type = 'ortho')
    ax5 = fig.add_subplot(gridspec[i][5], projection='3d', proj_type = 'ortho')
    ax6 = fig.add_subplot(gridspec[i][6], projection='3d', proj_type = 'ortho')

    format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)
    format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view3Qpos)


    addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)
    addCosGrid(ax2, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)
    addCosGrid(ax3, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)
    addCosGrid(ax4, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)
    addCosGrid(ax5, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)
    addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (mincrz,maxcrz), 5, xLevels=1, color='lightgrey', linewidths=0.05)


    ax1.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=abs(map_S.xma)*scale, c=map_S.xma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
    ax4.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(map_D.xma)*scale, c=map_D.xma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
    ax2.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=abs(map_S.xma)*scale, c=map_S.yma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')    
    ax5.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(map_D.yma)*scale, c=map_D.yma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
    ax3.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=abs(map_S.xma)*scale, c=map_S.zma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
    ax6.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=abs(map_D.zma)*scale, c=map_D.zma, cmap='seismic', vmin=-1, vmax=1, depthshade=False, edgecolors='None')
    for axis in [ax0, ax1, ax2, ax3, ax4, ax5, ax6]:
        axis.axis('off')
        axis.dist = 6

    hOffset = 0.4
    vOffsetA = 2.25
    vOffsetP = 1.75
    a_fsize='12'
    p_fsize='10'
    ax0.annotate(niceName, (0.5, 0.5), xycoords='axes fraction', va='center', ha='center',annotation_clip=False, weight='bold', fontsize='medium', fontproperties=roboto_italic)
    if i==0:
        ax2.annotate('Tegu', (hOffset, 3), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', fontsize='large', fontproperties=roboto_black)
        ax5.annotate('Opossum', (hOffset, 3), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', fontsize='large', fontproperties=roboto_black)

        for axtriple in [(ax1,ax2, ax3),(ax4,ax5,ax6)]:
            axtriple[0].annotate('X', (hOffset, vOffsetA), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black',  fontproperties=roboto_black, fontsize=a_fsize)
            axtriple[0].annotate('+ Pronation  ', (hOffset, vOffsetP), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red', fontproperties=roboto_bold_condensed, fontsize=p_fsize)
            axtriple[0].annotate('  Supination -', (hOffset, vOffsetP), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue', fontproperties=roboto_bold_condensed, fontsize=p_fsize)
            axtriple[1].annotate('Y', (hOffset, vOffsetA), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold', fontproperties=roboto_black, fontsize=a_fsize)
            axtriple[1].annotate('+ Elevation  ', (hOffset, vOffsetP), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red' , fontproperties=roboto_bold_condensed, fontsize=p_fsize)
            axtriple[1].annotate('  Depression -', (hOffset, vOffsetP), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue', fontproperties=roboto_bold_condensed, fontsize=p_fsize)
            axtriple[2].annotate('Z', (hOffset, vOffsetA), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold', fontproperties=roboto_black, fontsize=a_fsize)
            axtriple[2].annotate('+ Protraction  ', (hOffset, vOffsetP), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red' , fontproperties=roboto_bold_condensed, fontsize=p_fsize)
            axtriple[2].annotate('  Retraction -', (hOffset, vOffsetP), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue', fontproperties=roboto_bold_condensed, fontsize=p_fsize)
    if i==length-1:
        for axtriple in [(ax1,ax2, ax3),(ax4,ax5,ax6)]:
            axtriple[0].annotate('X', (hOffset, -vOffsetA/1.5), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black',  fontproperties=roboto_black, fontsize=a_fsize)
            axtriple[0].annotate('+ Pronation  ', (hOffset, -vOffsetP/2), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red', fontproperties=roboto_bold_condensed, fontsize=p_fsize)
            axtriple[0].annotate('  Supination -', (hOffset, -vOffsetP/2), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue', fontproperties=roboto_bold_condensed, fontsize=p_fsize)
            axtriple[1].annotate('Y', (hOffset, -vOffsetA/1.5), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold', fontproperties=roboto_black, fontsize=a_fsize)
            axtriple[1].annotate('+ Elevation  ', (hOffset, -vOffsetP/2), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red' , fontproperties=roboto_bold_condensed, fontsize=p_fsize)
            axtriple[1].annotate('  Depression -', (hOffset, -vOffsetP/2), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue', fontproperties=roboto_bold_condensed, fontsize=p_fsize)
            axtriple[2].annotate('Z', (hOffset, -vOffsetA/1.5), xycoords='axes fraction', va='center', ha='center', annotation_clip=False, color='black', weight='bold', fontproperties=roboto_black, fontsize=a_fsize)
            axtriple[2].annotate('+ Protraction  ', (hOffset, -vOffsetP/2), xycoords='axes fraction', va='center', ha='right', annotation_clip=False, color='red' , fontproperties=roboto_bold_condensed, fontsize=p_fsize)
            axtriple[2].annotate('  Retraction -', (hOffset, -vOffsetP/2), xycoords='axes fraction', va='center', ha='left', annotation_clip=False, color='blue', fontproperties=roboto_bold_condensed, fontsize=p_fsize)
        

def plotMiniMMAs3up(df, muscle_name_dict):
    plt.close('all')
    fig = plt.figure(figsize=[15,12], constrained_layout=True)
    outer = gridspec.GridSpec(13,1, figure=fig, hspace=0.5)
    indices =  range(len(muscle_name_dict))
    gs = [gridspec.GridSpecFromSubplotSpec(1, 7, subplot_spec=outer[i])  for i in indices]                 
    for i, (k, v) in enumerate(muscle_name_dict.items(), 0):
        plotDivergingMini3up(fig, gs, i, df, k, v, normalizeAcrossSpecies=False, scale=0.1)

    fig.set_constrained_layout_pads(w_pad=2./72., h_pad=8./72.,
        hspace=0., wspace=0.)
plotMiniMMAs3up(dfOS, {k: muscle_name_dict[k] for k in ('pec_avg', 'lat_avg', 'dsc','dac','trs','bic','ssp', 'isp', 'tmn','sbs','cbb','cbl','tmj')})
plt.savefig(os.path.join('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/spacePlots/mmaTable3up.png'), dpi=300, transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-37-fd0a9437f326>:93: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  plt.savefig(os.path.join('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/spacePlots/mmaTable3up.png'), dpi=300, transparent=True)


### Figure: extrinsics and superficials batch

In [478]:
plot6gridMMAs(dfOS, {k: muscle_name_dict[k] for k in ('pec_avg', 'lat_avg', 'dsc','dac','trs','bic')})
plt.savefig(os.path.join('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/spacePlots/extrinsics.png'), dpi=300, transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Figure: intrinsics

In [153]:
plot6gridMMAs(dfOS, {k: muscle_name_dict[k] for k in ('ssp', 'isp', 'tmn','sbs','cbb','tmj')})
plt.savefig(os.path.join('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/spacePlots/intrinsics.png'), dpi=300, transparent=True)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Figure: Plot divisions of pec and lat for supplement

In [1494]:
plot6gridMMAs(dfOS, {k: muscle_name_dict[k] for k in ('pec_crn', 'pec_int', 'pec_cdl','lat_crn','lat_int','lat_cdl')})
plt.savefig(os.path.join('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/spacePlots/supp_peclat_crancaud.png'), dpi=300, transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Table: correlation matrix

In [1931]:
pd.set_option('display.max_rows', 20)
def plotCorrAxis(df, species, axis, r_threshold=0.7):
    plt.close('all')
    division_names = [name for name in df.columns if ('int' in name)|('cdl' in name)|('crn' in name)|('bicr' in name)]
    has_divisions = dfOS.loc[dfOS['species']==species].filter(regex='model.*glen_[XYZ]ma').filter(regex='^|(?!.*cdl).*$')
    names_to_drop = list(set(division_names).intersection(set(has_divisions)))
    df_for_corr = has_divisions.drop(names_to_drop, axis=1)
    corr=df_for_corr.filter(like=axis.upper()+'ma').corr()
    corr[abs(corr)<r_threshold] = np.nan
#     corr[corr==1.0] = np.nan
#     corr = corr.dropna(axis=0, how='all')
#     corr = corr.dropna(axis=1, how='all')
    mask = np.triu(np.ones_like(corr, dtype=bool))
    fig, ax = plt.subplots(figsize=(10,10))
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    sns.heatmap(corr, mask=None, cmap='seismic', vmax=1, vmin=-1, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.tight_layout()
    return ax
plotCorrAxis(dfOS, 'Opossum','x', 0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [1859]:
def plotCorr(df, species, muscle_name_dict_colnames, r_threshold=0.7):
    plt.close('all')
    division_names = [name for name in df.columns if ('int' in name)|('cdl' in name)|('crn' in name)|('bicr' in name)]
    has_divisions = dfOS.loc[dfOS['species']==species].filter(regex='model.*glen_[XYZ]ma').filter(regex='^|(?!.*cdl).*$')
    names_to_drop = list(set(division_names).intersection(set(has_divisions)))
    df_for_corr = has_divisions.drop(names_to_drop, axis=1)
    col_list = df_for_corr.columns
    label_list = [muscle_name_dict_colnames[key.rsplit('_',1)[0]] for key in col_list]
    unique_label_list=[]
    for label in label_list:
        if label not in unique_label_list:
            unique_label_list.append(label)
    corrX=df_for_corr.filter(like='Xma').corr()
    corrY=df_for_corr.filter(like='Yma').corr()
    corrZ=df_for_corr.filter(like='Zma').corr()

#     corr[corr==1.0] = np.nan
#     corr = corr.dropna(axis=0, how='all')
#     corr = corr.dropna(axis=1, how='all')
    mask = np.triu(np.ones_like(corrX, dtype=bool))
    fig, ax = plt.subplots(1,3, figsize=(24,8))
    for i, corr in enumerate([corrX, corrY, corrZ]):
        corr[abs(corr)<r_threshold] = np.nan
        sns.heatmap(corr, mask=mask, cmap='seismic', vmax=1, vmin=-1, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, xticklabels=unique_label_list, yticklabels=unique_label_list, ax=ax[i])
    plt.tight_layout()
    return ax

In [1860]:
plotCorr(dfOS, 'Opossum', muscle_name_dict_colnames,0.7)
plt.savefig(os.path.join('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/corr_opossum.png'), dpi=300, transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1873]:
for i, (corr, color) in enumerate(zip(['a', 'b', 'c'],['r','g','b'])):
    print(i, corr, color)

0 a r
1 b g
2 c b


In [1924]:
from matplotlib.colors import ListedColormap

def plotCorrRGB(df, species, muscle_name_dict_colnames, r_threshold=0.7):
    plt.close('all')
    division_names = [name for name in df.columns if ('int' in name)|('cdl' in name)|('crn' in name)|('bicr' in name)]
    has_divisions = dfOS.loc[dfOS['species']==species].filter(regex='model.*glen_[XYZ]ma').filter(regex='^|(?!.*cdl).*$')
    names_to_drop = list(set(division_names).intersection(set(has_divisions)))
    df_for_corr = has_divisions.drop(names_to_drop, axis=1)
    col_list = df_for_corr.columns
    label_list = [muscle_name_dict_colnames[key.rsplit('_',1)[0]] for key in col_list]
    unique_label_list=[]
    for label in label_list:
        if label not in unique_label_list:
            unique_label_list.append(label)
    corrX=df_for_corr.filter(like='Xma').corr()
    corrY=df_for_corr.filter(like='Yma').corr()
    corrZ=df_for_corr.filter(like='Zma').corr()
    corrX_pos=corrX.copy()
    corrY_pos=corrY.copy()
    corrZ_pos=corrZ.copy()
    corrX_neg=corrX.copy()
    corrY_neg=corrY.copy()
    corrZ_neg=corrZ.copy()
    corrX_pos[corrX_pos<=0.0] = np.nan
    corrY_pos[corrY_pos<=0.0] = np.nan
    corrZ_pos[corrZ_pos<=0.0] = np.nan
    corrX_neg[corrX_neg>=0.0] = np.nan
    corrY_neg[corrY_neg>=0.0] = np.nan
    corrZ_neg[corrZ_neg>=0.0] = np.nan
    mask = np.triu(np.ones_like(corrX, dtype=bool))
    fig, ax = plt.subplots(2,3, figsize=(24,8))
    for i, (corr,hsl) in enumerate(zip([corrX_pos, corrY_pos, corrZ_pos],[(0,100,54),(120,100,88),(240,100,30)])):
        corr[abs(corr)<r_threshold] = np.nan
        cmap = sns.diverging_palette(hsl[0], hsl[0], 100, as_cmap=True)
        my_cmap = cmap(np.arange(cmap.N))
        my_cmap[:,-1] = np.logspace(0, 1, cmap.N)/10
        my_cmap = ListedColormap(my_cmap)
    

        sns.heatmap(corr, mask=mask, cmap=my_cmap, vmax=1, vmin=0, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=False, xticklabels=unique_label_list, yticklabels=unique_label_list, ax=ax[0,i])
    for i, (corr,hsl) in enumerate(zip([corrX_neg, corrY_neg, corrZ_neg],[(0,100,54),(120,100,88),(240,100,30)])):
        corr[abs(corr)<r_threshold] = np.nan
        cmap = sns.diverging_palette(hsl[0], hsl[0], 100, as_cmap=True)
        my_cmap = cmap(np.arange(cmap.N))
        my_cmap[:,-1] = np.logspace(1, 0, cmap.N)/10
        my_cmap = ListedColormap(my_cmap)
        sns.heatmap(corr, mask=mask, cmap=my_cmap, vmax=0, vmin=-1, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=False, xticklabels=unique_label_list, yticklabels=unique_label_list, ax=ax[1,i])
    for i1 in [0,1]:
        for i2 in [0,1,2]:
            for tick in ax[i1,i2].xaxis.get_major_ticks()+ax[i1,i2].yaxis.get_major_ticks():
                tick.label.set_fontsize(7) 
    plt.tight_layout()
    return ax
plotCorrRGB(dfOS, 'Opossum', muscle_name_dict_colnames,0)
plt.savefig(os.path.join('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/corr_opossum_RGB.png'), dpi=300, transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1926]:
np.linspace(1, 0, 255)

array([1.        , 0.99606299, 0.99212598, 0.98818898, 0.98425197,
       0.98031496, 0.97637795, 0.97244094, 0.96850394, 0.96456693,
       0.96062992, 0.95669291, 0.95275591, 0.9488189 , 0.94488189,
       0.94094488, 0.93700787, 0.93307087, 0.92913386, 0.92519685,
       0.92125984, 0.91732283, 0.91338583, 0.90944882, 0.90551181,
       0.9015748 , 0.8976378 , 0.89370079, 0.88976378, 0.88582677,
       0.88188976, 0.87795276, 0.87401575, 0.87007874, 0.86614173,
       0.86220472, 0.85826772, 0.85433071, 0.8503937 , 0.84645669,
       0.84251969, 0.83858268, 0.83464567, 0.83070866, 0.82677165,
       0.82283465, 0.81889764, 0.81496063, 0.81102362, 0.80708661,
       0.80314961, 0.7992126 , 0.79527559, 0.79133858, 0.78740157,
       0.78346457, 0.77952756, 0.77559055, 0.77165354, 0.76771654,
       0.76377953, 0.75984252, 0.75590551, 0.7519685 , 0.7480315 ,
       0.74409449, 0.74015748, 0.73622047, 0.73228346, 0.72834646,
       0.72440945, 0.72047244, 0.71653543, 0.71259843, 0.70866

In [1925]:
np.logspace(1, 0, 255)/10

array([1.        , 0.99097567, 0.98203278, 0.97317059, 0.96438838,
       0.95568542, 0.947061  , 0.93851441, 0.93004495, 0.92165191,
       0.91333462, 0.90509239, 0.89692454, 0.8888304 , 0.8808093 ,
       0.87286058, 0.8649836 , 0.8571777 , 0.84944225, 0.8417766 ,
       0.83418013, 0.82665222, 0.81919223, 0.81179957, 0.80447363,
       0.79721379, 0.79001947, 0.78289008, 0.77582502, 0.76882372,
       0.7618856 , 0.75501009, 0.74819663, 0.74144466, 0.73475362,
       0.72812296, 0.72155214, 0.71504061, 0.70858785, 0.70219332,
       0.69585649, 0.68957686, 0.68335389, 0.67718708, 0.67107592,
       0.66501991, 0.65901855, 0.65307135, 0.64717782, 0.64133747,
       0.63554983, 0.62981442, 0.62413076, 0.6184984 , 0.61291687,
       0.60738571, 0.60190446, 0.59647267, 0.59108991, 0.58575572,
       0.58046966, 0.57523131, 0.57004024, 0.56489601, 0.5597982 ,
       0.55474639, 0.54974018, 0.54477914, 0.53986288, 0.53499098,
       0.53016304, 0.52537868, 0.52063748, 0.51593908, 0.51128

In [1811]:
test = dfOS.loc[(dfOS['glenoid.rz']<-60)&(dfOS['glenoid.ry']>20)]
# dfOS.loc[(dfOS['glenoid.rz']<-60)&(dfOS['glenoid.ry']>20)]['model_dac_glen_Zma'].plot()
# dfOS.loc[(dfOS['glenoid.rz']<-60)&(dfOS['glenoid.ry']>20)]['model_dsc_glen_Zma'].plot()
corr = signal.correlate(test['model_dsc_glen_Xma'].dropna(),test['model_dac_glen_Yma'].dropna())
corr/=(np.sign(corr)*np.)
plt.close('all')
fig,ax = plt.subplots()
ax.plot(corr)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1841]:
def plotCorrAll(df, species, muscle_name_dict_colnames, r_threshold=0.7):
    plt.close('all')
    division_names = [name for name in df.columns if ('int' in name)|('cdl' in name)|('crn' in name)|('bicr' in name)]
    has_divisions = dfOS.loc[dfOS['species']==species].filter(regex='model.*glen_[XYZ]ma').filter(regex='^|(?!.*cdl).*$')
    names_to_drop = list(set(division_names).intersection(set(has_divisions)))
    df_for_corr = has_divisions.drop(names_to_drop, axis=1)
    col_list = df_for_corr.columns
    x_dict = {key+'_Xma': muscle_name_dict_colnames[key]+' X' for key in muscle_name_dict_colnames}
    y_dict = {key+'_Yma': muscle_name_dict_colnames[key]+' Y' for key in muscle_name_dict_colnames}
    z_dict = {key+'_Zma': muscle_name_dict_colnames[key]+' Z' for key in muscle_name_dict_colnames}
    xyz_dict = x_dict.copy()
    xyz_dict.update(y_dict)
    xyz_dict.update(z_dict)
    xyz_dict = sort_by_keys(xyz_dict)
    label_list = [xyz_dict[key] for key in col_list]
    corr=df_for_corr.corr()

    mask = np.triu(np.ones_like(corr, dtype=bool))
    fig, ax = plt.subplots(figsize=(18,18))
    corr[abs(corr)<r_threshold] = np.nan
    sns.heatmap(corr, mask=mask, cmap='seismic', vmax=1, vmin=-1, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, xticklabels=label_list, yticklabels=label_list, ax=ax)
    for tick in ax.xaxis.get_major_ticks()+ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(8) 
    plt.tight_layout()
    return ax
plotCorrAll(dfOS, 'Opossum', muscle_name_dict_colnames,0.7)
plt.savefig(os.path.join('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/corr_opossum_all.png'), dpi=300, transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# SWAP CRX AND CRZ FOR ALL BELOW

In [1209]:
dfOS[(dfOS['glenoid.ry']>40)&(dfOS['species']=='Opossum')]
model_dac_glen.loc[5288]

species                Opossum
model_dsc_glen_Xma   -0.612285
model_dsc_glen_Yma    0.648245
model_dsc_glen_Zma    0.647396
Name: 5288, dtype: object

In [1218]:
model_dac_glen['model_dsc_glen_Xma'].mean()

-0.518973209007778

In [1212]:
dfOS.loc[5288]['glenoid.crz']
ax1.scatter(dfOS.loc[5288]['glenoid.rx'],dfOS.loc[5288]['glenoid.ry'],dfOS.loc[5288]['glenoid.crz'], s=100, c='yellow', vmin=0, vmax=1, depthshade=False, edgecolors='None')

ax3.scatter(dfOS.loc[5288]['glenoid.rx'],dfOS.loc[5288]['glenoid.ry'],dfOS.loc[5288]['glenoid.crz'], s=100, c='yellow', vmin=0, vmax=1, depthshade=False, edgecolors='None')

ax5.scatter(dfOS.loc[5288]['glenoid.rx'],dfOS.loc[5288]['glenoid.ry'],dfOS.loc[5288]['glenoid.crz'], s=100, c='yellow', vmin=0, vmax=1, depthshade=False, edgecolors='None')


In [ ]:
#center and superimpose joint alpha shapes
def centerJoints(df, species):
    df = df[df['species']==species[0]].copy()
    RCols = [col for col in df.columns if re.match(r'(c*R[xyz])', col)]
    for col in RCols:
        df[col+'_centered'] = df[col]-df[col].mean()
    return df
    
salCentered = centerJoints(df, 's')
didCentered = centerJoints(df, 'd')

rotationsCentered = getRotationRanges(salCentered, didCentered)
CmaxRx, CmaxcRz, CminRx, CmincRz, CmaxRy, CminRy, CmaxRz, CminRz = rotationsCentered.values()

alpha = 25
alphaSalVert,alphaSalEdge, alphaSalTri = alpha_shape_3D(np.array([salCentered.cRz_centered,salCentered.Ry_centered,salCentered.Rz_centered]).T,alpha)
alphaDidVert,alphaDidEdge, alphaDidTri = alpha_shape_3D(np.array([didCentered.cRz_centered,didCentered.Ry_centered,didCentered.Rz_centered]).T,alpha)


plt.close('all')


fig = plt.figure(figsize=[15,10], constrained_layout=True)
ax0 = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(122, projection='3d', proj_type = 'ortho')



format3dPlot(ax0, 'Tegu', (CmincRz, CmaxcRz), (CminRy, CmaxRy), (CminRz, CmaxRz), view3Q)
format3dPlot(ax1, 'Opossum', (CmincRz, CmaxcRz), (CminRy, CmaxRy), (CminRz, CmaxRz), view3Q)

addCosGrid(ax0, (CminRx,CmaxRx), (CminRy,CmaxRy), (CminRz,CmaxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (CminRx,CmaxRx), (CminRy,CmaxRy), (CminRz,CmaxRz), 5, xLevels=1, color='grey', linewidths=0.1)


ax0.plot_trisurf(salCentered.cRz_centered, salCentered.Ry_centered, alphaSalTri, salCentered.Rz_centered, shade=True, color=teguColor, linewidth=0, antialiased=True)
ax1.plot_trisurf(didCentered.cRz_centered, didCentered.Ry_centered, alphaDidTri, didCentered.Rz_centered, shade=True, color=opossumColor, linewidth=0, antialiased=True)

ax0.scatter(salCentered.cRz_centered,salCentered.Ry_centered,salCentered.Rz_centered, s=2, c=orangeYellowSeafoam[6], depthshade=False, edgecolors='None', alpha=0.95)
ax1.scatter(didCentered.cRz_centered,didCentered.Ry_centered,didCentered.Rz_centered, s=2, c=orangeYellowSeafoam[2], depthshade=False, edgecolors='None', alpha=0.95)


# ax0.add_patch(PolygonPatch(hullDXZ, fill=True, color='blue'))






connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

plt.show()
# plt.savefig('3q.png', dpi=150, transparent=True)

In [ ]:
scaled_Zma_cols = [col for col in scaled_t_ma_all_data.columns if col[-3:] == 'Zma']
colsForForr = scaled_Zma_cols
# dfSforCorr = dfS.filter(colsForForr)
# dfSforCorr = dfSforCorr.dropna(axis=1, how='all')
# corr = dfSforCorr.corr()
# corr.style.background_gradient(cmap='bwr')
dfDforCorr = dfD.filter(colsForForr)
dfDforCorr_colnames_raw = [col.split('_',1)[1].rsplit('_',1)[0] for col in colsForForr]
dfDforCorr.columns = dfDforCorr_colnames_raw
dfDforCorr = dfDforCorr.dropna(axis=1, how='all')
corr = dfDforCorr.corr()
corr.style.background_gradient(cmap='bwr')

In [ ]:
from scipy.spatial import Delaunay
import numpy as np
from collections import defaultdict

def alpha_shape_3D(pos, alpha):
    """
    Compute the alpha shape (concave hull) of a set of 3D points.
    Parameters:
        pos - np.array of shape (n,3) points.
        alpha - alpha value.
    return
        outer surface vertex indices, edge indices, and triangle indices
    """

    tetra = Delaunay(pos)
    # Find radius of the circumsphere.
    # By definition, radius of the sphere fitting inside the tetrahedral needs 
    # to be smaller than alpha value
    # http://mathworld.wolfram.com/Circumsphere.html
    tetrapos = np.take(pos,tetra.vertices,axis=0)
    normsq = np.sum(tetrapos**2,axis=2)[:,:,None]
    ones = np.ones((tetrapos.shape[0],tetrapos.shape[1],1))
    a = np.linalg.det(np.concatenate((tetrapos,ones),axis=2))
    Dx = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[1,2]],ones),axis=2))
    Dy = -np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,2]],ones),axis=2))
    Dz = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,1]],ones),axis=2))
    c = np.linalg.det(np.concatenate((normsq,tetrapos),axis=2))
    r = np.sqrt(Dx**2+Dy**2+Dz**2-4*a*c)/(2*np.abs(a))
    # Find tetrahedrals
    tetras = tetra.vertices[r<alpha,:]
    # triangles
    TriComb = np.array([(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)])
    Triangles = tetras[:,TriComb].reshape(-1,3)
    Triangles = np.sort(Triangles,axis=1)

    # Remove triangles that occurs twice, because they are within shapes
    TrianglesDict = defaultdict(int)
    for tri in Triangles:TrianglesDict[tuple(tri)] += 1
    Triangles=np.array([tri for tri in TrianglesDict if TrianglesDict[tri] ==1])
    #edges
    EdgeComb=np.array([(0, 1), (0, 2), (1, 2)])
    Edges=Triangles[:,EdgeComb].reshape(-1,2)
    Edges=np.sort(Edges,axis=1)
    Edges=np.unique(Edges,axis=0)

    Vertices = np.unique(Edges)
    return Vertices,Edges,Triangles

In [859]:

def divergingMa(df, species, muscleName, normalizeAcrossSpecies=True):
    df = df.copy()
    dfspecies = df.filter(items=['species'])
    if not normalizeAcrossSpecies: 
        df = df[df['species']==species[0]]
    df = df.filter(like=muscleName)
    df = df.replace(np.NaN, 0)
    mmMax = df.max().max()
    mmAbsMax = abs(df).max().max()
    mmMin = df.min().min()
    mmAbsMin = abs(df).min().min()
    MaxMinMax = max((abs(mmMax),abs(mmMin)))
    def MinMaxScalerPrescribedRange(X, min_input, max_input, min_output=0, max_output=1):
        X_std = (X - min_input) / (max_input - min_input)
        X_scaled = X_std * (max_output - min_output) + min_output
        return X_scaled
    Xma = df.filter(like='Xma')/MaxMinMax   #maxabs scale across all 3 axes
    Yma = df.filter(like='Yma')/MaxMinMax
    Zma = df.filter(like='Zma')/MaxMinMax
    df['Xmag'] = abs(Xma)   #magnitude of moment arm
    df['Ymag'] = abs(Yma)
    df['Zmag'] = abs(Zma)
    df['Xscale'] = MinMaxScalerPrescribedRange(Xma, -1, 1)    #get color mapping var by normalizing moment arms to lie between 0 and 1 (with -1 and 1 as min and max input values)
    df['Yscale'] = MinMaxScalerPrescribedRange(Yma, -1, 1)
    df['Zscale'] = MinMaxScalerPrescribedRange(Zma, -1, 1)
    if normalizeAcrossSpecies:
        df = dfspecies.join(df)
        df = df[df['species']==species[0]]
    return(df)



pec_crn_S = divergingMa(df, 'salvator', 'scaled_dac', normalizeAcrossSpecies=True)
pec_crn_D = divergingMa(df, 'didelphis', 'scaled_dac', normalizeAcrossSpecies=True)

plt.close('all')


fig = plt.figure(figsize=[12,18], constrained_layout=True)
ax0 = fig.add_subplot(321, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(322, projection='3d', proj_type = 'ortho')
ax2 = fig.add_subplot(323, projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(324, projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(325, projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(326, projection='3d', proj_type = 'ortho')


# 'Tegu Stance R: Prn, G: Add, B: Ext'
# 'Opossum Stance R: Prn, G: Add, B: Ext'

format3dPlot(ax0, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax1, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax2, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax3, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax4, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax5, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax2, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax3, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax4, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax5, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=pec_crn_S['Xmag']*30, c=pec_crn_S['Xscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=pec_crn_D['Xmag']*30, c=pec_crn_D['Xscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax2.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=pec_crn_S['Ymag']*30, c=pec_crn_S['Yscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax3.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=pec_crn_D['Ymag']*30, c=pec_crn_D['Yscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax4.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=pec_crn_S['Zmag']*30, c=pec_crn_S['Zscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax5.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=pec_crn_D['Zmag']*30, c=pec_crn_D['Zscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')

connection6 = fig.canvas.mpl_connect('motion_notify_event', on_move6)

plt.show()

# plt.savefig('dac_div_3Q.png', dpi=150, transparent=True)




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'mincRx' is not defined

In [ ]:
#color by momentarm
dfRgb = df.copy()
dfSrgb = df[df['species']=='s']
dfDrgb = df[df['species']=='d']

scaled_muscle_cols = [col for col in dfRgb.columns if col.split('_')[0] == 'scaled' and col[-2:] == 'ma']
dfRgb[scaled_muscle_cols] = dfRgb[scaled_muscle_cols].replace(np.NaN, 0)

lat_crn = dfRgb[scaled_muscle_cols].iloc[:, 3:6]
latmax = lat_crn.max().max()
latmin = lat_crn.min().min()

latNewX = lat_crn.scaled_lat_crn_Xma+abs(latmin)
latNewY = lat_crn.scaled_lat_crn_Yma+abs(latmin)
latNewZ = lat_crn.scaled_lat_crn_Zma+abs(latmin)

latNewXnorm = latNewX/(latmax+abs(latmin))
latNewYnorm = latNewY/(latmax+abs(latmin))
latNewZnorm = latNewZ/(latmax+abs(latmin))



# scaler = sklearn.preprocessing.MinMaxScaler().fit(dfRgb[scaled_muscle_cols])
# dfRgb[scaled_muscle_cols] = scaler.transform(dfRgb[scaled_muscle_cols])
# DrgbXYZ = scaler.transform(dfDrgb[['scaled_Tx','scaled_Ty','scaled_Tz']])

# # SrgbXYZ = np.array([SrgbX, SrgbY, SrgbZ]).reshape(3,len(SrgbX)).T
# # DrgbXYZ = np.array([DrgbX, DrgbY, DrgbZ]).reshape(3,len(DrgbX)).T
plt.close('all')


fig = plt.figure(figsize=[12,5], constrained_layout=True)
ax0 = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(122, projection='3d', proj_type = 'ortho')

format3dPlot(ax0, 'Tegu', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)
format3dPlot(ax1, 'Opossum', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=2, c='black', depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=2, c='black', depthshade=False, edgecolors='None')
ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=2, c=np.array([np.zeros(len(latNewXnorm)),latNewYnorm,np.zeros(len(latNewXnorm)),latNewXnorm]).T[dfSrgb.index], depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=2, c=np.array([np.zeros(len(latNewXnorm)),latNewYnorm,np.zeros(len(latNewXnorm)),latNewXnorm]).T[dfDrgb.index], depthshade=False, edgecolors='None')

connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

plt.show()

In [ ]:
 dfRgb[scaled_muscle_cols]

In [ ]:
#color by translation
dfRgb = df.copy()
# dfSrgb = df[df['species']=='s']
# dfDrgb = df[df['species']=='d']

scaled_Ts = dfRgb[['scaled_Tx','scaled_Ty','scaled_Tz']]
tMin = scaled_Ts.min().min()
tMax = scaled_Ts.max().max()

scalerMinMax = preprocessing.MinMaxScaler((tMin,tMax)).fit(dfRgb[['scaled_Tx','scaled_Ty','scaled_Tz']])
dfRgbMinMax = scalerMinMax.transform(dfRgb[['scaled_Tx','scaled_Ty','scaled_Tz']])
scaler01 = preprocessing.MinMaxScaler((0,1)).fit(dfRgbMinMax)
dfRgb01 = scaler01.transform(dfRgbMinMax)

dfSrgb = dfRgb01[dfRgb[dfRgb['species']=='s'].index]
dfDrgb = dfRgb01[dfRgb[dfRgb['species']=='d'].index]


plt.close('all')


fig = plt.figure(figsize=[12,5], constrained_layout=True)
ax0 = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(122, projection='3d', proj_type = 'ortho')

format3dPlot(ax0, 'Tegu', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)
format3dPlot(ax1, 'Opossum', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=3, c=dfSrgb, depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=3, c=dfDrgb, depthshade=False, edgecolors='None')

connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

plt.show()


In [ ]:
sklearn.preprocessing

In [ ]:
#PCA
from sklearn.decomposition import PCA

XYZ = df[['cRx','Ry','Rz']].to_numpy()
XYZ = sklearn.preprocessing.normalize(XYZ)
aPCA = PCA().fit(XYZ)

sXYZ = dfS[['cRx','Ry','Rz']].to_numpy()
sXYZ = sklearn.preprocessing.normalize(sXYZ)
sPCA = PCA().fit(sXYZ)

dXYZ = dfD[['cRx','Ry','Rz']].to_numpy()
dXYZ = sklearn.preprocessing.normalize(dXYZ)
dPCA = PCA().fit(dXYZ)


aPCAs = aPCA.transform(sXYZ).T
aPCAd = aPCA.transform(dXYZ).T
sPCAs = sPCA.transform(sXYZ).T
sPCAd = sPCA.transform(dXYZ).T
dPCAs = dPCA.transform(sXYZ).T
dPCAd = dPCA.transform(dXYZ).T

maxPC1a = max([max(aPCAs[0]),max(aPCAd[0])])
minPC1a = min([min(aPCAs[0]),min(aPCAd[0])])
maxPC2a = max([max(aPCAs[1]),max(aPCAd[1])])
minPC2a = min([min(aPCAs[1]),min(aPCAd[1])])
maxPC3a = max([max(aPCAs[2]),max(aPCAd[2])])
minPC3a = min([min(aPCAs[2]),min(aPCAd[2])])

maxPC1s = max([max(sPCAs[0]),max(sPCAd[0])])
minPC1s = min([min(sPCAs[0]),min(sPCAd[0])])
maxPC2s = max([max(sPCAs[1]),max(sPCAd[1])])
minPC2s = min([min(sPCAs[1]),min(sPCAd[1])])
maxPC3s = max([max(sPCAs[2]),max(sPCAd[2])])
minPC3s = min([min(sPCAs[2]),min(sPCAd[2])])

maxPC1d = max([max(dPCAs[0]),max(dPCAd[0])])
minPC1d = min([min(dPCAs[0]),min(dPCAd[0])])
maxPC2d = max([max(dPCAs[1]),max(dPCAd[1])])
minPC2d = min([min(dPCAs[1]),min(dPCAd[1])])
maxPC3d = max([max(dPCAs[2]),max(dPCAd[2])])
minPC3d = min([min(dPCAs[2]),min(dPCAd[2])])

plt.close('all')



fig = plt.figure(figsize=[12,5], constrained_layout=True)
ax0PCA = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1PCA = fig.add_subplot(122, projection='3d', proj_type = 'ortho')

format3dPlot(ax0PCA, 'Tegu', (minPC1a, maxPC1a), (minPC2a, maxPC2a), (minPC3a, maxPC3a), viewXY, mode='pca')
format3dPlot(ax1PCA, 'Opossum', (minPC1a, maxPC1a), (minPC2a, maxPC2a), (minPC3a, maxPC3a), viewXY, mode='pca')

ax0PCA.scatter(aPCAs[0],aPCAs[1],aPCAs[2], s=3, depthshade=True)
ax1PCA.scatter(aPCAd[0],aPCAd[1],aPCAd[2], s=3, depthshade=True)



plt.show()


In [ ]:
def alpha_shape(points, alpha, only_outer=True): #c/o "Harold" on stackoverflow
    """
    Compute the alpha shape (concave hull) of a set of points.
    :param points: np.array of shape (n, 3) points.
    :param alpha: alpha value.
    :param only_outer: boolean value to specify if we keep only the outer border
    or also inner edges.
    :return: set of (i,j) pairs representing edges of the alpha-shape. (i,j) are
    the indices in the points array.
    """
    assert points.shape[0] > 3, "Need at least four points"

    def add_edge(edges, i, j):
        """
        Add a line between the i-th and j-th points,
        if not in the set already
        """
        if (i, j) in edges or (j, i) in edges:
            # already added
            if only_outer:
                # if both neighboring triangles are in shape, it's not a boundary edge
                if (j, i) in edges:
                    edges.remove((j, i))
            return
        edges.add((i, j))

    tri = Delaunay(points)
    edges = set()
    # Loop over triangles:
    # ia, ib, ic, id = indices of corner points of the tetrahedron
    print(tri.vertices.shape)
    for ia, ib, ic, id in tri.vertices:
        pa = points[ia]
        pb = points[ib]
        pc = points[ic]
        pd = points[id]

        # Computing radius of tetrahedron Circumsphere
        # http://mathworld.wolfram.com/Circumsphere.html

        pa2 = np.dot(pa, pa)
        pb2 = np.dot(pb, pb)
        pc2 = np.dot(pc, pc)
        pd2 = np.dot(pd, pd)

        a = np.linalg.det(np.array([np.append(pa, 1), np.append(pb, 1), np.append(pc, 1), np.append(pd, 1)]))

        Dx = np.linalg.det(np.array([np.array([pa2, pa[1], pa[2], 1]),
                                     np.array([pb2, pb[1], pb[2], 1]),
                                     np.array([pc2, pc[1], pc[2], 1]),
                                     np.array([pd2, pd[1], pd[2], 1])]))

        Dy = - np.linalg.det(np.array([np.array([pa2, pa[0], pa[2], 1]),
                                       np.array([pb2, pb[0], pb[2], 1]),
                                       np.array([pc2, pc[0], pc[2], 1]),
                                       np.array([pd2, pd[0], pd[2], 1])]))

        Dz = np.linalg.det(np.array([np.array([pa2, pa[0], pa[1], 1]),
                                     np.array([pb2, pb[0], pb[1], 1]),
                                     np.array([pc2, pc[0], pc[1], 1]),
                                     np.array([pd2, pd[0], pd[1], 1])]))

        c = np.linalg.det(np.array([np.array([pa2, pa[0], pa[1], pa[2]]),
                                    np.array([pb2, pb[0], pb[1], pb[2]]),
                                    np.array([pc2, pc[0], pc[1], pc[2]]),
                                    np.array([pd2, pd[0], pd[1], pd[2]])]))

        circum_r = math.sqrt(math.pow(Dx, 2) + math.pow(Dy, 2) + math.pow(Dz, 2) - 4 * a * c) / (2 * abs(a))
        if circum_r < alpha:
            add_edge(edges, ia, ib)
            add_edge(edges, ib, ic)
            add_edge(edges, ic, id)
            add_edge(edges, id, ia)
            add_edge(edges, ia, ic)
            add_edge(edges, ib, id)
    return edges

## Deprecated
### other tests for alpha quality
#### check if all vertices are enclosed, returns same index as criticality test

In [ ]:
def getVerticesForAlpha(vertices_df, alpha):
    alpha_vertices_df = vertices_df.copy().loc[vertices_df[alpha]<3]
    alpha_vertices_df['subindex'] = range(len(alpha_vertices_df))
    return alpha_vertices_df
def getTrianglesForAlpha(triangles_df, alpha):
    alpha_triangles_df = triangles_df.copy().loc[triangles_df[alpha]==2.0]
    alpha_triangles_df['subindex'] = range(len(alpha_triangles_df))
    return alpha_triangles_df
def getNeighborTriangles(alpha_triangles_df, triangle_index):
    alpha_triangle_vertices = list(alpha_triangles_df.loc[triangle_index][['index1','index2','index3']])
    other_alpha_triangles_df = alpha_triangles_df.drop(triangle_index)
    neighbor_triangles_df = other_alpha_triangles_df.loc[(other_alpha_triangles_df['index1'].isin(alpha_triangle_vertices))|(other_alpha_triangles_df['index2'].isin(alpha_triangle_vertices)|(other_alpha_triangles_df['index3'].isin(alpha_triangle_vertices)))]
    return neighbor_triangles_df

# allEnclosed_d = float([key for key in params_d if params_d[key]['allEnclosed']][0].replace('_','.'))
# allEnclosed_s = float([key for key in params_s if params_s[key]['allEnclosed']][0].replace('_','.'))
# waterTight_d = float([key for key in params_d if 'waterTight' in params_d[key].keys() and params_d[key]['waterTight']][0].replace('_','.'))
# waterTight_s = float([key for key in params_s if 'waterTight' in params_s[key].keys() and params_s[key]['waterTight']][0].replace('_','.'))

In [ ]:
alpha_s_model


#### check watertightness

In [ ]:
# %pip install trimesh
import trimesh
# trimesh.util.attach_to_log()

def getVertexFaceRepresentationForAlpha(alphaObject_df, alpha):
    verts = getVerticesForAlpha(alphaObject_df['vertices'],alpha)[['x','y','z','index','subindex']]
    triangles = getTrianglesForAlpha(alphaObject_df['triangles'],alpha)[['index1','index2','index3','subindex']]
    for i in triangles.index:
        for indexn in ['index1','index2','index3']:
            index_to_find = float(triangles.loc[i, indexn])
            triangles.loc[i, indexn] = int(verts.loc[verts['index']==index_to_find]['subindex'])
    return([np.array(verts[['x','y','z']]), np.array(triangles[['index1','index2','index3']], dtype='int')])
def paramsByAlpha(alphaObject_df, alphaCols):
    result = {}
    for alpha in alphaCols:
#         print('testing '+alpha)
        encloses_all_vertices = np.all(alphaObject_df['vertices'][alpha]<3.0)
        volume = alphaObject_df['volumes'][alpha]
        if encloses_all_vertices:
            verts, triangles = getVertexFaceRepresentationForAlpha(alphaObject_df, alpha)
            mesh = trimesh.Trimesh(vertices=verts, faces=triangles, process=True)
            waterTight = mesh.is_watertight
            euler_characteristic = mesh.euler_number
            result[alpha] = {'allEnclosed':encloses_all_vertices, 'volume': volume, 'waterTight':waterTight,'euler':euler_characteristic}
        else:
            result[alpha] = {'allEnclosed':encloses_all_vertices, 'volume': volume}
    return result

# params_d = paramsByAlpha(alpha_d, alphaCols)
# params_s = paramsByAlpha(alpha_s, alphaCols)

### JCS conversion

In [ ]:
# plt.close('all')
# dfAll3 = pd.read_csv('/Users/phil/Downloads/sundaycheckboth.csv', index_col=0)
# dfAll3_p= dfAll3.filter(regex='philJCSdata.r')
# dfAll3_rD = dfAll3.filter(regex='rob_ddata.r')
# dfAll3_rS = dfAll3.filter(regex='rob_sdata.r')

# dfAll3_p.columns=['Rx','Ry','Rz']
# dfAll3_rD.columns=['Rx','Ry','Rz']
# dfAll3_rS.columns=['Rx','Ry','Rz']


# phil_mat = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])[0:3,0:3]
# rob_tegu_mat = np.array([[0.922104644399, -0.191552602191, -0.336200275687, 0.0], [0.0969171034844, 0.955507396629, -0.278590541905, 0.0], [0.374606593416, 0.224306075666, 0.899642531558, 0.0], [7.24897435154e-17, 1.6897613437e-16, -2.22044604925e-16, 1.0]])[0:3,0:3]
# rob_opossum_mat = np.array([[0.98966482419, -0.0841148709321, 0.116138814565, 0.0], [-0.0345598571996, 0.646116773228, 0.762455724369, 0.0], [-0.13917310096, -0.758589351258, 0.636532044855, 0.0], [7.83488092761e-17, 1.03588051488e-16, -6.66133814775e-16, 1.0]])[0:3,0:3]

# tegu_converted = convertDataBetweenJCS(phil_mat, rob_tegu_mat, dfAll3_rS)
# opossum_converted = convertDataBetweenJCS(phil_mat, rob_opossum_mat, dfAll3_rD)


# fig, ax = plt.subplots(1,3, figsize=(15,5))
# dfAll3_p.plot(style=['r','g','b'], ax=ax[0],title='Target (Phil)')
# tegu_converted.plot(style=['r','g','b'], ax=ax[1],title='Rob Tegu')
# opossum_converted.plot(style=['r','g','b'], ax=ax[2],title='Rob Opossum')

In [ ]:
# ## actual tegu matrix
# df_teguComp = pd.read_csv('/Users/phil/Desktop/rotoscoping/Utility scenes/tegutest.csv', index_col=0)
# df_teguComp_p= df_teguComp.filter(regex='philJCSdata.r')
# df_teguComp_rS = df_teguComp.filter(regex='robJCSdata.r')

# df_teguComp_p.columns=['Rx','Ry','Rz']
# df_teguComp_rS.columns=['Rx','Ry','Rz']


# phil_mat = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])[0:3,0:3]

# tegu_converted = convertDataBetweenJCS(phil_mat, rob_tegu_matrix, df_teguComp_rS)


# fig, ax = plt.subplots(1,2, figsize=(15,5))
# df_teguComp_p.plot(style=['r','g','b'], ax=ax[0],title='Target (Phil)')
# tegu_converted.plot(style=['r','g','b'], ax=ax[1],title='Rob Tegu')


In [ ]:
# def convertDataBetweenJCSreversedCS(destination_matrix, source_matrix, source_df):
#     source_from_dest_matrix = np.matmul(source_matrix, destination_matrix.T)
#     rotations = Rotation.from_euler('XYZ', source_df.loc[:,['Rx','Ry','Rz']].to_numpy(), degrees=True)
#     converted = rotations*Rotation.from_matrix(source_from_dest_matrix)
#     eulers = converted.as_euler('XYZ', degrees=True)  
#     result_df = source_df.copy()
#     result_df.loc[:,['Rx','Ry','Rz']] = eulers
#     return result_df


# ## actual opossum matrix
# df_opossumComp = pd.read_csv('/Users/phil/Desktop/rotoscoping/Utility scenes/opossumtest.csv', index_col=0)
# df_opossumComp_p= df_opossumComp.filter(regex='philJCSdata.r')
# df_opossumComp_rD = df_opossumComp.filter(regex='newRobJaxdata.r')

# rob_opossum_matrix = np.array([[-0.988997647945, -0.0952296421902, -0.113203213768, 0.0], [0.144901479974, -0.777702952339, -0.611703914508, 0.0], [-0.0297861286566, -0.6213770459, 0.782945306754, 0.0], [0.0, -2.22044604925e-16, -8.881784197e-16, 1.0]])[0:3,0:3]


# df_opossumComp_p.columns=['Rx','Ry','Rz']
# df_opossumComp_rD.columns=['Rx','Ry','Rz']


# phil_mat = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])[0:3,0:3]

# opossum_converted = convertDataBetweenJCSreversedCS(phil_mat, rob_opossum_matrix, df_opossumComp_rD)


# fig, ax = plt.subplots(1,2, figsize=(15,5))
# df_opossumComp_p.plot(style=['r','g','b'], ax=ax[0],title='Target (Phil)')
# opossum_converted.plot(style=['r','g','b'], ax=ax[1],title='Rob Opossum')
